In [1]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

class SentenceEmbedding(nn.Module):
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN

    def batch_tokenize(self, batch, start_token, end_token, bash):

        def tokenize(sentence, start_token, end_token):
            if(bash):
                sentence_word_indicies=[]
                for token in bash_tokenizer.encode(sentence.replace(" ", "<space>")).tokens:
                  if(token in self.language_to_index):
                    sentence_word_indicies.append(self.language_to_index[token])
                  else:
                    sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
                if start_token:
                    sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
                if end_token:
                    sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
                for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                    sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            else:
                # sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
                sentence_word_indicies=[]
                for token in list(sentence.split()):
                  if(token in self.language_to_index):
                    sentence_word_indicies.append(self.language_to_index[token])
                  else:
                    sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
                if start_token:
                    sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
                if end_token:
                    sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
                for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                    sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append(tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())

    def forward(self, x, start_token, end_token, bash): # sentence
        x = self.batch_tokenize(x, start_token, end_token, bash)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model,3*d_model)
        self.linear_layer = nn.Linear(d_model,d_model)

    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x.clone()
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x

class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob,
                 num_layers, max_sequence_length, language_to_index,
                 START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token,False)
        x = self.layers(x, self_attention_mask)
        return x

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # in practice, this is the same for both languages...so we can technically combine with normal attention
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask) # We don't need the mask for cross attention, removing in outer function!
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out

class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y

class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token, True)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

class Transformer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, kn_vocab_size, english_to_index, kannada_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, kannada_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, kn_vocab_size)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def forward(self, x, y,
                encoder_self_attention_mask=None, decoder_self_attention_mask=None, decoder_cross_attention_mask=None,
                enc_start_token=False, enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out

In [2]:
!pip install datasets
from datasets import load_dataset
# dataset = load_dataset("AnishJoshi/nl2bash-custom")
dataset = load_dataset("prabhanshubhowal/natural_language_to_linux")
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


data/train.json:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

data/dev.json:   0%|          | 0.00/442k [00:00<?, ?B/s]

data/test.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2307 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2286 [00:00<?, ? examples/s]

In [3]:
# Updated tokenizer to deal spapces
bash_code_data=train_data["bash_code"]+validation_data["bash_code"]+test_data["bash_code"]
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from tokenizers.pre_tokenizers import Split

# Initialize a BPE tokenizer
bash_tokenizer = Tokenizer(models.BPE())

# Define a pre-tokenizer to preserve spaces and symbols
bash_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace() #Split(r'([^a-zA-Z0-9])', behavior="isolated")

# Set up a trainer for the tokenizer
trainer = trainers.BpeTrainer(
    vocab_size=13000,
    special_tokens=["<pad>", "<sos>", "<eos>", "<unk>", "<space>"]
)

# Replace spaces with '<space>' before training
bash_code_data = [code.replace(" ", " <space> ") for code in bash_code_data]

# Train the tokenizer
bash_tokenizer.train_from_iterator(bash_code_data, trainer)

# Save the trained tokenizer to disk
bash_tokenizer.save("bpe_tokenizer_bash_code_6l.json")

# Test the tokenizer
sample_code = "find . rm -f"
sample_code = sample_code.replace(" ", " <space> ")  # Replace spaces for testing

encoded = bash_tokenizer.encode(sample_code)
print(f"Tokens: {encoded.tokens}")
print(f"IDs: {encoded.ids}")

# Decode back to text
# decoded = bash_tokenizer.decode(encoded.ids).replace("<space>", " ")
# print(f"Decoded: {decoded}")


Tokens: ['find', '<space>', '.', '<space>', 'rm', '<space>', '-', 'f']
IDs: [155, 4, 18, 4, 189, 4, 17, 74]


In [4]:
# import torch
# from google.colab import drive
# drive.mount('/content/drive')
# load_path = '/content/drive/MyDrive/bpe_tokenizer_bash_code.json'
# bash_tokenizer = Tokenizer.from_file(load_path)
# # transformer.eval()
# print("Tokenizer loaded successfully!")

In [5]:
# Load the tokenizer
# bash_tokenizer = Tokenizer.from_file("bpe_tokenizer_bash_code.json")

encoded = bash_tokenizer.encode(("ls -la /root").replace(" ", " <space> "))
print(encoded.tokens)

['ls', '<space>', '-', 'la', '<space>', '/', 'root']


In [6]:
a = ["find -mtime +2 -mtime -5" ,"find ./ -type f -empty -print0 | xargs -0 rm" ,
     'echo "abcdefg" | fold -w1' ,'ls -d * | grep -v foo | xargs -d "\n" chown -r camsoft',
    'find / ! -newer /tmp/timestamp -exec touch {} \;' ,'find . -name *20120805.gz -exec zcat {} \;' ,
    "find . -name '*.[ch]' | xargs grep -l thing" ,"file -l $(python -c 'import sys; print')",
    "shopt -p globstar" ,"#!/bin/bash '\n'threshold='100''\n' processes=$(ps aux --sort=-%io | awk 'nr<=5{print $11}')'\n'while read -r process; do '\n'io_usage=$(iotop -b -n 1 | grep '$process' | awk '{print $10}') \
    if [ '$io_usage' -gt '$threshold' ]; then '\n'echo 'process $process is performing high disk i/o: $io_usage kb/s (root user)' | mail -s 'root high disk i/o process alert' user@example.com '\n'fi '\n'done <<< '$processes'" ,
   'find . -name ".txt" -exec rm "{}" \;' ,"find . -name .snapshot -prune -o \ " ,"find -mtime +1" ,'yes y | rm -ir dir1 dir2 dir3']

for x in a:
  encoded = bash_tokenizer.encode(x.replace(" ", " <space> "))
  print(encoded.tokens)

['find', '<space>', '-', 'mtime', '<space>', '+', '2', '<space>', '-', 'mtime', '<space>', '-', '5']
['find', '<space>', './', '<space>', '-', 'type', '<space>', 'f', '<space>', '-', 'empty', '<space>', '-', 'print0', '<space>', '|', '<space>', 'xargs', '<space>', '-', '0', '<space>', 'rm']
['echo', '<space>', '"', 'abcdefg', '"', '<space>', '|', '<space>', 'fold', '<space>', '-', 'w1']
['ls', '<space>', '-', 'd', '<space>', '*', '<space>', '|', '<space>', 'grep', '<space>', '-', 'v', '<space>', 'foo', '<space>', '|', '<space>', 'xargs', '<space>', '-', 'd', '<space>', '"', '"', '<space>', 'chown', '<space>', '-', 'r', '<space>', 'c', 'am', 'soft']
['find', '<space>', '/', '<space>', '!', '<space>', '-', 'newer', '<space>', '/', 'tmp', '/', 'timestamp', '<space>', '-', 'exec', '<space>', 'touch', '<space>', '{}', '<space>', '\\;']
['find', '<space>', '.', '<space>', '-', 'name', '<space>', '*', '20120805', '.', 'gz', '<space>', '-', 'exec', '<space>', 'zcat', '<space>', '{}', '<space>'

In [ ]:
# Get the vocabulary from the tokenizer
vocab = bash_tokenizer.get_vocab()

# Create bash_to_index dictionary
bash_to_index = vocab

# Create index_to_bash dictionary by reversing the key-value pairs
index_to_bash = {id: token for token, id in vocab.items()}

# Verify the dictionaries
print("bash_to_index:", bash_to_index)
print("index_to_bash:", index_to_bash)

# Example: Access specific mappings
print("Token for ID 0:", index_to_bash[0])
print("ID for token 'rm':", bash_to_index.get("rm"))

In [8]:
bash_vocabulary = [token for token,id in vocab.items()]

In [9]:
# bash_vocabulary = [ START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
#                       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
#                        ':', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`',
#                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
#                        'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN ]

# nl_vocabulary = [ START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
#                       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
#                       ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`',
#                       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
#                        'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '‘', '’', '“', '”', PADDING_TOKEN, END_TOKEN ]

# print(f"NL Vocabulary: {len(nl_vocabulary)} words")
# print(f"Bash Vocabulary: {len(bash_vocabulary)} words")

from collections import Counter
PADDING_TOKEN = "<pad>"
START_TOKEN = "<sos>"
END_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"
# SPACE_TOKEN = "<space>"
special_tokens = [PADDING_TOKEN, START_TOKEN, END_TOKEN, UNK_TOKEN]

def build_vocabulary(sentences, min_freq=1, special_tokens=None):
    special_tokens = special_tokens or []
    word_counter = Counter(word for sentence in sentences for word in sentence.split())
    vocabulary = {token: idx for idx, token in enumerate(special_tokens)}
    for word, freq in word_counter.items():
        if freq >= min_freq:
            vocabulary[word] = len(vocabulary)
    return vocabulary

nl_to_index = build_vocabulary(train_data["nl_command"]+validation_data["nl_command"]+test_data["nl_command"], min_freq=1, special_tokens=special_tokens)
# bash_to_index = build_vocabulary(train_data["bash_code"]+validation_data["bash_code"]+test_data["bash_code"], min_freq=1, special_tokens=special_tokens)
nl_vocabulary = list(nl_to_index.keys())
# bash_vocabulary = list(bash_to_index.keys())

index_to_nl = {idx:token for token, idx in nl_to_index.items()}
# index_to_bash = {idx:token for token, idx in bash_to_index.items()}

def tokenize(sentences, vocab):
    return [[vocab.get(word, vocab[UNK_TOKEN]) for word in sentence.split()] for sentence in sentences]

def bash_tokenize(sentences, vocab):
    # res=[]
    # for sentence in sentences:
    #   encoded = bash_tokenizer.encode(sentence)
    #   print(encoded.tokens)
    #   res.append(encoded.ids)
    # return res
    return [[vocab.get(word, vocab[UNK_TOKEN]) for word in bash_tokenizer.encode(sentence.replace(" ", " <space> ")).tokens] for sentence in sentences]

train_nl = tokenize(train_data["nl_command"], nl_to_index)
train_bash = bash_tokenize(train_data["bash_code"], bash_to_index)
validation_nl = tokenize(validation_data["nl_command"], nl_to_index)
validation_bash = bash_tokenize(validation_data["bash_code"], bash_to_index)
test_nl = tokenize(test_data["nl_command"], nl_to_index)
test_bash = bash_tokenize(test_data["bash_code"], bash_to_index)

def pad_sequences(sequences, max_len, pad_token):
    return [seq + [pad_token] * (max_len - len(seq)) for seq in sequences]

In [ ]:
print(bash_vocabulary)

In [11]:
dataset['train']["bash_code"][:5]

['find -mtime +2 -mtime -5',
 'tempFile="$(mktemp "${TMPDIR:-/tmp/}$(basename "$0")-XXXXX")"',
 "find . -type f -name '* *'",
 'find . -iname foo -type f',
 'find /etc -size +100k']

In [12]:
print(len((nl_vocabulary)))
print(len((bash_vocabulary)))

12426
12665


In [13]:
index_to_bash = {k:v for k,v in enumerate(bash_vocabulary)}
bash_to_index = {v:k for k,v in enumerate(bash_vocabulary)}
index_to_nl = {k:v for k,v in enumerate(nl_vocabulary)}
nl_to_index = {v:k for k,v in enumerate(nl_vocabulary)}

In [ ]:
print(index_to_bash)

In [15]:
bash_to_index["<space>"]

7188

In [16]:
# Extract sentences from dataset
nl_sentences = dataset['train']["nl_command"]
bash_sentences = dataset['train']["bash_code"]
validation_nl_sentences = dataset['validation']["nl_command"]
validation_bash_sentences = dataset['validation']["bash_code"]
test_nl_sentences = dataset['test']["nl_command"]
test_bash_sentences = dataset['test']["bash_code"]
# TOTAL_SENTENCES = len(nl_sentences)
# nl_sentences = nl_sentences[:TOTAL_SENTENCES]
# bash_sentences = bash_sentences[:TOTAL_SENTENCES]

# -----------------------------------------------------------------------------------------------
# Changed train+validation
nl_sentences=nl_sentences+validation_nl_sentences
bash_sentences=bash_sentences+validation_bash_sentences
# -----------------------------------------------------------------------------------------------

# Preprocess sentences (e.g., remove whitespaces and lowercase nl commands)
nl_sentences = [sentence.strip().lower() for sentence in nl_sentences ]
bash_sentences = [sentence.strip().lower() for sentence in bash_sentences ]
validation_nl_sentences = [sentence.strip().lower() for sentence in validation_nl_sentences ]
validation_bash_sentences = [sentence.strip().lower() for sentence in validation_bash_sentences ]
print(len(nl_sentences))
print(len(bash_sentences))


20664
20664


In [17]:
print(nl_sentences[19657])
print(bash_sentences[19657])

find all the files whose permissions are 777 in the current directory
find . -type f -perm 0777 -print


In [18]:
print(nl_sentences[:10])
print(bash_sentences[:10])

['look for any files that were modified 2-5 days ago', "creates temporary file in a tmpdir folder or /tmp folder if tmpdir doesn`t defined, with file name like current shell name and '-xxxxx'-formatted suffix, and saves created path to the 'tempfile' variable.", 'find all files with space in their names under current directory', 'find regular which case-insensitive name is foo in current directory', 'find all the files under /etc directory which are larger than 100k', 'find all directories under current directory', 'search for the regular/normal file "dateiname" in the entire file system', 'download 10000 web pages "https://www.example.com/page{}.html" 100 at a time where "{}" ranges from 1 to 10000', 'go to previous working directory.', "change the group of all directories (except those with a '.') under current directory tree to a group with the same name as the directory name"]
['find -mtime +2 -mtime -5', 'tempfile="$(mktemp "${tmpdir:-/tmp/}$(basename "$0")-xxxxx")"', "find . -typ

In [19]:
max_nl_length = max(len(s.split()) for s in nl_sentences)
max_bash_length = max(len(s.split()) for s in bash_sentences)
print("Length of longest nl sentence: ", max_nl_length)
print("Length of longest bash sentence: ", max_bash_length)
max_nl_length = max(len(s.split()) for s in validation_nl_sentences)
max_bash_length = max(len(s.split()) for s in validation_bash_sentences)
print("Length of longest nl sentence: ", max_nl_length)
print("Length of longest bash sentence: ", max_bash_length)

Length of longest nl sentence:  56
Length of longest bash sentence:  50
Length of longest nl sentence:  56
Length of longest bash sentence:  39


In [20]:
max_bash_length = max(len(bash_tokenizer.encode(s).tokens) for s in bash_sentences)
print("Length of longest bash sentence: ", max_bash_length)
res=""
arr=[]
max_bash_length = 0
for s in bash_sentences:
  if(len(bash_tokenizer.encode(s.replace(" ", " <space> ")).tokens)>max_bash_length):
    max_bash_length=len(bash_tokenizer.encode(s.replace(" ", " <space> ")).tokens)
    res=s
    arr=bash_tokenizer.encode(s.replace(" ", " <space> ")).tokens

print(res)
print(max_bash_length)
print(arr)
res=""
arr=[]
max_bash_length = 0
for s in validation_bash_sentences:
  if(len(bash_tokenizer.encode(s.replace(" ", " <space> ")).tokens)>max_bash_length):
    max_bash_length=len(bash_tokenizer.encode(s.replace(" ", " <space> ")).tokens)
    res=s
    arr=bash_tokenizer.encode(s.replace(" ", " <space> ")).tokens

print(res)
print(max_bash_length)
print(arr)

Length of longest bash sentence:  142
rsync -av --progress --exclude=*.vob --exclude=*.avi --exclude=*.mkv --exclude=*.ts --exclude=*.mpg --exclude=*.iso --exclude=*ar --exclude=*.vob --exclude=*.bup --exclude=*.cdi --exclude=*.iso --exclude=*.shn --exclude=*.mpg --exclude=*.avi --exclude=*.dat --exclude=*.img --exclude=*.nrg --exclude=*.cdr --exclude=*.bin --exclude=*.mov --exclude=*.goutputs* --exclude=*.flv --exclude=*.mov --exclude=*.m2ts --exclude=*.cdg --exclude=*.ifo --exclude=*.asf --exclude=*.ite /media/2tb\ data/data/music/* /media/wd/network_sync/music/
175
['rsync', '<space>', '-', 'av', '<space>', '--', 'progress', '<space>', '--', 'exclude', '=*.', 'vob', '<space>', '--', 'exclude', '=*.', 'avi', '<space>', '--', 'exclude', '=*.', 'mkv', '<space>', '--', 'exclude', '=*.', 'ts', '<space>', '--', 'exclude', '=*.', 'mpg', '<space>', '--', 'exclude', '=*.', 'iso', '<space>', '--', 'exclude', '=*', 'ar', '<space>', '--', 'exclude', '=*.', 'vob', '<space>', '--', 'exclude', '=*

In [21]:
# max_sequence_length = 256
# def is_valid_tokens(sentence, vocab):
#     for token in bash_tokenizer.encode(sentence).tokens:
#         if token not in vocab:
#             return False
#     return True

# def is_valid_length(sentence, max_sequence_length):
#     return len(sentence.split()) < (max_sequence_length - 1) # need to add the end token so leaving 1 space

# valid_sentence_indicies = []
# for index in range(len(bash_sentences)):
#     bash_sentence, nl_sentence = bash_sentences[index], nl_sentences[index]
#     if is_valid_length(bash_sentence, max_sequence_length) \
#       and is_valid_length(nl_sentence, max_sequence_length) \
#       and is_valid_tokens(bash_sentence, bash_vocabulary):
#         valid_sentence_indicies.append(index)

# print(f"Number of sentences: {len(bash_sentences)}")
# print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

In [22]:
# bash_sentences = [bash_sentences[i] for i in valid_sentence_indicies]
# nl_sentences = [nl_sentences[i] for i in valid_sentence_indicies]

In [23]:
import torch

d_model = 640
batch_size = 32
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 6
max_sequence_length = 192
bash_vocab_size = len(bash_vocabulary)

transformer = Transformer(d_model, ffn_hidden, num_heads,  drop_prob, num_layers, max_sequence_length,
                          bash_vocab_size, nl_to_index, bash_to_index,
                          START_TOKEN, END_TOKEN, PADDING_TOKEN)

In [24]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(12426, 576)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=576, out_features=1728, bias=True)
          (linear_layer): Linear(in_features=576, out_features=576, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=576, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=576, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (attention): MultiHea

In [25]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, nl_sentences, bash_sentences):
        self.nl_sentences = nl_sentences
        self.bash_sentences = bash_sentences

    def __len__(self):
        return len(self.nl_sentences)

    def __getitem__(self, idx):
        return self.nl_sentences[idx], self.bash_sentences[idx]

In [26]:
dataset = TextDataset(nl_sentences, bash_sentences)
validation_dataset = TextDataset(validation_nl_sentences, validation_bash_sentences)

In [27]:
print(len(dataset),dataset[0])

20664 ('look for any files that were modified 2-5 days ago', 'find -mtime +2 -mtime -5')


In [28]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)
validation_loader = DataLoader(validation_dataset, batch_size)
validation_iterator = iter(validation_loader)

In [29]:
# for batch_num, batch in enumerate(iterator):
#     print(batch)
#     if batch_num > 3:
#         break

In [30]:
from torch import nn
criterian = nn.CrossEntropyLoss(ignore_index=bash_to_index[PADDING_TOKEN], reduction='none')

for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [31]:
NEG_INFTY = -1e9
def create_masks(nl_batch, bash_batch):
    num_sentences = len(nl_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      nl_sentence_length, bash_sentence_length = len(nl_batch[idx]), len(bash_batch[idx])
      nl_token_to_padding_mask = np.arange(nl_sentence_length + 1, max_sequence_length)
      bash_token_to_padding_mask = np.arange(bash_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, nl_token_to_padding_mask] = True
      encoder_padding_mask[idx, nl_token_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, bash_token_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, bash_token_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, nl_token_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, bash_token_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [32]:
bash_to_index["<space>"]

7188

In [33]:
a = ["find -mtime +2 -mtime -5" , "find ./ -type f -empty -print0 | xargs -0 rm" , 'echo "abcdefg" | fold -w1' ,
     'ls -d * | grep -v foo | xargs -d "\n" chown -r camsoft' , 'find / ! -newer /tmp/timestamp -exec touch {} \;' ,
     'find . -name *20120805.gz -exec zcat {} \;' , "find . -name '*.[ch]' | xargs grep -l thing" ,"file -l $(python -c 'import sys; print')",
     "shopt -p globstar" , 'find . -name ".txt" -exec rm "{}" \;' ,
     "find . -name .snapshot -prune -o \ " ,"find -mtime +1" ,'yes y | rm -ir dir1 dir2 dir3']

for x in a:
  encoded = bash_tokenizer.encode(x.replace(" ", " <space> "))
  print(encoded.tokens)

['find', '<space>', '-', 'mtime', '<space>', '+', '2', '<space>', '-', 'mtime', '<space>', '-', '5']
['find', '<space>', './', '<space>', '-', 'type', '<space>', 'f', '<space>', '-', 'empty', '<space>', '-', 'print0', '<space>', '|', '<space>', 'xargs', '<space>', '-', '0', '<space>', 'rm']
['echo', '<space>', '"', 'abcdefg', '"', '<space>', '|', '<space>', 'fold', '<space>', '-', 'w1']
['ls', '<space>', '-', 'd', '<space>', '*', '<space>', '|', '<space>', 'grep', '<space>', '-', 'v', '<space>', 'foo', '<space>', '|', '<space>', 'xargs', '<space>', '-', 'd', '<space>', '"', '"', '<space>', 'chown', '<space>', '-', 'r', '<space>', 'c', 'am', 'soft']
['find', '<space>', '/', '<space>', '!', '<space>', '-', 'newer', '<space>', '/', 'tmp', '/', 'timestamp', '<space>', '-', 'exec', '<space>', 'touch', '<space>', '{}', '<space>', '\\;']
['find', '<space>', '.', '<space>', '-', 'name', '<space>', '*', '20120805', '.', 'gz', '<space>', '-', 'exec', '<space>', 'zcat', '<space>', '{}', '<space>'

In [ ]:
transformer.to(device)
total_loss = 0
num_epochs = 15

for epoch in range(num_epochs):
    transformer.train()
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        nl_batch, bash_batch = batch
        # print(nl_batch)
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(nl_batch, bash_batch)
        optim.zero_grad()
        bash_predictions = transformer(nl_batch, bash_batch, encoder_self_attention_mask.to(device), decoder_self_attention_mask.to(device), decoder_cross_attention_mask.to(device),
                                     enc_start_token=False, enc_end_token=False, dec_start_token=True, dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(bash_batch, start_token=False, end_token=True, bash=True)
        loss = criterian(
            bash_predictions.view(-1, bash_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == bash_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"NL: {nl_batch[0]}")
            print(f"Bash Provided : {bash_batch[0]}")
            bash_sentence_predicted = torch.argmax(bash_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in bash_sentence_predicted:
                if idx == bash_to_index[END_TOKEN]:
                  break
                if idx == bash_to_index["<space>"]:
                  predicted_sentence += " "
                  continue
                predicted_sentence += index_to_bash[idx.item()]
            print(f"Bash Predicted: {predicted_sentence}")
            print("---------------------------------------------------------------------------------------------------------------------------------")

In [60]:
import os
import torch

# Define the save path in Kaggle's working directory
save_path = '/kaggle/working/tranformer_6l.pth'

# Save your tokenizer
torch.save(transformer, save_path)

# Confirm the save location
print(f"Model saved to {save_path}")

Model saved to /kaggle/working/tranformer_6l.pth


In [61]:
transformer.eval()
def translate(nl_sentence):
  nl_sentence = (nl_sentence,)
  bash_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(nl_sentence, bash_sentence)
    predictions = transformer(nl_sentence,
                              bash_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_bash[next_token_index]
    if next_token == END_TOKEN:
      break
    if next_token == "<space>":
      bash_sentence = (bash_sentence[0] +" ", )
      continue
    bash_sentence = (bash_sentence[0] + next_token, )
  return bash_sentence[0]

In [62]:
# !pip install pydrive
# import torch

In [63]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# # Authenticate
# gauth = GoogleAuth()
# # gauth.LocalWebserverAuth()  # This will prompt authentication
# drive = GoogleDrive(gauth)

# # Save the model locally
# local_save_path = '/kaggle/working/transformer_model_6l.pth'
# torch.save(transformer, local_save_path)

# # Upload to Google Drive
# gfile = drive.CreateFile({'title': 'transformer_model_6l.pth'})
# gfile.SetContentFile(local_save_path)
# gfile.Upload()
# print("Model uploaded to Google Drive!")


In [64]:
# from google.colab import drive
# drive.mount('/content/drive')
# save_path = '/content/drive/MyDrive/bpe_bash_tokenizer_6l.pth'
# torch.save(bash_tokenizer, save_path)
# print(f"Model saved to {save_path}")

In [65]:
# import torch
# from google.colab import drive
# drive.mount('/content/drive')
# load_path = '/content/drive/MyDrive/transformer_model_6l.pth'
# transformer = torch.load(load_path)
# transformer.eval()
# print("Model loaded successfully!")

In [66]:
# import torch
# from google.colab import drive
# drive.mount('/content/drive')
# load_path = '/content/drive/MyDrive/bpe_bash_tokenizer_6l.pth'
# bash_tokenizer = torch.load(load_path)
# # transformer.eval()
# print("Model loaded successfully!")

In [67]:
translation = translate("find all java files starting with 'a'")
print(translation)

find . -type f -name "*.php"


In [ ]:
generated=[]
for i in range(0,len(test_nl_sentences)):
  generated.append(translate(test_nl_sentences[i]).strip())

In [69]:
for i in range(0,len(test_nl_sentences)):
  print("--------------------------------------------------------------------------------------------------------------------------------------------------------")
  print("Prompt    :: ",test_nl_sentences[i])
  print("Provided  :: ",test_bash_sentences[i])
  print("Generated :: ",generated[i])

--------------------------------------------------------------------------------------------------------------------------------------------------------
Prompt    ::  change the ownership of all regular/normal files in the current directory
Provided  ::  find . -type f | xargs chown username
Generated ::  find . -type f -print0 | xargs -0 chown username
--------------------------------------------------------------------------------------------------------------------------------------------------------
Prompt    ::  Write "2-1.1.1" to standard output and to file "/sys/bus/usb/drivers/usb/unbind"
Provided  ::  echo '2-1.1.1'|sudo tee /sys/bus/usb/drivers/usb/unbind
Generated ::  echo "au' | sudo tee /sys/bus/usb/devices/usb/usb3/power/level
--------------------------------------------------------------------------------------------------------------------------------------------------------
Prompt    ::  list all aliases that name contents the string "xlogs"
Provided  ::  alias | grep 

In [70]:
# for i in range(500,1000):
#   print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#   print("Prompt    :: ",test_nl_sentences[i])
#   print("Provided  :: ",test_bash_sentences[i])
#   print("Generated :: ",generated[i])

In [71]:
# for i in range(1000,1500):
#   print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#   print("Prompt    :: ",test_nl_sentences[i])
#   print("Provided  :: ",test_bash_sentences[i])
#   print("Generated :: ",generated[i])

In [72]:
# for i in range(1500,2000):
#   print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#   print("Prompt    :: ",test_nl_sentences[i])
#   print("Provided  :: ",test_bash_sentences[i])
#   print("Generated :: ",generated[i])

In [73]:
# for i in range(2000,len(test_nl_sentences)):
#   print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
#   print("Prompt    :: ",test_nl_sentences[i])
#   print("Provided  :: ",test_bash_sentences[i])
#   print("Generated :: ",generated[i])

In [74]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = [sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(test_bash_sentences, generated)]

average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score: ",average_bleu_score)

Average BLEU Score:  0.45116628018233407


In [75]:
!pip install sacrebleu

In [76]:
import sacrebleu
bleu_score = sacrebleu.corpus_bleu(generated, test_bash_sentences)
print(f'BLEU score (SacreBLEU): {bleu_score.score}')


BLEU score (SacreBLEU): 1.2227969686201814


In [77]:
def exact_match_score(provided, generated):
    if len(provided) != len(generated):
        raise ValueError("Both arrays must have the same length.")
    exact_matches = sum([p == g for p, g in zip(test_bash_sentences, generated)])
    exact_match_metric = (exact_matches / len(provided)) * 100
    return exact_match_metric

score = exact_match_score(test_bash_sentences, generated)
print(f"Exact Match Metric: {score:.2f}%")


Exact Match Metric: 8.75%


In [78]:
!pip install nltk
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [79]:
import warnings
warnings.filterwarnings("ignore")

In [80]:
provided = [[x] for x in test_bash_sentences]
# print(provided)


In [81]:
# print(generated)

In [82]:
from nltk.translate.bleu_score import sentence_bleu
weights = (0.25, 0.25, 0.5, 0)

score = sentence_bleu(provided, generated, weights=weights)
print(score)

0.5531391357804657


In [83]:
print(bash_vocabulary)

['li', 'ser', "#^./##'", ".\\*'", 'MAPPER', ']>', 'FIELD', '125', 'FOO', 'your_file', '(?=', 'myProcessName', 'ffix', 'ors', 'ecked', 'zo', 'vars', 'link1', 'Or', 'GIN', 'dir_a', 'ook', 'LC_MESSA', 'newline', 'each', 'geoiplookup', '/\'$//;"`', ',$!', 'olddir', 'UU', 'thisHOSTNAME', 'имя_файла', 'mygz', '"$.', 'FILES_TO_', 'SU', 'ging', 'AT', 'architecture', '$$`', 'delf', 'checkold', 'project', '=$/$', 'myvolume', "=$($)'", 'gravity', 'cla', 'c02', 'shots', '=*/', '/<', 'aeiou', '929', '4444', 'ez', 'markdown', ']\\+\\)).*/\\', 'NID', 'stefan', 'u2', '="=";}{', '3q', 'my_iso_file', '70', 'strict', '".[', 'colocatedserver', 'rUS', 'Ssh', 'rn1', 'used', 'am', 'HTM', 'order', 'escape', 'Chapter1', 'co', '\'*\'"$*"\'*\'', 'copies', 'apl', ':]]*/', '++`', 'hometest', 'failglob', '864', 'cour', '}".*', 'sha1sum', 'doing', 'MAT', '666', 'xml', 'ware', 'STRING_TO_', 'sionName', 'ew', 'bootstrap', 'BUFSIZ', '^-', "/\\//'|", "/%%'", 'LST', "/]'", 'mand', 'sle', 'PATH2', '-\\*\\.', '%%\\', 'huzz

In [84]:
# bash_vocabulary=['stu', 'new_', '_prog', 'students', 'title', '[^', 'ane', 'wont', 'rectory', 'TF', '|.*/\\', '[^=]*=,', 'tadata', 'IMAG1806', 'SOCKE', '>>/', 'rsp', 'webtatic', 'To', 'list_part', '}\'"', 'MONTH', 'nrk7', 'sh', 'same', '{}-', ')$/', 'mage', 'XPID', 'mode', 'somegroup', 'lib64', 'Tr', 'Lo', 'inal', '+{', '//;', 'ttySome', 'ML', 'BEGIN', 'riend', 'searches', 'MAN', 'wbur', 'confirm', '0a', 'qv', 'abcdefg', 'my_test_ex', 'prin', 'was', 'foldertoexclude', 'ddress', 'symdir', ']")', '>"${', 'ther', "'*\\.", '31mHello', '...|', 'addition', 'admin', 'jfs', "'-", 'b928', 'Sr', 'founddata', "'|'", 'target_dir', 'x270', 'backingfile', 'H', '20M', 'fpt', 'ftpuser', '24p', 'lot', 'rtime', 'SO', 'printenv', 'type_de', '/^>\\', '">\\|<"', 'bod', 'history', 'avz', 'dence', 'MLIN', '"*.*"', 'La', "'/---/,", 'EK', 'sampledomain', 'disc', 'LGT_TEMP_FILE', 'ssh_host_ecdsa_key', '#|\\', 'items', '_del', 'l40', 'test_ex', '1f', 'I_Custom', 'tic', 'RETENTION', ';`', 'tz', '2048', "'^\\'", 'mix', 'volume', '(?=', 'ns1', 'my_script', '>).*(?=</', 'unifor', 'kov', '\\]\\', 'lv', 'Set', 'sublime2', '/&&!/', 'gp', 'IR', 'procs', '"../', '16224', '(:', 'zero', 'created', 'modules', 'afile', 'strace', 'av', 'ust', 'docker', 'm3u', '=*/', "'<?", '401', ']>', '_FROM_LOCKFILE', 'scala', 'ties', 'cken', '‘.*’', 'coppies', '/^\\|:', '_hal', '":")', 'file001', 'AT', "~$//'", 'tops', 'jk', 'your_user_name', '?).*', 'lec', '#<', 'lab', 'L', 'Desktop', 'restart', 'bravo', 'warwick', 'def', 'some_pattern', 'remote_host', 'some_file', "'\\./", 'colorconvert', 'origlink', 'echo', 'S1A', 'xOf', 'PacketLoss', 'pattern_name', '0bee89b07a248e27c83fc3d5951213c1', '|&', 'cmin', 'directory2', '314167125', '*\\/\\', 'sz', '".*\\."', 'fbi', '"$:$"', '`;[', '\\(', 'BJECT', '{\\}', 'D10', '$)-${', 'CurrentLocalMachine', 'newBaseDir', 'copy_dir', 'mT', 'icanhazcheezburger', 'linkname', 'dok', "/(.*)//'", '1337', 'FOO_NO_EXTERNAL', 'these', 'is', '}\\{', 'bn1', 'Search', 'last', "\\\\\\/'", '/#\\', '/\\\\\\', 'P_FROM', 'article', 'ksk', "'./\\(", 'operator', '8888', 'IVE', 'BIT', '/\'$//;"`', 'RECO', "'¤", 'Xj1', 'online', 'OT', 'uvf', '[@', 'BB', '.<', 'g', 'mongo', "'\\''...'\\''", "'($", '*.*/\\', 'put', ':\\', 'kent', 'log4j', '/.*/', 'YourProgramName', 'ded', 'ц', 'kees', 'some_command', 'ans', 'pgodt', '==', ';"', 'keyFile', 'dir_to_exclude', '"*/', 'orapw', 'in2', 'cr', 'S43', 'HSTD', '[^>]*', 'LIST', 'processid', 'tailCo', 'Journalism', 'о', ']\\.\\)*[', '/~/', 'PERSONAL', 'Vd', '/.*\\', 'bar2', "')|", '=\\[`', 'amazonaws', 'TextFor', '{}[', 'prettyjson', "'++", '_ecdsa_key', ',="\\', 'TIMEFORMAT', 'session', 'gento', 'TION', 'NC', 'sma', 'likes', 'unsetvar', 'alienco', 'thefile', "=$'\\", 'YOUR_PASSWORD', ']*[', '500', 'index_disabled', 'name_of_group', 'FU', ')/\\', "'$.==", 'CHUNK_FILE_PREFIX', '.*#$"', 'DAYS_TO_SAVE', '(.*\\)', '14ubuntu1_amd', 'logspace', '12', 'code', '"*$@*"', 'EXT', 'SSH_COMMAND', 'elasticsearch', 'resen', 'DATECOMING', 'dvc', '00', 'rede', 'hu', '777', ']+[.', 'vp', 'naspath', '/^.*\\.*$/\\', 'bi', 'mogr', 'jdk1', 'VER', '*\\\\', '&`"', 'umbs', '/.*[', '})/', 'TYPE', '/*/@*"', 'dc', '.*~$\\|^', '300', 'nullmaile', 'accès', 'suppress', 'wef4fwef54te', 'eval', 'query', 'file_list', 'Ril', 'ffff', "+$//'", 'fink', 'NEW', '7zip', 'anaconda', 'S44', 'ai', 'BG', '"^\\', 'mov', 'pipefail', 'cygdrive', '7n', 'Min', ':~/\\.', 'RAW_LOG_DIR', 'aws', 'DirectiveFiles', 'import_script_', ']*\\)$~\\', 'pas', "\\./,,'", ')\\"', 'linkedin', '/.*/\\”&\\”/’|', "'+%", 'lockfile', 'buff', 'hn', 'Ses', 'UM', '\\{}.', '[/]?.*"', 'values', '\\)\\(.*\\)', 'REMOTE_PATH', 'punct', 'DataCopy', 'iframe', 'current_directory', 'PDFs', '="$', '[[', 'ag', 'foo_', '",$;', 'Do', 'arguments', 'the_', 'tst', 'k1n', 'dirsonly', 'bill', '%-', 'ghi', 'Bash', 'ular', "'/\\/\\\\./", 'pdf', 'suepe', 'phil', 'ccze', 'V', 'django', 'GOR', 'S_TO_', '".*$@*"', 'eth4', 'Here', 'fore', 'yourfilterprogram', 'private', 'setup', '\\}$/\\', '379', 'runIn', 'arr', '}/${', 'WHO', 'ktop', 'Xx', '\'")', 'LOGFILE', 'mynewplaylist', ',.', 'containing', 'myUS', '8631', 'YOUR_USER', 'cook', 'YourPrgram', 'Dw32', 'upper', 'common_', '$@`)', 'memu', 'Con', 'linkin', 'ity', 'CHUN', 'ordinary_file', 'vim', 'END_ABS', 'eof', 'john_do', 'LR', 'Log', 'WE', 'Movies', 'ENTERED_PASSWORD', 'apsho', 'lain', 'nbytes', '})-([', 'aaa', 'AS_', '"/*', 'WD_Disk', "'$!", '/\\([', 'your_script', 'my_tmp_dir', '前', 'ID', "/\\/[^/]*$//'", '="$$"', '_to_remove', '′', '../../../../', '|/', 'l100000', '\\-]{', "'!'", 'X_', 'stl12', 'RETENT', '=/\\', 'project_dir', 'Bp', '\\)?\\|', "-*'", "*//'", 'fasta', '};', 'report_P', 'mjson', 'vote', '$#;', '16187430', ']*/\\', 'wh', 'bsdtime', "{}`'", '‘', '\\]")', 'anm', '_ud', ']*.', 'sess_', 'theDirname', 'in_save', 'processme', 'c_', '/+/', 'venet', '"<--', 'stor', 'bash', ".*/@@'", 'AN', '~$|\\.', 'evidence', 'COMMIT_EDITMSG', '1278999', 'boostinst', 'SCRIPT_NAME', 'arsign', 'ler', 'mylog', 'tile', 'ress', 'exp', 'JBOSS_LOG_DIR', 'mynetaddr', 'That', 'subs', 'my_pattern', "$'", 'ping', 'fileB', "\\''", ")//'", 'mz', 'iTMSTrans', 'ya', '_replaced', 'djvu', "]\\+='/", '++', 'gawk', 'not_e', 'time', '-"}{', '(.[', '/^\\(.*\\)\\/.*\\', 'doctor', 'MNf', 'core', 'lite', ')`', 'headdump', 'var1', 'dir_list_2', 'runProgram', 'grc', 'CONSOLE', 'WID', 'mouse', 'tp', 'Anone', 'edited_blah', '3_tn1_tn2', 'dummyvalue', ':*/', ':]+', 'mask', 'rmc', 'Directory', '$((', 'overflow', 'note', 'googleplus', '56', 'arok', '7125', 'ian', 'calvin', "'.+\\-", ".*'", 'pvf', '::', '\\+\\/\\', 'abc1', 'OPTS', 'LO', 'texi', '="*/"', '25360', 'ddl', 'chavz', '>/"$\'\\', 'old_time', 'vol', ']+%', 'libdir', '3_tn1_t', 'lilo', '$/,"",$', '"^', 'deli', 'empty2', 'oyss', 'uate', 'Dateiname', '\\-', '://', 'firewall', '-#', 'timex', '4444', 'dmesg', 'exlu', '/<', 'icecre', 'CURR_DIR', 'access_token', 'lines', 'somewheree', 'old_file', 'v4', 'auv', 'Authorization', "}')", 'dEOE', '\\$/\\/', 'fixmbox', 'cmd', 'vOFS', '","\\\\', '{};', 'KEBOX', '5g', 'SimpleHTTPServer', 'Eoi', 'MyFile', 'cp1252', "][^/]*$'", '\\+\\(.\\+\\)/\\', 'READ', 'my_share', 'This', 'vent', '`)', 'Free', 'extract', 'lspci', '1k', '111', 'NewDataCopy', 'Stock', 't_', '"^[^;', 'Emp', '9fa', "'*`$", 'my_won', '!:', 'Spam', 'myHostName', 'tldp', 'gems', 'rm100m', '"^$"', '/&', 'noclob', 'xcode', ":]]+$//'", 'oot', 'custlist', 'END_', 'nkey', 'ical', 'TAG', 'dec', 'devn', 'jbos', 'tssh', '+=/^.*.*$/;', 'ifjar', '/\\.\\///', '://$.$/', 'local', 'cd', 'Nov', '"{}")/', 'ando', '-.', 'hoge', 'cnewer', '12000c', '"\\.[^\\.', 'rand', 'LN', 'loader', '3000', 'myServersHostname', "+'%", '=:', './[^/]\\+', 'blu', 'stored_exception_source', 'Fltr', '=>', 'XXXXXXXXXX', '1b', '0000000000', 'ommended', 'chrome', 'zshrc', 'Etvr', 'Jan', '*-\\', 'fss', 'Root', 'orig', '“*/', 'greater', 'PROCES', 'd1', 'somebashstatement', "};'", '¤', 'cz', '==$"', 'softlink_name', 'INTER', '1hsS', 'searched', 'lags', 'tu', 'D_PIDS', '?.*/', "@'", 'L4432', 'stream_get', 'kmstools', 'never', 'read', 'fail2ban', 'ry2', 'rvz', 'j100', '"."', 'lica', 'Number', 'MYGROUP', '@\\', '@|%', 'width', '\\}\\)$/\\', 'npm', 'overallenergy', 'N2', 'SouthEa', 'Test_Version', '&&!/^', '32000c', 'dj', 'WY', ';\\.*;\\', 'dade', 'taku', 'needle', '23', 'fo', 'tsv', 'look', 'snozzber', 'exclude3', '\\.\\."', '$/$', 'PTR', 'ар', 'AA', '[:', 'Theroux', '$\\|.*\\.', 'Ta', 'PHPBB_VERSION', '}\'`"', 'tunnel', ":]]*'`", 'number', '001_', '..)\\', 'erage', 'newuser', 'quit', 'Minimum', '}"/', "\\|$\\)'", "'/^[", ":]+$//'", 'weedly', 'Store', 'bledge_ce', 'jcho', 'MPLE_OP', '<(./', '----------------', 'thday', 'myfiles', 'somefile', 'mount_point', 'fold', 'shred', 'ustar', 'grep1', ":]]'", '_lines', '@$"', 'configuration', 'active_rec', "/'`", 'longline', 'thon', 'sometext', '(|', 'paudit', 'fcgiwrap', '_read', 'c02', 'dummyfile', 'cro', "|./||'", 'NON', 'bigfile', 'overwr', 'pattern_to_INCLUDE', 'dropbox_up', "\\(/.*\\|$\\)'", 'EOI', 'the_modified_copy', '/:\\(.*\\)', 'MAPPER', 'PL', 'blah_server', 'MyDocs', 'LES', 'row', 'fancyhomepage', 'clo', 'stamp', 'аргумент', '\\\\.[', 'pr', '"\\[$', 'n3', '020', 'gnome', 'si', 'somefolder', 'emptydir', 'app_name', "'^$\\|^", ']&&{', 'Fo', 'ten', 'spammer', 'clude', ':]]*$//', 'disablerepo', 'Fx', 'iforni', '20131', 'hobbes', '_save', 'Pro', 'c', 'FOO_NO_TRAIL', '+\\.', 'TI', 'id3v', 'iving', 'csv', 'stralia', 'rcn', '1618', 'romedia', '<<', '/^\\.\\{', '254', 'ual', 'copy_of', 'F0000', 'logdir', 'targ', 'video', '[^/]*\\)/&', '_e', 'proc', '://$(', 'rest', 'jetty', 'noname', '.*\\)', '/**/*', 'Authentication', 'fileToCopy', 'HATE', 'movf', 'EM', 'm_', 'Vi', '1S', "./##'", 'namePrefix', 'sda7', 'em', 'Al', '=', '8ISO', '_dbg', '2006', '6579c', "'{}/", 'LOCATION', 'ont', 'dweet', 'RAN', 'zom', 'ascii', 'brun', 'Bc', '8f1', '929', '://[^/]*/\\).*;\\', 'sources', 'test_file', 'sA', 'Day', 'Input', '(/^', ".*//'", ',}*', 'centos', 'needed', 'fips', '();', 'DS_', 'node_modules', 'SSHSOCKET', '.\\([^', 'prj1', 'vermin', 'P0', '=\\"\\)\\(.*\\)', 'MAVEN_DEBUG_OPTS', '.*;\\', 'ис', 'ameterFile', '/"$', '\\\\\\', '"$\'\\', '/([', '/.*\\./', 'targetfile', '2644', 'reads', 'merge', 'SED_CMD', '_produ', '"&"', 'html_files', 'sure', '8te', 'won', 'LAN', 'livewire', '"[:', '"-', 'ABOR', 'medium', '660', ';:', "/'\\''", 'main', 'src', '="-----------\\', '{}\\;', 'netmask', ']+")', '>\\&', 'where', 'fe', 'cleanup', '$"\';', "'[^\\.]*'", 'allfiles', 'gn', 'inver', 'tgz_', 'pre_existing', 'peg', 'alph', 'still', '\\*\\\\?', 'DataBaseSynchronizerClient', 'SEARCH_PATH', "'\\./$'", 'MYVAR', "'{:", 'aqz', '");', 'GIVEN_DIR', '760', 'search', 'sd_', '#$', 'Work', "'/[^/]+:'", 'perry', 'Cal', '=/', 'FOO_NO_EXTERNAL_SPACE', 'fue', 'remove', 'secondacc', '>\\(.*\\)', 'IT', 'groff', 'gzcat', 'comp', 'tries', 'mysql_access_server', '...*', 'myscreen', 'light', 'unknown', '(/^\\.\\//,"")', '/[[:', 'pf', 'wordpress', '10485', 'Oo', '1__CUSTOM_LINE_M', 'filename2', '412', 'text1', 'userAgent', 'Transfer', '_DEBUG_OPTS', ';!(', 'LC_COLLATE', 'SSH_CLIENT', ':;,;', '919', 'yesterday', '\\|^', 'DataFile_', 'excludeddir', 'denti', 'NEW_', 'string1', 'nogroup', '}\\.){', 'ion', 'lsusb', 'extX', 'j4', 'ure', 'all_html', 'hosan', '##*/};', 'der', 'ADDR', 'x4e00', 'version2', '39', '/(.*)$/', 'newhost', '\\(.', 'defin', 'special', 'gl', 'sda', 'Cp', 'vsan', ';', 'ad', '$))', 'bor', 'nPID', 'dot', 'T_COMMAND', '_NUMB', 'xvzf', 'Dis', 'queror', '15sec', 'р', '0F', 'GO', 'plus', '//\\', '1__CUSTOM_LINE_MARKER', 'JBOSS_CONSOLE_LOG', 'Wi', '-??', 'KEEP', 'sub', 'pretty', '44', 'cur', 'czv', 'break', 'OLLATE', '"/$"', 'abc', '"^{}$"', 'amdcccle', 'fname', 'vob', 'push', '"\\$', 'prefs', 'urlencode', '"~/', "+/'", 'porter', '=($)', "'\\<", 'tomatical', ':]]*$/{++', '_FROM', "'\\.$'", 'xpi', 'date', '004', '}/";', 'adm', 'ception', 'HIGHMEM', 'dirToExclude', 'lh', 'memb', 'Thumbs', '||', 'bz', 'oxm', ':]+"', 'admin_fla', 'rTested', 'OLD_DB_NAME', 'cnt', 'Part', 'dir_context', 'or', '“*.[', 'Continue', 'w79', 'lazy', 'yourRemoteScript', "'.#*'", 'punbb', 'practice', '_search', 'autocd', '/,/@', '.*\\.', 'mong', 'ostat', '("$', 't', '>&-', '0sbla', 'gnuwin32', 'setaf', '\\$!', 'LogLevel', 'pyo', '\\$', 'PAST_RC_files', 'nginx', 'clj', '4nr', 'sav', 'pXXXXX', 'rav', "'^[^;]*'", 'pany', "}'|", '],', 'wagoneer', 'tif', 'atime', '##', 'empty_dir', '-<', 'sic', '_too_any', 'PATH_TO_', 'vara', 'lastdir', 'motd', '~/$', 'otemp', 'teacher', 'DIRECTORY_TO_PROCESS', '++)', ']\\)', '”/’', ']\\+[', '".*/\\.', 'PER_', 'command_cache', '/.*$/&<', 'MDSImporte', 'who', '_SUB', '8h', "'+'", 'surfboard', 'bd', '/`"', '\\-]\\{', 'stat', "'.*\\.\\'", 'log', 'gitignore', 'Template', '?$', 'geny', 'buffered', 'WEB', 'download', '29', 'ROOT_DIR', '20140226', '-@', 'ctime', '$//"', '}.*', 'destination_path', ';)', 't1_', '!.],}*', 'list_1', 'OfNumber', 'AME', 'SION', 'sudo', 'COMMAND', 'Series', "'[^", 'OP', 'axu', '="$/$"', '‘{}', 'ond', 'php_admin_fla', 'nawk', 'Little_Commas', 'R1', 'PASSWORD', 'Z', '1rc', 'Tools', 'somelink', 'cle', 'PWD', 'ant', 'files_and', '4096', '"$*"', '="./', "/\\/$//'", 'Lawler', 'BYTES', 'jpe', 'lski', "'.'", 'TO_IGNORE', 'unexpected', 'pies', ',</*', ')[$', 'OUR', 'v1', 'FOLDER', 'SU', '[?].', 'Mm', 'common_values', 'imagin', '[^:', '):$(', 'qs', 'filecopy', 'rentLo', "'^..$')", '].*$/\\', 'out', 'eso', '".[', 'printdi', 'MAX_', 'files_to_skip', '".*\\.(', 'utub', 'part0', 'greppattern', 'he', 'dry', 'Config', 'GS', 'FinalDate', '"*")', 'cda', ']*\\([^(]*\\).*/\\', 'catalina', 'MF1pp', 'OS', '"/,/"', 'ses', 'MD5', '0F0', '"$/..";', '9999', 'imum', 'FILE_PATH', "'.*\\(", '2571', 'bla', 'mysql', "'\\.[^\\.]+$'", 'what_produ', 'OUTPUT', 'CURRENT', 'files_and_folders', 'mysession', 'fileA_', 'progress', '\\"\\${', "+')", 'zn', 'j', 'litt', 'leila', 'histgrep', 'able', '1M', 'ttySomething', 'Agent', 'anna', ']*\\)$', 'doing', 'flv', '-$.', 'ive', 'applicationX', 'int', 'q1', 'de', 'Fxvf', 'ld', 'bledge_', 'mtnde', 'RAID', 'ew', ':","', 'nu', 'otn', 'oo', 'ej', '/\\=/;', 'Hit', 'gd', 'com', 'f13', 'dia', 'GM', 'plsql', '@:', '(\\"$\\",', '\\./.*', '"\\.[^\\.]+$"', 'playlist', 'а', 'ing_msg', 'les', 'Stories', 'derful_sed_script', 'ffix', 'spaces', 'ures', 'NEWFILE', '286', 'jbd', 'logfile', ",,'", ':]]*/', '02x', '`/../', '\\(\\.[', 'github', 'question', '^-', 'Labs', 'czvf', 'keep', 'x', 'come', 'dd', 'middleuser', 'front_element', ')==$(', 'launchctl', 'ronizerClient', 'ret', 'driver', 'alaxy02', 'epub', 'qui', ':]{', '"}]"', '"*\\', 'ssh_server', '0660', 'UUUUUUUUU', 'HOME', '/(\\', '*(?', 'DAYS', "'\\!'", 'job_XXXXXXXX', 'raid', 'app', 'TemporaryItems', 'Hs', '"????????????????????????????????', '=\\\\', 'log_dir', 'dir', '$"\'', 'DEV_', "'.*(?=", 'et', '_fn', "\\*'", 'dir_', 'your_command_', './$', '\\(-', '...]', '/^\\', 'hadoop', 'one', '.}', 'у', '{,', 'Cache', '_path_names', '\\/].*$/\\', '821', '})', 'USR', 'somehost', 'ALLRECOR', 'PS1', 'compile', 'zdj', 'op', "'('", 'trac', 'Target', 'TMP_ORI', '644', '$,\\', '{"', 'farm', '=","}$', 'dl', '\\:\\', 'Thero', 'HISTTIMEFORMAT', 'csxb', 'tmux_version', 'iTunesArtwork', 'Ham', '*~', '_Rec', 'd_aliases', 'SimpleHTTP', '\\".*', 'KMST_DataFile_', 'upvo', '/.', 'Login', '+)\\.', 'baumerf', 'Big', '\'$\'"', 'linux', 'spec', 'bad_string', 'mypid', 'tymac', 'boot', '033', '_sca', 'symbolic', 'Sm', 'jfs2', 'y17', 'nAs', '$|.', 'PENDING', 'Au', 'located', 'prepended', 'bro', 'pool', '\\}\\)\\([', 'white', 'MAX_LINES_PER_', 'EBML', 'my_group', 'iTunes', '/$-$', 'testdir', 'nk1', '|./', "...'\\''", 'Tk', '0bee89', 'thir', 'fil', '=.]");', '_sd', "/:.*$//'", 'xyzz', 'IdentitiesOnly', '-[\\', 'Lib', 'DayOfWee', 'img', 'Updated', '3fc3d59', 'Fxvz', '‘*/', '}\\(\\:[:', 'file_changed', 'dstfolder', 'Ho', 'mydata', '/*.{', 'leng', 'arg0', 'RDBMS', 'evi', '.[^.]*', '258', "!.]*'", ':]]\')"', '"",', 'ever', 'soc_', '14', 'jmT', 'Man', 'TES', 'COMMANDS', 'LISTEN', '/[^:]*$!!', 'q', 'mice', 'byte', 'alia', 'CustTmp', '>]*', "'[", 'absolute', 'myVar', 'script_7zip', '&', 'AH', 'format', 'IOUY', '/?', 'replacetext', ':]]+\\.', '!(*@)', 'dref', 'mypipe', '4t5teer', 'ToFind', '":"[^"]*"', 'zip2', '12np', 'gin', '_backup', ')))', 'contentstore', 'shm', 'T', 'pogtEtvr', 'ie', '2073', 'uration', 'i_still_rememb', 'before', 'jsbuild', 'ME', 'compres', '_CUST', 'cmdline', '1988884', 'WD', 'acl', 'prefered', 'file_a', 'source_host', 'a248e27c83fc3d59', 'dami', '_cmd', '$[', 'ns', '][`', 'convert', 'bo', 'bad', ']+[.]+[', ',/"', 'nonsen', 'Ch', '/,', 'bluher', 'while', 'ScriptAlias', 'kpt', 'nocasematch', '$/&', '/’', 'OB', 'keys', "',,\\", 'concat', '200557600', '..$#', 'UTE', 'redhat', '"?????????????????????????????????"', '\\.]){', 'pec', 'esp1', '\'*[+{;"\\\\=?~', '%.*', 'direx', '["', 'id', 'chm', '+"', '\'"[', 'az', 'join', 'uteDate', 'GUI', "'/\\/\\", 'pycache', 'ds1337', 'Old', 'dockerroot', 'CLUDE', 'Bpqr', 'mycool', 'WSFY', 'sessions', '71', 'nodisplay', 'mycompcore', 'allica', '"([', 'overwrite', 'view', 'fwef54te', 'n2', 'manage', 'zzber', 'Synch', '$|~$|\\.', 'images', 'fal', 'l5000000', 't1_t2_t3_tn1_tn2', '$\\|^', 'sed', 'INI', ",\\}$//'", '_WI', '$()', 'the_peasant', 'myappdata', 'fed', 'cho', 'hduser', 'foo_bar', 'hobb', 'newThi', '″', 'IMA', 's_12345_moreleter', '`:`', '"&', 'xFE', 'box', '$', 'sorted', 'PE', 'PC', 'Sample_51', 'wZ', '+(?=', 'WebM', 'FIELDWIDTHS', "'^#|^", 'pamvd0', 'it', 'swt', '".*/[', '409', 'P_', "/.*\\///'", 'shared', 'ispell', 'insa', '03d', '3u', 'T_OPTS', 'virtualenvwrapper', 'apache2', '_FILES', '<\\?', 'wzorzec', 'sample', 'originals', 'Compress', 'inst', 'sen', 'ream', 'took', 'nel', '/{/.}.', 'once', '"./[^/', '\\+/', 'tab', 'xB', 'Sc', 'ment', 'lime', 'goo', '**...**', 'mit', ']*?', '840', 'Of', "...}'", 'l0', './<', 'tobeexcluded', "()}'", 'cust', 'computer', 'compressFileName', 'multi', 'feed', '/\\<$\\>/"$\'\\', 'somedata', "'`", "'.*\\(\\(", 'T_DIR', '}:', 'Empty', ']*)', 'friend', 'stackoverflow', ').$(', 'ra', '_m', '\'}"', 'rt', '/^.*.', 'iconv', 'usb3', 'Deve', 'vprvt', 'R6', "'/;;", 'к', 'COLUMN', 'big', 'USERTAB', 'person', 'work', 'ata', 'INCLUDE', 'USERS', '900M', 'chan', 'nail', '551', 'DISPL', '_dir', 'fileName', 'KR', 'someletters_12345_moreleters', '_private', 'destinationFolder', '*.*”', 'ile', 'CDC', 'Namese', "'.\\|", '"$/$.', '_token', '[[:', '18582', 'tice', '..\\{', '9A', 'channels', 'level1', 'msw', 'prn', 'foobar', '_tcp', 'APP', 'wildcards', '".*/\\(', 'rub', '2g', 'rev', 'apter', 'mrg', 'content_dir2', 'rst', 'Memory', '0002', 'urandom', 'magic', 'NOPORTDOCS', 'anity', ':.*//.*:\\', 'leclick', 'thegeek', 'gui', 'search_terms', 'rpm2cpio', '\\)\\(.....\\)/\\', 'il', '<&', '/\\\\..*/\\', 'mux', 'launchd', 'JSON', 'joomla', 'icecream', 'line_to_be_replaced', 'alina', 'my_new_folder', 'Remote', '{}`', 'go', 'Prefix', 'rvf', '(.*));', '$!\\', '"==>', 'errex', 'PRE', 'compaudit', "'/", 'en', 'front', ':\\([', 'myfolder', 'mplayer', 'processname', 'AST_', 'char', '/!', 'mydir', '\\*\\.', '-------";}\'', '17', 'n_jobs', 'stuff1', 'monthly', '.]\\+\\).*$/\\', '-.*', 'releases', 'NEWUSER', 'haystack', '751', 'Shel', 'Djava', 'LINENUM', 'cmd2', 'pubs', 'sorce_dir', 'ize', 'joebob', 'n', '#\\/', '015', '20131220', 'yyyy', 'Vim', '$!!', '{}\\"', 'names', 'i_still', 'date_222days', 'JPGjpg', 'myScript', '(\\./', 'stephencheng', "&'", 'dir_b', 'lu', 'Mainfolder', 'Za', 'blank', 'Enter', 'lR', 'cyg', 'mess', 'Y_TEST_FOLDER', '2121', 'GIVEN', '35', '/(.)', 's_CUSTOM_', 'mybuild', 'erm', 'inser', 'neer', 'mediaimg', 'report', '128', 'gdb', 'libG', ',/,;', 'filesPermissions', 'iTM', '="%', 'ov', 'wick', 'comparring', 'fc22', 'my_large_file_2', '_variable', 'semb', 'SQL', 'alnum', 'public_html', '+`', 'create', 'accts', 'buf', "|\\'", "|$'", 'webm', 'including', 'find_excludes', '/.*\\$/\\', 'xzf', 'DOWNLOAD', "'^\\./", 'tin', 'DIR_DESCARGA', 'had', 'RAMNAME_KER', '(/^\\.', 'lots', 'tracing', 'FOO_NO_WHITESPACE', 'drush', 'struct', '`";', 'upwd', 'DELIMITER', '`\\]\\', 'links', 'andrew', '_FOLDER', 'vek', 'SearchString', 'Hello', 'CO', 'oip', 'ER_OF_DAYS', 'mp3collection', 'wire', 'yaml', '[^\\', ']"\'', 'CONSO', ']*/', 'party', '\\"$', 'themes', 'subdomainA', 'IDENT', ']\\)/', 'BASH_SOURCE', 'nde', 'ql', 'K', 'minutes', '@)', 'Hin', 'filedum', 'findpdf', '222', 'dcard', 'otify', '_graph', 'csvtopretty', 'lists', 'fgrep', 'JBOSS_AS_USER', '$)-$', 'tignore', 'tqdm', 'PORT_NUMB', '"{', '11990929', 'Archive', 'dumpversion', '8f1db2863be3', 'SharedSupport', 'AMP', '"%"', 'vims', '428x270', 'file_pattern_name', '/\\(.*\\)$,\\', 'Pre', 'newname', '11', 'zon', 'ibak', 'file_2', 'ww', 'agg', 'latency', '=\\"', '/{.[', '\\;`', 'orange', "'^\\(", 'david', "'?????????????????????????????????'", ']\\/', 'pls', 'start', 'skype', '=\\', 'TOR', 'Test_Ver', 'd4', 'Output', 'TERNAL', 'part_of_', '_chroot', '1234c', 'daemonXXXXXXX', 'IL', 'ute', '@^', '|$)', '"%', 'p2p', 'small', 'upper2', '7zhelper', '_last', 'SELECT', 'TE', 'source_file', '1100', 'subfolder', 'Views', 'inotifywait', 'HUGEFILE', 'Ff', 'version1', 'batch', "'^.", '1plus', '\\(.\\{', 'B1', 'var2', '-!', 'Files', "'.*~$'", 'SG', 'LINE_M', 'YEAR', 'AD', 'largecalculation', '#&/', 'different', '16186', 'itter', 'timestamps', 'less', 'IZl', ']*(.[', '?}"', '_jk', 'goes', 'quick', '2015', '_tmp', 'xFF', "||'", '"^\\./', 'launcherfile', '="$"', 'YOUR_USERNAME', 'httpdocs', 'secure_private', '1p', 'rdf', '/{.,}*', '$!"', '.*//', '04d', 'gate', 'Makefile', 'foreach', 'йс', 'sp', 'я_ф', 'addnums', 'Californi', '001', 'failglob', '1000000c', 'reverse', 'common', 'inode', 'theDirectory', 'Beispi', '>.*', 'en_US', '$\\', 'bib', '/\\(.*\\)\\/.*$/\\', 'blog', 'inputrc', '/\\.\\/', 'tmpfile', 'INER_ID', 'antom', 'Pool', 'DIR_NAME', 'JAVA_', 'Pp', 'sal', 'timeout', "/\\//'", ':";', '51200', 'May', 'tx1c', 'share', '.\\\\..', 'Contents', 'grep2', 'CONDITION', '—', '"./.', 'letter', 'RED_FILES', '.*<\\/', 'xargstest', 'PNG', 'dns', 'big_lines', 'Image_', 'rite_log', '"*"', 'ues', '94', 'apdisk', 'sho', 'lection', 'TEST', 'Micro', 'your_', 'infolder', 'ult', 'textfiles', '_build5638', 'ие', '5000000b', 'cupcake', 'sshd', 'level', 'htmlfiles', 'ge', 'dailyRe', 'ndekle', 'readlines', '<\\/', ',//,/,', 'BOOT', '_signed', 'amzadm', 'tle', '+$', '_51', 'BEG', 'cronenv', '*]"', 'arg2', 'filetype', '\\.[^.][^.]*', 'backuppath', 'fix', '$$', 'GNU', 'Xnoagen', '2001', 'argument', 'PIDS', 'cAn', '600M', 'spool', 'Unix', 'modulesgar', 'GZIP', 'n500', '‘.*', "'^.*/\\.\\(", '_of_', 'whaty', '}$.', 'repos2', 'full_backup', 'complement', 'expect', ':])', 'qf', 'PAST_', 'es', "]}'|", 'pattern2', 'return', '")-', 'stder', 'libvor', 'a10', '#-', '192', '50', 'pmd0', '>##', 'mine', 'templateName', 'unch', '40', ');/', 'user_filer_project_dir', '$/\\.', '{}`/`', 'uniform', 'sco', '\\".*/$', 'sas', 'lint', 'zxvf', 'drive', 'fla', 'generated', 'hel', '/{', 'T_A', 'cccle', 'topdir', 'QT', '/.*//};', 'rq', '/?(?:.)', 'active', 'javaplugin', 'svnworkspace', 'dir_name', "'.?*'", 'SAFE', '\\",', '$*', '1003', '/*', ">)'", 'first', 'path_to_save', 'cesToFind', 'pg', '\\*\\\\?\\*', 'll', 'domain', "'.+\\.", 'calMa', '/^[^/]*', 'UserKnown', '@/@-@', 'run', 'NG', '890', '╲);', '4000', 'lock3', 'rmv', 'QL', 'fn', 'IS', 'ash', '/(\\).*/\\', 'yum', 'dstdir', '<<<$', 'pc', 'full_backup_dir', 'chiro', 'dayDay', 'ke', 'Mock', 'age', '/$/.', "='%", "../'", '-$()', 'qiF', '".*/', 'FINALPATH', '10485760c', 'WP_', '3816', "'^$\\|^#\\|^\\", 'mplay', '1r', 'ajans', 'drivers', '"|"', '.*$/\\', 'seti', '"~"`', 'compar', ']+[.][', '"}', '.[^', 'DP', '".*"', 'setuid', 'VER_DIR', 'filename_regex', 'instr', '/.*\\(', 'OR', 'workspace', 'AAA', 'Matching', 'tcp', '#---*#\\', 'UPLOT_', 'mirror', 'thumbnail', 'a1a2', 'hiddendir', 'or_directory', 'webupd8team', 'python2', 'nk', 'script_dir', 'username', 'YourFile', 'autoload', 'course', '16085', 'dition', "='$", 'somePath', 'COMMIT_', 'ority', 'databases', 'printf', 'mate', 'rm1g', 'wrap', '03g', 'shuf', '/[]//', '*\\/\\//', "'$'", "'\\.", 'mysite', 'hg', 'ASCII', 'wmv', 'sour', 'lat', '\\/\\)\\(.', '/*`', '%/*', 'B999', 'windowsabc', 'web', '256', 'ele', 'srv', 'libgtest_main', 'gnutar', 'antoniod', 'empty1', '"[', 'JECT', 'slrn', 'Alast', 'myargs', 'hook', 'Cle', "\\>'", 'atch', 'mediaindex', '\'\\\'\')\\"', 'ecriv', 'args', 'cntps', 'CSRC', 'INTRANETDESTINATION', '/,/^}/', 'chapter1', 'XXXXXXXXXXXXXXXX', "]+:'", 'soft', 'incl', 'compat', 'FGID', 'ef', 'LFile', 'IP_M', 'fq', 'LEAS', 'seconda', 'has', '6d', 'users', '/^/"/\'', '\\""', '+(?=%', 'writable', 'Domain', 'Pa', 'loy', 'kata', 'numl', '[(][', 'my_program', '\\}.', 'citrant', 'discover', '置', '200000000', 'SCRIPTPATH', 'Mú', ')*', 'p33', "+'", 'unite', ":='", '_for', 'BBB', 'vzf', 'pe0', 'nobod', 'en1', 'SAVE', 'outfile', 'TARGET_FILE', 'FOO_NO_LEAD_SPACE', 'wordA', 'gg', 'patches', 'pppd', 'etags', 'ravz', 'subversion', 'WARDS', 'D_', 'Var', 'z1', '501', 'ial', 'ASYNC', './@@', '.....', 'NOPORTEXAMP', 'jen', '="#"$', '/^\\([^', '$:$', 'findpdftext', 'Sto', '=($', 'fileloc', '_path', 'Some', 'mint', 'sfn', 'dest_dir', '}{\\\\', 'arjhaiX4', 'compat31', '_hosts', "'^}'", 'version', '##*/}</', '=-', 'venet0', 'MLINK', 'name', '>\\<\\/', 'oy', 'stdout', '2770', 'xjobs', 'performan', "$');", 'ports', '+\\;', 'previous', 'My', 'lower2', '_cmd_', 'newlinks', 'actual_path', '="$-$.', '"$"`', '_sum', 'ecked', '4321', 'ype', 'itunes20140618', 'DAT', '65', 'TMPPS_PREFIX', 'short', 'RBENV_VERSION', 'the_peas', 'locale', 'edly', 'Modified', ']*=,', 'cjf', 'move', '720', 'limesurvey171plus_build5638', 'ADD', '_log', 'full_path_to_your_directory', 'tformat', 'remote', '_prefix', 'er', 'SEARCH', 'FILE_LIST', 'live', 'Ap', 'LastChecked', '\\/\\*\\*', 'line_to_', 'psu', 'calc_space', '}$@', 'fprint', '"*.[', 'colu', '{', '...\\', '([^)]*)', '“.*', 'bb', '/\\<', "'(?<==", "'/\\/", '}.*${', '9M', 'lines_per', 'D44', 'indent', 'QMAKESP', 'sparse', 'repe', '\\}\\([', 'TABLE', 'nPlease', '?"', 'cul', 'resize', 'COUNTRY', '_SITE', 'my_iso', 'wz', 'IPADDRESS', 'c10', ')/$', '--;', 'b22', 'dified', 'msdos', 'sciences', 'style', 'apresentacao', '"%%%%%%%"', 'workspace_ts', 'EOT', 'dfolder', '0b', 'Ban', 'Cnt', 'MIZ', '/\\-\\-', '1220', 's8d', 'CVS', 'fonts', 's1', 'PATTERN', 'stan', 'Str', 'AM', 'Re', '!#$', 'subdomain', ',:]', 'TIMEFOR', '#', 'erClient', 'head', 'Eow', '2011', 'cv', '/^/[', 'yahoo', '>./', 'sprintf', '_build', 'NO', 'MASTER_SITE_SUBDIR', '$/\\', 'xcfg', 'MAX_LINES_PER_CHUNK', 'GIN', 'SRCFOLDER', 'process', 'whoever', '203', '4t5', 'elast', 'nPREFIX', 'tips', ':","");', 'WE8ISO', 'anotherdir', 'postgis', 'mote', 'rename', 'signed', './%', 'tFile', '"', 'al_P', 'ut', 'parts', 'ejH', 'ann', 'hi', 'vier', 'FFF', '*\\', '1000', 'Direcoty', 'myShell', 'jassi', 'afi', 'ntwo', 'washere2', 'ron', 'friends', 'TeSt', "@$:'", 'ors', "'!/^[[:", 'dden', 'sites', 'sas_log_list', '=[^=]*\\)', 'ST_DataFile_', '0L', 'INPUTFILE', 'ground', '/.*(\\).*/\\', 'wiki', 'e2', 'pd0', '\\*.', 'dailyReports', 'dien', ']*\\([^', 'string', 'NCIALS', 'aHvz', '=~', '59', 'MDSI', 'zA', 'subdir', 'Can', '"%.', '1211', 'stats', 'joe', 'inotify', 'Aug', 'RC', '02g', 'pipe', 'monkees', 'MN', "\\'*", 'ACK', 'k1', ',%', '_IT', '\\")")', 'spe', 'mpleGrep', '007', '423', '?).*|\\', 'memo', 'ind', 'gra', '.\\{', 'Pattern', 'So', 'ug', '@$', 'PP', '`\\$', 'large', '_fla', 'pattern_to_exclude', 'PATH2', 'geek', ".*$//'", 'nta', 'iselect', 'product', '\')"', 'rv', '366', 'bero', 'CONTA', 'travel', '_moreleter', '_ports', 'solete', '/^.\\///"', 'file2', '\\\\}', '_load', '.\\/', 'round', 'G_OPTS', 'stack', '102', 'err', 'n1200', '][[:', 'bakup', 'tmp_file_upload', 'real_dir1', 'strings', '="', '\\*~', 'ls_results', 'cached', "='\\[\\", 'ext4', 'there', '\\/.*\\', 'ari', '++$', '+=/^.*(', '/\\(.....', '???"', '&>', '&>>', '$!', 'transport', 'target_tarball', 'TO_', 'clear', 'P_TO', 'my_', 'filter', 'repeated', 'PROCESS', '\\\\?\\|', 'lynx', 'oldest', 'someNamePrefix', 'host1', 'more', 'PHP', 'enter', '31246', 'testFile', 'job_list', 'dmS', 'auve', '}/^;;', 'kf', 'quotemeta', '\'\\\'\')\\""\'', 'www1', 'B5', '\\+\\)/,', 'Timeout', 'upgrade', '3be3', '/\'"\'"\'/\\\'"\'', 'SE', 'x80', '\'*[+{;"\\\\=?~<>&*|$', 'MANIFEST', 'itms', 'fedora', 'constan', 'ze', 'sqlite', '/,/', 'awk', 'UTF', 'gravi', 'sitesearch', 'server2', 'doexe', 'DIR_DESCARG', 'orni', 'FILES_PATH', 'PARENT_PID', '..;/;', 'webhome', '/(.*', '10080', '211028', 'psr', 'coff', 'CKE', 'R_HOST', '"???-???', '}:/', 'DELIM', 'Cd', "?)$'", 'JEC', 'linebufferedstdout', 'Ma', 'OUTPUTDIR', '*."${', 'WORKSTATION_IP', ']*.[', 'rom', '8e27c83fc3d59', 'fst', 'м', 'mysqladmin', '\\/\\.', 'cli', ':[^', 'evgeny', '%%', 'brown', 'limit', '10737418', 'thefoldertoexclude', 'subnet', '/^/+', 'target_group', 'www2', 'boost_1_54', 'PASS', 'dir4', "*'`|", 'testfile', 'ginpath', 'FSET', 'HostKey', 'Total', 'oper', 'myaddr', 'insaneweb', '1G', '[^"|', '"??', 'CUR', 'uk2', 'newSec', 'dir_two', '!', '*#/', 'zerobyte', '/$$/', 'kill', '\\(.*\\)/\\', '+', 'ges', 'html', 'pcx', 'subdir2', 'SUFFIX', '}/', '/.*/"', "'#*#'", 'REX', '3d', 'srch1', '((', '".*$', 'dt_socket', 'File2', 'ORIGIN', 'terminal', 'match4th', 'newerthan', 'order', "'^[[:", 'gallery2', 'RS', 'dumps', 'f1', 'patch', 'taca', 'symlinks', "'.*/", 'file_name', 'deb', "@@'", 'cshrc', '|$).*"', '-$', 'ca', 'fid', 'TIMIZATIONS', 'sharename', 'copy', 'exp_to_find_for_replacement', 'tomcat7', '/(\\(.*\\)', 'NCI', 'someport', 'monit', "'*[+{", 'systemPassword', 'surve', 'man1', '[@]}', 'rdWord', '250', 'fNT', 'zzz', 'text_2014', '12676', '&&(||){', 'xvf', 'monfichier', '/(/\\', "'-!", '1d', ';$', '_of', '18919', 'X2', 'LOGS', 'BAr', '>.*</', '201111301200', '($', ')"),', 'scm', ']*\\.){', '=\\"\\)\\(.*\\)#\\', 'mkcd', 'O', '006', '/--', '18', 'vpl', 'ulti', 'remote2', 'eaDir', '63000', 'ter', 'met', 'MicroMsg', '2400', '"$")"', '\xad-', 'ee_sd', 'isy', 'PPID', 'counter', 'copy_of_', '_disabled', '"");', '+="$(', '+/-', 'JBOSS_AS_', 'named', 'celain', 'gets', 'xdigit', "..$'", '/~/"', '";', '/\\-\\', "?*'", 'ge_', 'maild', 'cowsay', 'sound', '")}\'', 'Foto', 'n_', 'reportuser', "='+$(", 'project1', 'pids', 'sentence', 'ADDRES', 'cdl', '#"$', '_alias', '://[^/]*', 'P1', ':([^', 'ware', 'song', 'pkey', 'xBB', 'filname', '164', '"+$"', 'tmux', ']{', 'hits', 'sses', 'IW', 'die', 'NOT', '»', 'Cannot', '1Ssh', 'mygz', 'rect', 'List', '"[/\\\\]"', 'my_archive', 'lne', 'sel', '_nullglob', 'sfvn', '""}', 'compta', 'BES', 'data_report_PD_import_script_ABF1_6', '1hs', 'f5', '/%"\'', 'parents', 'SOURCE', ']+[.]?[', 'tain', '//$', 'kg', '_to_avoid', 'wma', 'tty', '_log_list', 'blueco', "'^;'", 'jump', 'fwe', 'blue', 'j7', 'Attached', '_ts', 'line1', 'col', 'myenv', 'emlx', '"^${', 'pushd', 'mporte', ',:]//', 'outputprefix', 'SFY', 'SAS', '<space>', 'SCRIP', 'ubuntu', '\\*/.\\*', '"*.[!', '\\+\\)/\\', 'tinue', 'nold', 'jd', 'glob', '","",$', 'john_doe', 'COLUMNS', 'IABLE', 'pass', 'yes', 'certs', '08x', 'Thi', 'L_DIR', '30000', ']+$"', "'./[", '"*.$', 'l', '(][', 'are', 'overwriting', 'en0', '/$/"/\'', '*/*/*/*/*/*/*/*', 'aty', '\\//', '*|$', 'simpleGrepScript', 'srt', 'mypics', 'parent_directory', 'tern1', 'logout', 'build_graph', 'cgi', 'framerate', 'axvw', '10000000', '="="', 'alpha_2', '"/.*"', '\\\\}\\\\\\\\{', 'disk', 'auth', '/.*/&', 'REPLY', ':.*/\\', 'resolver1', 'bh', '++}', 'family', 'ufa', 'access', 'packages', 'jf', '[^:]*', ':\\/\\/', ">/'", 'pl', 'bum', 'ina', 'x86', '7600', '|^.*/', 'locate', '(', '--)', 'ender', 'pdvm', 'HI', 'EL', 'alun', '"$', 'NEF', 'lampp', 'runCommand', 'nsf', 'mysqldump', 'unset', '{}/*', '"){', 'Find', 'previousword', 'vcsupport', "()'", '../../$', 'gnore', '$/', 'rg', ',/', ']+\\', 'editor', 'preserve', 'PS_', 'Sur', 'orzec', 'let', '342', 'userdir', 'privatekeystuffdis88s8dsf8h8hsd8fh8d', '07', 'cx', 'Wal', 'dba', 'myUser', 'FAIL', '5000k', 'MALAWI', '"*/*/', '000k', "'%'", '¨', 'wait_event_interruptible', 'Tfv', "'*-", '@"$', 'UUUU', 'O3', '1pp', 'compilation_level', 'landoflinux', 'oremount', 'on', '"#"*', 'mtndew', 'reg', 'rm2g', 'movi', "]+$'", '1FV', 'filename_', 'place', '/\\/%', 'mod', '*”', 'homes', '";\'', ';\\/.*;\\', '\\*[', '_time', 'GIT_DIR', 'Company', 'Down', 'Folder1', 'Web', ';/^$/!{', '##*.', 'png', 'constants', '_Disk', 'lic', '_script_', '.*"`', 'route', 'fing', 'ado', 'BUP', '"~"', '/..', ')",', 'uptime', '\'"', 'prettify_json', "'.+\\.$'", 'rem_', '.]+)",', 'temp_dir', 'oldSt', '"$"', 'aa1', 'repeat', 'HIGH', 'ien', 'г', 'name1', '-{}', '(.*/\\', 'select', 'elevator', 'tempFile', 'findcommandexamples', "'.*\\(\\\\?\\|", 'mgmt', 'kos', 'daystart', 'base', 'vmail', 'gcal', 'xamp', 'ppid', 'wsay', 'daemon', '928', 'AR', '))', 'PASSWOR', 'bar1', 'pre', 'libxml', 'antigravity', "/'$//;", 'bashplay', 'ip_addresses', 'CLOSED_PORT', 'nocaseglob', 'ASF', '2d', 'yourRemote', 'vimrc', 'deps', 'ypi', '1267619929', 'css', '\\(.*\\\\)$/', 'nomb', '0f', 'okular', 'CS', 'lyIn', 'nx', 'At', "'#*'", '$/,"', 'lr', 'nc', 'x300', 'running', 'office', 'heading', 'FILES_TO_', 'column1', 'де', ':"}', 'errypi', 'pgod', 'ip', '`\\]\\#\\', 'PU', 'color', 'br', 'skip', '=?', 'dof', 'gr', '_IP', 'VARNAME', 'hey', '/.*;\\', '60s', 'KTEMP', '\\..*/\\', 'MEM', '$;', 'Th', 'rhosts', '/\\/\\', '>\\(.*\\)<\\/', 'architect', 'rpm', 'm1', '184', 'will', '};/^\\', 'ukul', 'Name', 'ce_test', 'var', 'xpic', 'foo', ")'}", '_1', 'cdi', '100s', 'vmwarevm', 'inodes', 'root', '/\\(^.*', 'cwdhist', 'leting', 'yo', 'Metallica', 'daisy', 'LEVEL', 'TA', 'epe', 'Ordner', 'port', '/(\\(.*\\)).*/\\', 'documentclass', 'dropbox', '_TodayDay', 'exampleString', 'myusername', 'space', '"*.', 'eld', 'lode', 'Deleting', 'some_dir', '05', 'db', 'dirall', 'StringBuffer', '"./*/.*"', 'fichi', "'/^\\//", 'serv_', '".*$"', 'log_del', 'jce', '-<$', 'StrictHostKey', 'Li', 'vu', 'changeset', 'Mounted', '\\([^', 'biglogfile', 'strap', 'roger', 'exit', 'red', 'ber', 'webroot', 'their', 'vy', 'OAT', 'remotemachine', '"]', ')-$(', 'seta', '_for_replacement', 'rpmfu', 'sid', 'readme', ']/)', 'yourEvilProgram', 'ENTERED_', '/^\\//', "'/\\.", 'washere', 'checkpoint', 'noh', 'porcelain', 'ATE', 'i686', 'Rf', "...'", 'np', '178', 'BINARY', 'JBOSS_USER', 'esxcfg', '~"', 'my_large_file', 'continue', '`/`', 'in1', ";}')", '_new', "/\\[\\]/'", 'cvf', 'fin', 'ZIP', 'lash', 'officefirewall', './*', 'l20', '02d', 'mb', 'copyright', '14ubuntu', 'Pict', 'timezone', 'failed', 'START_DIR', 'JAV', 'mybigfile', 'sourcePath', 'explode', "'*'$", 'remoteServer', "'.*$'", '\\(.*\\(\\..*\\)\\)$/', 'TMPFILE', '_kerberos', '`@`', 'both', 'mat', 'myFileName', '];', '/\\', 'Ko', 'lick', 'xbf', "'.*/[", '1900', 'pbcopy', '=$', 'FATAL', 'KESP', 'share_folder_vm', '{})/', "/%%'", 'an', '9k', "'{}'", '.*/', 'LD_', 'BUFSIZ', 'Dest', '"{}";', 'dir_1', 'bruno', '|<', 'full', 'psi', 'olivier', 'histexpand', '711', 'DIRNAME', 'iable', 'wasn', '({', "'/^#/", 'ls', 'ADVERTISER', 'plied', 'vr', '"]\'', '/*/"', 'traces', ')`"', ':%', "'-------", '".*(', 'fp', 'Chapter', 'EP', 'snozzberries', 'numeric', '_element', 'm4p', '(.*)/\\', '10000k', 'OBJECTDB_HOME', '200000', 'pattern', 'tw', 'lower1', "\\.[^.][^.]*$'", 'dated', '48', 'development', 'some_', 'TXT', 'dmask', '/.$$', '.*\\)\\)$/', 'kernel', 'sr', '="/"', '/:/;', 'conver', '":[^', 'NO_', '{}$"', '=\\"{}\\"', 'in', '125', 'w16', '\\\\\\\\\\', 'FROM', "',\\", 'd', "'/[", 'INPU', 'Excluded', 'environment', 'SPLITLIMT', '\\./\\.', 'job_XXXXXXXXX', 'aR', 'neth', '5minute', '="*', 'DIR_TEMPORAL', 'facl', 'ioh', "'^\\*'", 'UTRACE', '_VERSION', 'file4', 'mem', 'vuar', 'tomcat', '174', 'x200', '1234', "*';", 'our', 'TBD', "'/.\\/", 'COR', 'googleTestMock', 'log_output', 'orti', 'SomeLibrary', 'ScriptAli', '/\\*\\*', 'files0', '_o', 'TOM_', 'edw', 'PacketLo', 'l2000000', 'uff', 'facebook', 'n9999', 'nalism', 'sb', 'fud', 'os', 'EDITOR', 'cH', 'logged', 'Library', 'anythingElse', 'rand_str', '64_decode', 'ed', 'bash_', 'ция', '~$\\|^', '50000k', 'MyApp', 'strict', 'blah', '_ф', 'TESTFILE', 'pg_dump', '24901', '<=', 'end', 'nore', 'files_to_', '13c1', 'opt', '_1_54', '""}\'', 'star', '5k', '╱', 'hyphen', 'pun', 'sion', 'sorted_file', 'L4431', 'thirdWord', 'requi', 'hyphenate', 'xing', 'xception', 'found', '^[', '755', 'encode', 'tata2', '978', 'ameter', '_names', 'JBOSS_', 'Sup', 'GOB', 'è', '_ex', '....', 'd_Hello_World', 'fails', ";}'`", 'myapps', 'tarFile', '\\./', 'proc_load', 'fileshare', '}\\){', 'Searched', 'uffer', 'new_target', 'photos', 'arrow', 'GNUPLOT_', 'onlyme', '224', "'^.*/[", ']){', 'vpath', 'secondWord', 'hovering_msg', 'pPASS', 'gcalctool', 'glibc', '_test', 'nameoffile', '=\'[^"|]+\'', 'new_hard_link', 'd_', 'sin', 'myffmpeg', "'|/", '280', 'DRIVENAME', 'fuzz', "'/^", 'harder', 'identi', '="|"}{', '_udp', 'scss', 'Py', '4a', 't1', '90929', 'mogrify', 'owner', '="/";', '.`', 'tarfile', 'tools', 'nma', '2000k', 'ide', ']+/', 'fiddel', 'the_command_', '/\\(\\', 'tencent', '/.*|', '"$"`\'', 'XCOMMAND', 'sh_lock_last', 'zdjecia', '_wri', 'part1', '1g', ").*'", '("^[', 'TARGET', '.\\///"', "*'`", 'lyInfo', 'ero', 'Alert', 'nd', 'pngcoppies', '7z', 'ivqf', 'Nar', 'vT', '_etcd', 'ay', 'news', 'exists', 'serve', 'command_that_writes_to_stdout', 'SPOOL_FILE', 'Xdebug', 'inet6', 'n1_t', '"|%', '("-([', 'pamv', 'orphaned', '>\\', '\\]${', '(\\([^', '32', 'killa', '\\[/]', 'OCH', 'IF', ",^\\./\\.,,'", 'sdc', 'anged', '_tra', 'authors', 'comment', 'change', 'dvi', 'dump', 'dev_user', 'service', 'save_path', 'niH', 'udev', 'kin', '="";', 'mpg', 'unit_sca', 'directory', '"><\\/', 'rcfile', 'resul', "|'", "'/./", 'ative', '..\\(\\:..\\){', 'FileName', 'aP', '*)', '&>~/', 'generated_ports', '110', '-"*', 'pol', '{(', 'clipboard', '*»', '=]\\+=.*/\\', 'pe', 'peas', '3rd', 'ux', 'linproc', 'inated', '+-[', 'terest', 'ARGS', '_it', 'pdfunite', 'show', 'dbU0', 'MOGA', 'lL', 'oradata', 'auto', 'TM', ').*', '+|', 'charset', '_P', 'searchfor', "'/')", 'NameToExclude', 'trol', '\\!', 'input', 'downloads', 'True', 'lar', 'fastcgi', 'strip', 'chgrp', 'DAY', 'Example', 'RI', 'XPRI', 'wordB', 'path_in_question', 'lay', 'WS_GROUP', 'k3', 'ation', '|─', 'aliencoders', 'bookshop', '100', '@%', '2b', '500b', 'existing_dir', 'Met', 'aj', '557600', 'vi', '456', 'error', 'chown', 'orte', ',¤,', 'Rec', 'tar', '_host', 'unted', 'folderName', 'upload_directory', 'scal', 'chomp', 'ITED_PIDS', '">', 'FE', 'fluidpoint', 'à', 'lpp', 'helper', 'environments', "([^)]*)$//'", 'difference1', 'newRoot', '*\\)\\([', 'nor', 'svn', 'only', 'Tunes', 'ruby', '1441', 'community', 'rnw', 'stdc', 'USER_', 'REPLACE_WI', "?????*'", 'executeMyScript', 'T_NAME', 'v0', 'avl', 'user_filer_project_dirs', ';;', '="-!', '$//', 'popo', 'sq', 'mber', '\\$?"', 'unix_', '`)"', 'wish', "'\\.*'", 'chsuch', '\\"{}', 'Off', 'chour', 'tme', 'upd8te', 'ing_', 'ican', 'rece', 'ned', '119', 'dslrouter', ']=$', '"-":"', '\'\\\'\')"\'', '71138', 'LST', "..='", '</', "*$'", 'PROMPT_COMMAND', 'lane', 'т', 'wsuNID', 'input_file_type', 'G_', 'pvdmB', 'msg', 'temps', 'destination_directory', '))}', 'findcommand', 'burNad', 'pli', 'userlist', 'java_ee_sdk', 'perf', '"????', 'rpt', 'site', '|%', 'IP_MROUTE', 'mount_output', ',";"}/', ')/\\\\\\', '>"/', '"\\.', '/\\-//', '20', 'pied', "<'", 'marker', 'classes', "'.\\*", 'upgra', 'jul', 'ssl', ']?,', 'make', '&\\', 'newerat', '??/', '_FOR', '864', 'inc', 'iText', 'junk2', 'u2', 'w9', 'portNumber', 'fa', 'ini', 'efence', 'quer', 'tail', 'pack', 'launchpad', "'(?<=", 'OUN', '換', 'transparent', 'lrta', ';}{', '1500000000', "'[()", 'docs', 'Dm', '"[=', 'Foo', 'd8te', 'edwarda', 'buck', 'normal', 'Rp', 'xac', 'TTL', 'buffer', 'dirToCopyTO', ":'%", '.*#---', 'envi', 'post', '}\\', 'pxelinux', 'm2', 'posix', ',\\}/\\', 'RT', '60w', 'recal', 'sdb', 'W_', 'Bcast', '/\\(.\\{', 'Dax', 'program1', 'tmp_file_', '128k', '5q', 'mapper', "@/[^/]\\+$@:@'", '/,/\\', 's_12345', 'pm', 'el6', 'aAp', 'amazon', '\\".', '\'"\'"\'', 'EAD', '_av', ')/', 'J', 'so', 'xget', 'dumpl0', 'rections', 'zeroes', '\\+\\(.', 'command_that_wri', 'identifier', 'se_dir', 'mydir2', "*#'", 'tx1', '#\\\\', '_str', 'second', '?(?=', 'cheng', '_pending', 'mysqldbbackup', 'En', 'ENUM', 'bt2', 'penulti', 'SPL', 'sto', 'str', 'sitory', '4te', 'BIG_FILE', 'dal', 'in_file', "'[^/]*$'", 'spam', 'PathAn', '/$/<\\/', '90_2', '.[', 'aldF', 'dbdump', 'server', '$(', 'icl', 'such', 'umb', '\\>/', 'item', 'jpeg', 'loss', 'eq', 'Documents', 'k1M', "=]'", '1825', 'oovy', 'SYMLINK', 'TRY', 'oraclelicense', '_to_find', 'ifconfig', '6', 'accepted_', '_SUF', 'secret', 'ousearchfor', 'xbb', 'dont', ']/,', "'¤'", 'foo2', 'loggroup', 'WP_ROOT', 'cdg', ',\\(.*\\)/\\(.*\\)$,\\', 'Folder2', 'lab4a', 'ild', '/^$/', ')/$(', 'tabs', '0644', 'Hid', 'this', '99966', 'logge', 'awsc', 'groupname', ':', 'B', 'rem_dup', 'maxdepth', 'ab', 'TEXT', '176', '-*"', '[^/]*$/', 'SSH_C', "'*/.", 'vig', 'mpv', 'demo_b', 'sshtunnel', 'quali', 'tober', 'sample_0001', 'bul', '_3', 'file_b', 'orph', '".{', 'fifo', 'fixed', '=”', 'PUT_YOUR_', 'fastqc', 'SKIP_FILE', 'fice', 're', '"$/$"', '/\\./', '`\\', 'General_P', 'ERROR', '/$/"/', '$",', 'myAr', '\'*\'"$*"\'*\'', 'performance_test', ":''", ')"/*', 'fgets', '90_2a5', "].*'", 'Cli', 'labra', 'onstruct', 'haz', 'phrase', 'identity_file', 'dhu', '0pdv', 'mac', 'grp', 'Eo', 'array', '_7zip', 's_build', 'devices', 'sTfv', 'ave', 'CORES', 'con', '200x200_', 'exludeddir1', 'thumbs', 'someletter', 'aaaa', 'ppies', 'proxy_server', 'graph', 'master', 'prod', '`*"', 'As', 'flossblog', '_files', '));', '@^./@@', 'chi', 'eeee', 'LOG_DIR', '*/*/*', 'a5', 'loops', 'CONFI', 'aeiouyAEIOUY', 'STDERR', 'mybinary', 'syslog', 'lc', '/.../', 'sess', 'rent', '>$', 'ir', '_data', 'sdcard', 'RED', '\\(.*$\\)/\\', 'large_file', ')|', 'rss', 'randn', 'ort', 'java_home', '_term', "..'", 'randfiles', 'zless', 's_to_', 'cvAnone', 'network', "'^.+:", 'ste', 'branch', '_NAME', 'sv', 'mydomain', '@', '40G', 'ARG', 'name_to_find', 'results2', 'inclu', 'nullglob', 'folder', 'ories', 'SOME', 'BZIP', '4180', 'HUP', "/./'", "\\/>/'", 'disassemble', 'ily', 'sive', 'selection2', '="$(', 'ignored_directory', 'matical', 'мент', 'lace', 'PiIP', '/\\\\/\\', ',{', 'wi', 'daily', "'^>'", 'ачение', 'sock', 'deviceuser', '$)/$"', 'mongodb', 'textfile', 'hH', '/^/$(', 'dfi', 'attach', '_sort', '"*/.*"', 'redeem', '//\\\\\\', 'someWhere', 'skipdir1', 'bukovina', 'newdirname', '{^}{\\"};', 'niord', 'tave', 'separ', 'MacVim', 'Database', 'TARGE', '}"\'*\'', 's_CUSTOM_TEXT', ':/;', 'parentdir', '."}\'', 'ABF1_6', 'converted', 'javaUsrLib', 'SA', '\\:\\:.*/\\', "!!'", '1c', 'ence', 'NONE', 'GNUPLOT_DRIVER_DIR', 'SC', 'dirlist', '2048c', 'blash', 'duser', 'test1', 'map', "'*.*'", 'loper', 'outputfile', 'gyp', 'mmm', '..?*', "'\\''{", 'column', '>>', 'nenv', 'bukov', 'WE8ISO8859P1', '/.*<', 'MYBOX', 'ecode', 'California', 'torrent', 'ree', './', '/*[', '.-', '9725', 'excludeddir2', 'perl', 'reco', '4q', 'jars', 'bz2', 'nameserver', ');', "'^[", "{}'", 'python', ']\\+', '12345678', 'PATH', 'unmatch', 'vies', ':-}"\'*\'', 'HW', 'oe', 'ission', 'linenum', 'setfacl', 'rve', 'p', '\\.*$/\\', 'teer', 'rvv', 'twitter', '()', 'REL_DIR', 'states_to_csv', 'jsmith', 'WIN', 'FINANCIALS', '/*/*', 'Bytes', 'summary', '/./\\', 'absolute_dir_path', 'some_variable', "/^\\\\?[^=]\\+='/", 'srch', 'Tc', '\\[/]\\$/\\/', 'mary', '.][', 'php_', '[^:]*\\):.*$/', '"+$(', '("', '000c', '=",$', "''`", 'Ord', 'mon', 'omit', 'va', 'sl', 'masi', '200x200', 'pattern_to_', '_PREFIX', 'objdump', '<pad>', "'.*,", 'nulls', 'resses', "'^[^[:", '/[', 'ff', 'ki', 'dfsg', 'path1', 'mpp', 'ExampleClass', 'makeallsizes', 'charm', 'parallel', 'Le', 'ple', 'refox', 'studio', 'sno', '*/&\\', 'parec', '".+-[', 'Image_200x200_', '_main', "'./.", 'rm', "]\\+'", 'multitran', ']:/', 'ESTATUS', 'gle', 'une', 'dos2unix', "'*.", 'cats', 'csp_build', 'new_destination', 'JPEG', 'edi', 'DB', 'java_', 'mytarfile', 'MB', 'snapshot', 'iwholename', 'Your', 'searchstring', 'UST', '".*/.*\\.\\"', '_version', '/\\.\\/(.+)\\.', 'Col', '=${', '/^.*$/<--', '_results', 'lease', 'ABC', 'mytemp', '"{}"', '>"', '*\\*', "//')", 'dio', '"^[^;]"', 'cart1', 'ba', 'press', 'remoteserver', 'war', '}"', 'prompt', 'Uvh', 'fol', 'FILES_', 'newword', 'samba', '2s', 'sizeFile', 'LEVE', '/...\\\\\\./\\', 'LINE', 'inum', 'FILES', 'bbb', 'h8d', '54', '`$', 'cla', 'qdm', '*"`', 'whatever', 'uti', '"*~"', 'U', '^[\\>\\<]', 'ENSION', 'licen', 'zec', '")\\', 'MYBOXES', 'suepearson', '~":', 'exec', '"$:', '6379', 'empty_variable', 'syncedpreferences', '_me', 'clean', 'nameoffiletoprintto', 'octave', 'kdown', '5000000k', '/\\(.*\\)', 'ix', 'Not', 'unlink', 'hidden', 'dsk', 'DIRECTORY', 'target', 'OLATE', 'ages', "'/<\\?", 'OfWee', 'tx', 'perm', 'dH', 'RM', 'ext', '----', 'xvdf', 'PATH1', '02', 'MOD_DATE', 'xdev', 'x0D', 'ENT_', 'AF', 'ла', 'og', 'CONFIGFILE', '+}\\', 'target_', "]'|", 'ACTUAL_DIR', '/;/\\', "':[^:]*", 'DataBase', 'sS', 'TEMP_FILE', 'SONAL', 'expand_aliases', 'amerate', 'HFf', 'procN', 'basename', '=./', '/"', '>=', '//', 'external', 'foldername', ":'/", "='*/'", 'doll', 'cture', 'ster', "$'^", 'orge', 'MATCH', 'SY', 'commit', '18884', ')+', 'fits', '..*', '/\\([^', "$'/", 'repla', 'RAM', '\\..*$//;', 'uucp', 'l1000000', 'stdin', '<eos>', '0m', ':$', 'textTo', 'Pathname', '2000000', 'source', 'ccccccc', "'^.$'", '_XXXXXXXX', 'lsize', 'c4', 'ColumnOfNumber', 'ORE', 'esplit', 'remoteTo', 'ce_dir', '_', 'tempWork', 'LAST', 'lse', 'respon', 'bun', 'Marcel', "'/\\/\\//", '\')")', 'launch', 'mesg', '\\}\\).\\{', 'htac', 'Tes', '{}\\', 'xampp', '1494', 'list', 'larg', 'CHUNK', 'synta', 'aPSHiv', 'CY', ':).*?(?=', 'y171plus_build5638', ']*"', '".*/(', '[^.]*"', 'href', 'StringB', 'fcgi', '1858', 'Xvfb', 'OU', 'l1', 'blocking', '88', '].*?:"', 'czf', 'passwd', 'mmss2s', 'plzip', '377', 'w130', 'stdbuf', 'ND', 'hours', "[^:]*$'", '++){', 'TARGET_', 'testExpress', ",'|", 'mo', 'td2', '_de', '/.*(\\([', 'neluser', 'lvim', 'SOME_NUMBER_OF_DAYS', 'r_Tested', 'ENDFILE', 'WKD', 'LC_', 'xbacklight', 'now', 'A', '_rec', 'checkwin', 'SH', 'vel', 'IAS', 'kat', '".*\\\\.', 'azvu', '"./%', '#<[^>]*', 'mbol', 'mp', 'bash_profile', '1_R1', '"}$.==', 'my', 'save', "'[\\", 'LogLe', 'uplot', ':)?(([', 'ignoreme', '(\\', 'bop', 'mydatafile', 'joeuser', 'vaut', "'.*\\.\\(", 'bg', 'man5', '_numbers', 'uch', 'ri', 'pb', "#*'", 'OPTAR', '4e', '/,$/\\', 'nobody', 'Assemb', 'cmd1', 'di1', 'pandoc', "/\\.\\///'`;", '64', 'SSA', 'expand', '226', "'*~'", 'mysong', 'fastq', 'globstar', 'onical', "'^.*", 'description', 'mbstring', 'path_in', 'loo', 'runprogram', 'Images', 'execfail', 'structure', 'NOPORTEXAMPLES', '\\){', 'shop', '="*.', 'wa', '#.*\\$#\\', 'monitor', 'MS', 'roniz', 'nums', '‘*', 'ignore', ']\\\\{', 'tdir', 'input_cmd', "'*[", 'poo', 'ET', ":'$(", '(?:.)', 'parent', "'-------')", 'michael', 'twi', '>${', 'pvdm', "'", 'mylist', '+$()', 'AV', 'tos', 'split_', 'soa', 'LOGNAME', 'your_email', 'checksum', 'Cook', 'ServerAliveInterval', 'ema', 'TAB', 'ln', 'pw', 'lbzip2', 'boo', "=)'", 'endl', 'backup', 'Folder', 'SSH', '_CACH', 'stop', 'NameHere', 'codes', '150k', "'.*/\\(", 'RIN', '\\}[^', 'fi', 'localized', 'gitlab', ';\\', 'USERNAME', '}}"\\', 'dat', 'criteria', 'ord', '/,$', '?.*/??;', 'ARCH1', 'musicdir', '\\(.*.', '1024', 'upload', 'cheez', 'whoam', 'mmv', '=="', 'derni', 'lst', "'>'", 'mvn', 'ipcrm', 'bund', '..\\){', 'tempd', '-.*[\\', 'directory_path', ':).*', 'Identi', 'sche', 'chromium', 'destinationfolder', 'lement', 'Inc', '_command_', 'prettify', 'crash', 'c1', 'fields', 'save_pid', '65000', "'/^\\.$/", 'processlist', 'la', 'permissions', '(]*', '\\"$(', '120', '~\\]*\\)/\\([^]*\\)$~\\', 'mod_jk', 'fullpath', 'index', 'optipng', '1973', ']/', 'aesthesys', '199053', 'S_', 'OURCE', 'dv', 'Author', '_level', '42', 'ets', '<unk>', 'myhome', 'sul', 'articles', 'LETE', '2120', 'зн', 'Prints', 'sX', '”&\\', 'FooBar', 'outside', "|/)'", 'ecking', 'ntest2', 'keyword', 'puts', '(.*', 'do', '__pycache', "/;'", 'replacement_phrase', 'Macromedia', 'Modif', 'alog', 'odNameHere', "'/$(", 'my_previous_alias', 'MY_DIR', 'map_metadata', '#\\', '30M', 'new_something', '-/', ']*)?', 'split', 'eral_P', '/#(', '\\\\?', 'trap', 'WannaFind', 'DataBaseSynch', 'Since', ')?"', 'FILEPATH', 'TIMES', 'zipinfo', 'avo', 'messages', ')[^', '(",","",$', '-]/', '+;', '120024000', '{}/', 'w29', 'files_to_find', 'ipy', '=","}', '/}', 'val', ',$', 'spawn', "]+=)'", 'everyone', 'myDB', '[-', '://[^/]*\\)/.*;\\', '\\\\\\\\', 'AB', 'link', 'mindepth', 'ipo', "/'{}'", 'should', 'NET_IF', 'ban', '"$")', 'NUM', '.*\\)\\|\\(^', 'mystring', '/^\\.\\///"', '="${', 'MERGE', '\\:"', '="$.', ")$'", '_vimrc', 'REPLACE_WITH', 'dvtcolorconvert', '(]*\\)', 'matching_', 'a2', 'source_file_or_directory_name', 'bootstrap', 'via', 'значение', 'creation', 'tal', 'ul', '-"`', '23c', '(([', 'data', "='/", 'example', 'fc21', 'background', 'xyzzy', '\\"{}\\"', 'root_path', 'arp', '1500c', 'call', 'stem', 'nonExist', 'months', '/-/\\', '_empty', '..."', 'sedlav', 'older_log_', 'Ag', 'SOURCES', 'XUS', 'alice', 'Ac', 'ansfer', 'yload', '.*/#&/’', ']\\{', 'absolu', 'thub', 'OATS', 'ENTER', '@*"', 'unittest', 'ADDRESSS_FILE', 'un', 'IOU', 'esoft', '500m', '/^\\\\?', 'syntax', 'resuming', 'some_file_name', 'permission', '".*\\.\\(', 'appdir', 'clob', 'USER_NAME', ')/*', 'RANDOM', 'Louis', "'{}", 'dit', '\\*"', 'readlink', '/(*)', 'Date', 'storage', '//.*:\\', 'FIX', 'emea', '654321', '__checksums', 'my_user', 'p481', 'Directories', "!.*/!!'", 'homeDirData', 'BIN', 'vxF', '=#;', 'dir_race', '.....\\)/\\', "'[^=]*$');", 'rop', ',\\\\,/,', 'aac', '‘%', 'dir_one', 'somedirB', 'input_file', 'loc', '+\\', 'Csu', 'SED_', '41', ':-', '?[', '".', "%'", '}:"{}"', '/”', 'Rd', 'ee', '/^/', '_event', 'something', '\\`', '&/\\.', 'abc2009', 'abled', 'homeDir', 'myprog', 'fndr', 'NAME', '204', "'.", ")}'", 'used', '_FILE_PREFIX', 'my_log', 'lb', 'takuya', 'myNewPathOn', 'delf', '__', '$)', '03', 'With', 'getline', 'list_files', 'histverify', '2012080', 'SConstruct', '${${', 'CD', '92', ":'`", '_to_your_directory', "=';'", 'ium', 'link1', 'modulesgarden', 'acqui', "\\+'", "==)'", 'o0', 'clang', '/../&', 'jdump', 'lete', 'filesOf', 'CDATE', 'Working', 'wav', 'file1', 'ei', 'Art', '++;', 'Wed', 'EXTERNAL', 'unt', '/...\\\\\\./', 'cess', 'ZipFileName', '2222', '_pattern', '24q', '27', 'Statisti', 'nTester', 'toupper', 'twofish', 'tisti', 'coreutils', 'aa', 'unit_scale', 'my_previous', 'lastdaymonth', 'thegeekstuff', 'опция_де', 'mtab', 'install', 'myvariable', 'ns2', 'bigFile', '15000000', 'arson', 'unzip', 'L44', 'seamstress', 'virtualenvwrap', 'eth', '1n', 'alpha_dir', '$/*', '}")"', 'Class', ']*\\))', '))/$(', 'print', 'pP', 'universe', 'rmtry', 'ray', "'^#|^$|", '_relative', 'Dv', 'rvi', 'YourParameterFile', 'avi', 'runscript', 'mla', "/^/./'", '/\\\\\\\\\\', 'Server', '333', "$//'", 'rewrite_log', 'LC_ALL', "./'", '/"$\'\\', '\\"\\$', '=..', 'dcx', '231', 'correctFilePathAndName', '1Tb', ').', 'f4', 'i1', '8909', '/.\\', 'ssh_host', '<(', 'rec', '500000', 'xpath', 'GH', 'andn', 'lig', 'us', 'rx', 'ly', 'lk', 'man', 'zp', 'w30', 'bills', '"};', ']+/\\', 'DB_HOME', './\\', 'mime', 'cake', 'gv', 'CES', '/}$-', "'./", 'ronment', 'v6', '$|', '"/../', 'enco', 'b2', '\\..', '"+%', 'EPOCH', 'stage', '.*.', 'acterSet', 'Features', 'RE', 'D_import_script_', '[][[:', '_folder', 'FINAL', 'ORIGINAL_FILE', 'or_directory_name', '?{', ",//'", '."', "'[^=]*", "*.*'", 'criter', 'paramName', ').*/\\', 'xvdf1', 'expected', '="="}', 'laide', 'wheel', 'Final', 'Tz', 'elinks', '/^...\\', 'QuickTime', 'comz', 'ummy', 'YOURSTRING', ':]]\\{', 'laSr', 'Qui', 'is_nullglob', 'String', 'longer_', 'ñ', 'interactive', '*.', 'vtx1', 'tor', '100k', '":");', "*'|", 'pan', 'gu', '[^.][', 'magie', '/|$/\\', 'lsR', 'VLAB', 'domains_', '‘+%', 'ec', 'qi', 'rb', 'mynet', 'vFf', 'bin', '=*.', 'oldwindow', '??;', 'rep', 'demo', 'l9', 'For', ':-/', 'qq', ']|', 'seen', 'laravel', 'STARTING_DIR', '/*.*', 'dir_list_1', 'largesplitter', 'SHEL', 'ao', 'isc', 'alpha', '\\|', '/$/"$\'\\', 'Shared', 'M_', 'WIDTH', 'x61', "'/[^/", '_b', 'PYTHO', 'kr', 'newArch', 'randn20M', 'iso', 'solu', 'Adelaide', '107', 'template', 'myprojects', 'Prt', 'etc', 'Ab', "/^/.+/'", 'Naur', '/./\\\\&/', '512k', 'handle', '/^(.*', "/'/\\\\\\", 'EAR', '=\\"{}\\">{}</', 'lo', 'LEAD', 'src_dir', 'less_in', 'General_Purpose', 'w99', 'vsantraces', 'Processed', '88s8d', '/\\(.....\\)\\(.....\\)/\\', "\\).*'", '”*', ':]]*$//"', '.{/,/}/', '2010', '/,+', 'IFILE', 'WI', 'yourfile', "'^/", '"+\\', 'EG', '"/[^/', ';\\(', '782263', 'nul', 'CRLF', '\\)/,', '_github', 'ager', 'leHTTP', '|.*/\\([', 'andnav', 'Thtml', 'SouthEast', 'libGL', 'pen', '_sed', 'sec', 'Td', 'EX', 'otherHost', 'ews', '/$/,/"', '\\}\\)...../\\', 'windows', 'kenneth', '123456789', 'zeni', ',"[', '+=\\$', 'hooks', 'check_script_call', '()"', '\',\'`"', '"}$.', '/[^(]*(\\([^)]*\\))', 'STRING_TO_REPLACE_IT', 'testEx', '\\(:[', 'nlE', 'st', 'res', 'vn', 'toreplace', 'ssh_user', 'TarFile', 'GROU', 'S_TO_SAVE', 'content_dir1', ']*\\([^(]*\\)', 'related', '/\\//', '/-/./', '177', '^}{\\"', 'newDir', './*/', 'sbo', 'wef4fwef54te4t5teerdfgghrtgdg53', 'pdir', 'execut', 'C1', '.*|', '4096k', 'mfon', '`"', 'serveripaddress', 'amarok', 'pony', 'soc_attach', 'af', 'message_tra', 'Tok', 'w10', 'someDirectory', '"`.', '*//', '"$\\', 'ict', 'ORI', 'PROMP', 'schemas', 'postscript', 'sI', '16256', '{?,', 'dents', '/*/*.', 'requ', 'address', 'anch', "'.+\\.(", 'xuser1', 'oser', 'ndeklein', '(\\"$', 'PATTERN1', 'Wanna', 'wordfrequency', '")"', 'system', 'gzilla', 'Fe', "'[:", 'INT', 'proc1', '10', 'cen', 'SessionName', 'empsal', 'Perm', 'doub', 'yourapp', 'pyrg', ']*[.][', 'unalias', 'Direc', '"}{', 'ECTOR', 'dollarsign', '1000M', 'GMT', ':.)', '?,', ']+"};', '.*/"\\', 'djzah', 'vin', 'CONFIG_A', 'od', 'fuddel', 'LC_CTYPE', 'oracle', 'LC', 'invalid', '/$"', '2013', 'john', 'ай', 'TESTVAR', "'~'", 'sched', 'D_20140', 'li', 'my_test_expect', 'keyname', 'sf', 'STRING_TO_REPLACE', 'doubleclick', 'xeyes', ',', 'hex', 'scien', 'c14', 'your_directory', 'lighttpd', '|^[[:', 'BUFSIZE', '_repos', '10022', 'dwe', '])', '3fc', 'YESNO', 'prev', '|.', 'getLine', ']}', 'installed', ')/$"', 'https', 'jun', '_rsa', '_up', '1199092913', 'file2015', 'ner', 'per', 'yourProgram', '700M', 'project_dev', '@/@', '~/[', 'LATE', '$,', '/,"",$', '1970', 'aux', 'corner', 'item_1', '5900', 'portgroups', 'ComputeDate', 'vacations', 'поис', '2080', 'yuv', '!(', "'/$'", 'bis', 'LOG', 'root_dir', 'Usage', 'some_function', '/^\\(.*\\)', ",/'", 'lan', 'FoooBar', 'FPAT', '04', 'snmp3', '{,/.', 'Australia', 'FILE2', 'VEN', 'SCRIPT_DIR', 'avlzp', "';'", 'params', 'имя_ф', ',\\/\\$,\\', 'DRO', 'does', 'fF', 'oldfiles', 'pho', 'my_editor', 'context', 'RELEASE', 'htdocs', 'late', 'test3', 'pg_restore', 'ч', 'я', 'itemlist', 'Comp', 'A999', 'zu', ':~', 'part', 'ency', 'FUID', '][^/]*', 'TARGET_FOLDER_ROOT', '19crf', '10gen', 'nolea', ')))/../', '|(\\./', '/\\/%\\', '[^/]\\+', 'psgrep', 'volume1', 'gno', 'reward', '$"`;', 'int0', 'ip_', "'\\\\|", '-"', 'to', 'script', 'cron', 'linked', 'VENAME', 'Oc', 'LIMITED_PIDS', '+-', 'submission', 'ip_address', 'iled', 'matching', 'avconv', 'cputime', '*[', '"\\.\\.\\."', 'sshpass', 'adle', 'Ca', 'sort_test', 'dir_a', 'Law', 'Compare', 'ORACLE_HOME', 'De', '*.$', 'ED_', 'BT', '16f', ':]]/', '*/*/*/*/*/*', '_bak', 'wobg', 'Se', 'surf', 'um', 'codec', 'beispieldatei', '0001', 'gpio', '-.\\.', '|$(', 'apt', 'OrWithoutSpa', '.%.', 'Friendly', 'MENT', 'BAG', 'CLEAN', 'jbo', 'mpeg', '{}', 'geoip', "',')", 'andrewr', '_CMD', 'Camsoft', ':]]+#\\', 'BASH_REMATCH', "#'", 'SIMPLE_OP', 'yargs', 'disassemb', '*/\\', '$\\)', '@^@/', '${', 'Pics', 'WNLOAD', '360', 'ф', '$"`', 'SEARCHPATH', 'WordPress', 'proxy_port', 'arhv', 'RIP', 'installe', 'gram', '}}', 'ta', 'xEF', 'local_file', 'path_to', 'date_222days_before_TodayDay', 'my_large_file_1', '26', '=%', '1339', 'svnlog', '\\."', 'dailyReport', '"????-??-??"', 'vpr', 'asf', '"${@:', 'arkon', 'repolist', "/\\..*//'", 'Uni', 'bgr', "'.'`", 'zzah', 'JP', '=<>;', ":]'`", '?({', '//${', 'lnE', 'MOV', '/...\\([', "]?'", '270M', 'Des', 'MAND', 'ary', 'TB', 'cessed', 'j1', '-------', 'includes', 'DUALCASE', '222days', 'exclude_me_too_any', 'grea', 'yourfilter', 'DB_NAME', '**/*.', 'dT', '1Ctrl', 'authority', '8746', 'digit', 'safe', 'readelf', '.*$//', 'ifcfg', 'clever', 'EMAIL', 'verScript', '20140618', 'libxml2', 'latexmk', 'IN', 'FNR', 'local_sd_customize', '?*', 'tutonics', 'SurfsUp', 'mydir1', '[^.][^.]*', 'tun', 'if', '123_', 'libstdc', "[^/]*$//'", '#.*/#', 'wil', 'notes', 'brary', 'fprintf', 'file0001', '}$"', '$//;', 'ull', "/,$//'", '10k', 'split_folder', 'nogpgcheck', 'tatic', 'PORT', 'xargs', '191211', '─', 'debu', 'bgxjobs', 'poop', '"/.', ".*\\'", 'xclip', 'usbdisk', '/^/:/;', 'lal', 'Rvn', '"\\.*"', 'itp', '#.*/', "<<'", '_before_TodayDay', 'aned', 'mpt', 'NEL_VERSION', '="{/}"', '/.*/"&"/\'', 'KFILE', "'.*\\.", 'Statistics', '/;', 'DayOfWeek', '770BL', '45678', 'MYSQL', "’\\'”", 'coun', 'mandoc', 'meta', 'Ver', 'f0', 'sor', '".*?"', 'PHPBB', '\\)$/', ']*', 'JBOSS_AS_DIR', '\\"{}\\""', 'trace', 'NET_', 'HOSTNAME', 'xs', ':\\\\', "|/[^/]*$||'", "'%-", 'ADVER', 'ization', 'CURR', 'Nr', 'cat', 'Xrunjdwp', 'Press', '/|', 'staff', 'YourProgram', 'StringA', '}/.', 'deep', 'Simp', 'INTRANET', '(.', 'fan', '"\\"")', '_id', 'mounted', '443', 'TW', 'bk', 'lugin', '="./"};', 'print0', 'r0', '0', '24297', 'moi', 'fit', 'json_', '(.*));.*\\?', 'g53', '/*"', 'complicated', 'RAND', '17709', 'allFolder', 'March', 'YES', 'unless', 'CONFIG_', '428', 'Ade', 'EA', '_ROOT', '25', 'rr', 'zadm', '!"', 'firm', 'mysq', 'privatekeystuff', 'fuel', 'list2', '987', 'errexit', "]+'", 'dsa', 'vzZ', 'ER_', 'robots', '_docs', 'ALC', 'd\u200c', 'User', '/@', 'ergep', 'ago', 'limesurve', 'io', 'atures', '}', 'ROR', 'stringYou', 'GE', 'jet', 'NEEDLE', ']}/', 'newEmpty', 'photoA', '770BL1_R1', 'alaxy', 'pull', 'compilation', '-){', 'csh', 'user3', "'/.*", 'aspell', 'pty', 'Else', '\\)$"', '_EOL', '}"`', 'gif', '5000000c', '=*', ':]', '2966p', '/:$//;"', 'calc_', 'block_num', 'mkdir', 'oldfile', 'lS', '"*[', 'devel', 'log_', '",$', '""', '82fe', '\\/\\', 'Development', 'n5', '.|', "/\\.*'", 'Image', 'endlooop', 'thor', 'jo', 'ALERT', "'^#'", 'myapp', 'currtime', '\\.[^.][^.]*\\.[^.][^.]*', 'CV', 'maps', 'OFS', 'awscli', 'spath', '"[[:', 'required', "}'\\''", ';/^$/', 'psql', '+"%', '){', 'sw', 'my_par', '("%-', 'TO', '\\\\\\./', ').*$/;', 'zipFile', 'y', '"+', 'photoC', 'canonical', '/,/^', 'and', 'mal', 'anythingElseIwanna', 'templateXXXXXX', 'allFolders', 'securebackup', 'WEEKS_TO_SAVE', '210', 'right', 'seconds', 'ENID', 'Windows', 'bj', '"./"', 'plu', 'cmd_str', 'Ph', 'gunzip', 'omnitest', '";$', 'DIR_PATH', 'HOSTZ', '_java', 'aaonly', 'pkgconfig', 'full_path', 'ireg', 'esofthub', 'remote_user', 'SCREENID', '10Dm', 'NEW_DB_NAME', '<', '*/*.', ',"[=.]");', 'tests', 'datafile', 'FG', '\\)*[', 'ico', '..\\(\\:', 'sampletest', '"[\\', 'sourceDir', '.]*-', 'trl', 'blah1', 'iwrite', 'E', "'/\\-/", 'pt', 'mykey', 'mapho', '_CUSTOM_', '-|', '${${$}[', 'ualenvwrap', 'take', 'COL', '"))', 'mi', '_dest', 'Tb', 'output', 'largecal', 'yoda', 'blan', 'XAMP', ']\\+\\.[', '201512', 'do1', '444', 'webapps', 'ckerContainer', 'files', 'noall', "('", '...../\\', 'faken', '8', 'didate', 'pathname', 'mx', 'ACE', '\\>', 'key', 'uiTotalSize', 'ble_vars', 'hr', '512', '84', 'scrip', 'ana', '15s', '/\\/', 'sti', 'user1', "'^\\", 'debian', 'tag', 'folderTo', 'nIPADDR', 'remote1', '0e', '*/=/', 'pstree', '!#^', '<"', '\\*\\:', '/\\-', "'^($", '"?*"', '2023', 's_to_csv', 'NT', 'gateway', 'SKIP', 'organic', 'cycy42', 'arsi', 'contains', 'bak', 'SCCS', '+"\\', 'far', 'd0', '/((:', 'CLE', '_t', '5M', 'newnameserver', "$/..'", 'rUS', 'tyle', 'my_file', 'FSIZ', '\'/"', '3400000', '}*', '647', '9f', '~\\', '$$/"$\'\\', 'cpio', 'hda1', 'pid', 'tmefndr', '"/"$', '_url', 'n1', '/.*$:', "'#'", 'myfiles_split', 'router', 'restore', '_output_file', 'echoargs', '100000k', 'INPUT_', '18581', 'опция', 'ps2pdf', "/^\\\\?[^=]\\+='//;", '?^', 'itunes', '$);', '"{}".', 'RC_files', 'xgettext', 'BAS', 'job', 'ger', 'myscript', 'Hn', 'tld', 'sort', 'wnl', '200510071138', "]*)$//'", 'targeth', 'gnuplot', '%/*}";', 'b', 'container', 'apache', 'END', '\\]', '????????????????', 'He', '$)\\', 'lastline', '/^...\\(.\\{', '(.*)/$', 'p3382', 'OUTPUT_FOLDER', '\\$"', 'relpath', 'KEY', 'url', 'dtrace', 'LANG', 'deve', 'fls', 'setar', '0r', 'ecal', '_201', '.*\\|', 'aes', 'tical', 'shared_', 'Sublime', 'Fvf', 'astyle', 'Scrip', '*-', 'want', 'ken', ':.*$/', 'Found', '_C', '///', ')*\\', 'est', 'php_session', "`*'", '\'))"', 'ped', 'UP', 'defg', 'filepat', 'EF', 'whatyousearchfor', "'./*'", 'grant', 'string3', 'xpvf', 'fileinput', 'й', 'BU', 'XSHARED', '767', '58', 'orman', 'sch', 'TIME', '5534', 'epth', 'HIST', 'pogt', 'back', 'ivd', 'headers', '9a', '"^./[^/]\\+', 'rela', 'MusicFiles', '2_version', '\\.', 'dropbox_uploader', 'pgrep', '#\\|^', 'priv', 'extglob', 'ypat', 'state', '330', '900', '$\\|', '}{', 'Arch', 'JPG', '.$$', '19', 'HOST', 'source_dirname', 'Tra', 'utf8', ']*)$/\\', 'ENwbur', 'egrep', "'\\", 'matching_text', 'sh_lock_lastPID', 'block', 'SPACE', 'nAut', 'dpkg', 'PH', 'unsigned', '&";', ')!\\', 'preci', 'ks', 'bravo_dir', 'year', '}.', '235', 'func', 'ated', 'Sta', 'avR', "'^[^;", 'eco', 'clip', 'que', '=$/$', '%%.', 'phra', 'UDE', 'sbin', 'manager', 'CONFIG_64BIT', '[^/]*', 'dgst', '--------------------------------', 'JAVA_USER', '=\\\\\\|^', 'ES', '\\.$/', 'aders', 'elin', '/$', 'PATH_TO_NEXUS', "'[-]*'", 'target_location', '-*\\.', 'xb', '+="$"', '../', 'dfm', 'Spool', '*;', '133986838', '/./', 'ez', 'arjhai', 'Dexpression', 'chael', 'Drop', '-`', 'cdable_vars', 'newhome', 'umask', '"|"$', 'QMAKESPEC', '].*//', 'nice', "'->", '1m', 'mp4', 'DIR', 'jm', 'ldb', '/^[^', '-$(', 'LLMENT', 'toexclude', '69', 'google', 'important', '1889', '"<\\?', '1C', 'correctFile', 'idpoint', 'postfix', '\\"', 'Checking', 'stress', "'.*/(", '_vm', 'mktemp', 'Wel', 'replacing', 'noleaf', 'spen', 'vet', 'arename', '3d3d', 'id_rsa', 'filesPermission', 'hashmove', 'sle', '+:', 'ces', 'dmz', '\\/.\\', 'viclient', 'Backup', 'SRC_DIR', '{^}{\\"', 'ssl_engine_log', '$\\)\\|\\(.*', ')$"', 'entries', 'ENT', '>,,', 'PAT', 'Onl', 'new_folder', 'exist', 'CLOSED_', 'RGE', 'bif', 'Hf', 'command_', ')"`', 'fakerandom', "$'\\", ')$(', 'VIR', 'binaries', 'MPEG', '4j', 'exe', 'z', '_out', 'cklist', 'groupinstall', 'for', 'candidates', 'wait_event', '~/', 'whatIWantToFind', 'unbind', '"\\"', '{$}{\\"}\'', 'btree', 'es_MX', '_before', 'Pictures', '//[', '"$.', 'whois', ")'", 'histappend', 'Alastmodified', "'\\\\", ':]]\\+', 'folders', '".*,', 'ION', '6s', ":'$'", 'script_7zipi', 'eva', 'semi', 'word2', '-/-/', 'ecia', 'my_ips', '321', 'nocase', 'dcr', 'modified_copy', 'kms', 'specialusers', 'konqueror', 'sar', 'bzip', 'longer_string', 'Examples', 'noreport', 'dm', 'huzzah', '.*\\"', 'elfc', 'otherscript', 'Sep', ']*\\)/\\([^', 'recursive', 'nestc', 'anewer', "'[+", 'm', 'pp', 'fincken', 'tive', 'unk', 'ServerAlive', 'fspec', 'erfile', '168', 'JAVA_HOME', 'cls', '/\\.\\./', 'DF', 'ATIONS', 'completion', 'neweb', '$`', '60', '\\}$/&', 'adbind', '1Ld', 'Token', 'LastCh', 'SPOOL', '$)"', '/@/', 'SECTION', 'rvc', 'speci', 'bw', 'NEED', '/$/$', '".*\\.${', 'Beispielinhalt', 'middle_id', '{},', 'commands', 'autumn', 'KD', '##*/}"\'', 'partial', 'crf', '".*\\(\\.', ')$"`', 'Word', 'nH', ']%/{', 'ujf', 'sib', 'SHiv', 'faststart', 'TMPPS_', '"==', ']+%?[', 'vali', ":/'", 'mainDir', 'deni', 'alpha_real', 'OF', 'xls', 'speed', '20131205', 'whatIW', 'TH', 'LC_MESSA', '=,', 'ming', '_dec', 'min', "*(?==)'", 'xsel', 'preview', 'Lc', '2008', 'R', '/(.*)\\', '"./[^/]+\\.', 'fig', 'x9fa5', 'hDax', '"./$', 'BASH_RE', '|──', '}\\;', 'test2', '20111130', 'Uploader', '")}', 'okdir', 'Acommand', 'uI', 'dest_file', 'libc', 'BA', '_RE', 'LECT', 'target_dir_1', 'с', 'mm', 'XXXXXXXXXXXX', 'send', 'september', 'xxx', '}@$', 'vpn', '3g', 'label', 'wo', 'ssan', 'trunca', '%.*}.', 'stuff', 'real_dir2', '\\/[', '_dir1', '*.???"', 'off', '-*-.', 'subject', 'Pr', 'madhu', 'GES', '/*/*/*/*/*', 'accè', 'fc', 'count_e', 'tomcat6', 'qu', 'srz1', '###', 'UTR', ',+', 'alwa', 'G_FILE', 'unes', "'(", 'Expansions', 'rgrep', 'dbms', 'sn', 'TextForRename', 'less_insecure_private', 'Mess', 'Par', '"^.*~$\\|^', 'hour', '.]\\+\\)\\', '=$(', 'du', 'pro', 'ine', '2M', '@`', 'not_empty', 'another_script', '/\\\\.', 'ook', '=;', 'nown', "-='", 'fif2', '\\(^', 'WAR', '-${', 'Hidden', '"!', '"\'/./', '_sd_custom', 'ifarr', '))[$', 'delete', 'another', '.*#', '})/\\', 'Test', 'k2rn', '!{', 'Welcome', 'target_path', 'cebook', 'XVIR', 'COM', 'irun', 'XMEM', 'stract', 'picture', 'ea', 'Remote_IP', '>{}', '"(?', "*//;'", 'we', 'get', 'Tux', '#-+-.*#---', 'ty_file', 'NEXUS', 'HISTFILE', '|[/]||', '".*/\\..*"', 'Cm', 'ble', 'max', 'nn', 'sblmtitle', 'vboxsf', 'FINA', 'chroot', 'scriptor', '*/*/*/*/*', 'aufs', '>.*/', '_get', 'REPLACEMENT', 'locked', 'MY_APP', 'lua', 'ECTORY', 'scriptname', 'thisHOSTNAME', 'nounset', '||"', ")'|", '/}$(', 'ect', 'ALS', ";}'", '="\\', 'imaginary', 'UUUUUUUU', 'current_working', 'Lemon', 'c7', 'L1d', 'appname', 'nano', 'Alive', '_etc', 'USER', 'cpu', 'shopt', 'In', '"^[[:', 'wr', '#|^', "}]}'", 'slc02', '72260', 'Message', 'rse', 'ac', 'Rename', 'someCommand', 'less_insecure_private_key', "--->/'", 'feeds', 'targetdir', 'profit', 'anE', '}/^', 'boost_1_54_0', 'attrib', '"*/.', 'cturer', '987654321', 'abstract', '.*\\', 'ER_OF', '28', '\'"{"', '`!"', 'pect', 'renumber', '"\'\'"\'"\'', '/^\\.\\///', '_ORI', 'Trash', '6b3c', 'J_PATH', 'nle', ')][[:', ']}")', '-\\', "!'", 'yourEv', 'mye', 'verify', 'Cam', 'rqx', 'device_id', "'.[^.]*'", 'Default', '\u200b', "',", 'DER', 'ADDRESS', 'staging', '_more', 'x3', 'my_home', 'sdt5', 'askapache', 'CP', 'GOROOT', 'gravity', '\\(.*', 'n4', 'serv_share', ",^\\./,,'", '\\(.*\\', 'f3', 'you', '!/^', 'ecke', 'msgs', 'somewhere', '\\\\\\$', '.*?', 'spa', 'prefix', 'MySearchStr', '20238', '?.*"', 'bash_completion', 'ctool', 'drop', 'PUT_YOUR_STRING_HERE', 'ALL_FILES', 'tommye', 'mylink', '!\\', '"\':', 'can', 'thEa', 'receiving', '"#"', 'MY_PATH', 'regular', 'ih', '"`', 'emptyfile', 'sarnold', '/,/}/', 'noclobber', 'aeiouyAE', 'oops', '="{}"', 'totalLine', 'bunzip2', 'makeallsize', '0777ne', 'System', '/(', 'pkey_new', '/\\..*//\'`"', 'builder', 'cannot', 'G', 'jce_policy', '%%"', "/]'", '\\).*/\\', 'xcb', 'Di', 'eme', 'php_admin_flag', 'icanhazcheez', '".*\\.\\"', '[%', 'DEST_DIR', 'spammy', 'untu', 'oldalias', 'dfgg', ',^\\./\\.', 'ABS', 'obind', '/}${', 'asia', 'ternative', 'sf8h', ',//', 'sg', '}[[:', 'phtml', 'WINCH', 'USER_AT_HOST', 'RECOVER', 'a86', '/^\\\\/\\', '".+\\.$"', 'OTH', 'prj112', 'wE', '="*"', 'FileWithColumnOfNumber', 'sysconfig', 'yest', 'ue', 'shif', 'xc', 'FRO', 'azP', '3307', 'serial', 'codepath', 'dWord', 'low', 'exception', 'toni', '2pipe', 'TIMESTAMP', '/:\\(.*\\)/="\\', '".*\\.', 'ath', 'invalidTemplateName', "':[", 'pa', 'ft', 'XT', 'wordfrequ', '212042', '259', 'DST', ']*\\', "'.*/\\..*'", '15M', 'AE', 'cancel', 'printdirections', 'CURRENT_PID', 'test_', "'*'", 'mnt', 'group', '>/', 'nb', 'bugzilla', 'toBe', '{,}', 'MAVEN', 'result', 'libglx', '[^;', 'myprocess', 'gc_maxlife', 'ces_relative_path_names', '*/$', "]$'", 'ISOLATE', '\\+:\\', '(.*));.*\\?>"', '"{}', 'chavzP', '-=', 'aba', ").\\*'", '53', 'tmpfs', 'ST_', 'webupd8te', 'san', 'gdg53', 'sum', 'po', 'IFS', '/^\\\\/.*\\', 'arg', '_in_folders', '-*#\\', 'insanewebproject', 'repos', 'sF', 'loop', '\\}\\', 'verse', 'nvm', 'artic', '3309', 'true', 'chapter', 'TEST_FOLDER', '"^$', 'vx', ':+}\\', 'ey', '$-', 'ticles', 'recursion', 'cf', 'Key', 'newstring', '*#|\\', 'extract_dir', "'/:", 'compone', 'pdumv', '")",', 'ext3', 'Partition', 'Apple', 'vmware', 'ap', 'yy', 'ar', 'XXXX', 'ys', 'erge', "'^..$'", 'ittp', 'ferences', 'anic', '|/[^/]*$', 'applied', 'NAL', 'pats', 'Added', 'nTested_Hello_World', '{?,[', 'AIL', 'toBeSearched', '20k', 'led', '\\(.*\\/\\)\\(.[^/]*\\)/&', 'orss', 'implicit', 'ched', 'proper', '":"[^"]*"\'|', 'doFirst', 'Hel', 'Iq', 'vEzZ', 'Mic', 'cwd', '/^\\([^:]*\\):.*$/', ']/{', '8000', 'tiveFiles', '`;', 'LearnAs', 'pmd', 'doit', 'tid', 'INER_', 'elinhalt', "[^=]\\+='/", '10M', 'CProgram', 'invert', '270', '_modules', 'modified', 'lshw', 'obsolete', 'P8', '##*/}";', '/,/^$/{', '},', 'xpg_', ')?', 'ived', '\\.$', 'nMEM', "\\}'", 'SW', '.*|.*', '../[', 'tdggen', 'acodec', '“.*”', 'folder_name', 'remote_0', '."\'', 'ight', 'hai', 'myDir', 'output_prefix', '.', 'page', '/^.*\\(:[', "'*.'", 'PAS', 'ging', 'Copying', '&>/', '011', 'kle', '86838', '.$$"', 'stable', 'gmt', 'Be', '@${', '\\"{}\\', 'ye', 'file3', '/.[!.]*', 'originalfile', 'uga', 'overallener', 'george', 'site_sd_customize', 'myvolume', '\\(.*\\(\\.', 'movie', 'iname', 'antoni', '~/.', 'param', 'xyz', 'mount_', 'pare', 'aliases', 'usernma', 'dmidecode', 'xf', 'uuidgen', "'/---", 'zegrep', 'MYPROGRAMNAME_KERNEL_VERSION', '*|', 'pub_url', 'import', "'#[", 'mydatabasename', 'o2', '16482', 'Jul', '+${', 'mtdblk', 'ders', '`!\\', 'ignore_read', '/.[!', 'в', 'all', 'path2', 'heredoc', "'\\./(.*\\.", '78226', 'twice', './.', 'kb', 'search_term', '".*\\.[', '400', 'bil', 'son', 'FY', 'scp', 'OL', 'firstWord', 'trace_pipe', '0L1', 'alrt', '5G', 'dirtoexclude', 'series', 'brew', 'CPU', '10_Recommended', 'essbase', 'during', 'Feb', 'Own', '_LR', 'b928a862', '\\`;', '}")', 'myld', 'EXCLUDE', 'NLOAD', 'nt', 'patterns', 'topretty', 'pibeta', '}{$', 'mirr', 'oldStuff', 'dirpath', 'YOUR_STARTING_DIR', 'device', 'interval', '/.\\+', 'MANIFE', 'management', 'hash', 'i', '418', 'destdir', 'some_string', '_type', 'date_', 'SOME_NUMB', 'rz', 'tgz', '*"', '_vars', '_M', 'pics', 'lav', 'STER', 'LIN', 'photoB', 'CPAN', 'directoryFor', 'fs', 'gy', 'VPN_CALLRECORD_20140409', 'ference', '.==', 'transfer', ']*){', 'x100', 'bridge', 'lastModified', '0D', '))/../', 'FULFILLMENT', 'UserKnownHostsFile', 'vator', 'FileB', ';<', 'bash_4', '1001', 's5', 'spl', '"\\""', 'quality', 'Out', 'way', 'hs', '/.*\\([', 'euler', 'gsed', 'm2ts', 'ah', 'xpg_echo', '}`"', '="{/', 'application', 'erred', 'DBPREFIX', 'zombie', 'ivL', ']*\\):.*$/', 'nohsib', 'PRE_', 'UI', 'unformatted', 'changes', 'dcraw', 'orig_dest', 'check', 'UR', 'dpre', 'Semapho', 'eth0', 'great', '|(\\./?).*|\\', 'posns', 'xvvf', ')\\(.', '/$/\\/', 'colrm', 'new', 'directory_name', 'jinfo', 'inputfiles', ':"/', 'NET_IP', 'NPID', 'bab', 'Sub', '1670', '150', 'Ite', 'FFPC', 'from', 'fD', '!.', '–', 'lea', "]*$//'", '.;', 'help', '"#', 'ur', '"="', '5', 'Result', 'Gen', 'folderToCompress', 'fixname', 'CLO', "]++'", 'etag', 'myCommand', 'illa', 'disk0s2', 'Expected', ')]*\\))', '20140624', 'xBE', '\xad', '"^\\+"', 'MAIL', 'ek', 'Lost', '50105', 'DiskName', 'tmp_dir', '85', 'little', 'avh', 'CONFIG', 'ARCH', 'document', 'exten', 'licit', '("%', 'fgh', 'nIP', 'qual', '78', '{/', '")"\'', 'dosetit', '+=$', 'PROJECT_A', 'оп', 'makeall', '%', "'/,|%$|~$|\\.", 'saxon_docs', '}\\.', 'den', 'pax', '?\\|', ',;,;', '##*/', '4th', 'act', 'Hc', 'VERSION', 'killaf', 'ildir', "''", 'hosangit', 'WHATE', 'Exclude', '++`', 'Ti', 'sir', 'lock', 'nogpg', '\\;`;', 'I_C', 'Tar', '/~', '"*\\.', 'Pat', '0d', 'brea', '\\\\/&', 'ctl', '?/[', '1def2010', '}\\"', '126M', 'network_sync', 'BEN', '$@*"', 'Sfields', 'SS_', 'program', '1553', '|"$', '\\}', 'er_', 'mpirun', 'INPUT_LOCATION', 'movew', ')}}', '160k', 'MD5SUM_OPTIONS', 'aWY', '+=/^.*.', 'avv', 'list1', 'Workers', '2004', 'dis88s8d', 'hnd', '\\"{}\\"/../', 'chmod', 'xBF', '_in', ')"', 'PENWIDTH', 'window', 'RIl', 'zZ', '~":"', '/[()]//', 'myuser', '[`', 'forward', "'{}'.", '13', 'gz_', 'sysdep', '_to_', 'athabasca', 'tool', 'clu', 'FOO_BACKWARDS', 'rs', 'FileWithColumnOfNumbers', '/^\\./', '2cpio', '1_', 'ingfile', 'Ali', 'repogroup', 'string_to_find', 'shtml', '"*@*"', 'a', 'k2', 'Dir', 'temp2', 'eml', 'BOM', 'dir_to_', 'kins', 'Downloads', 'javascript', 'integer', 'atever', 'your_pattern', '.`;', 'mid3iconv', '1024m', 'p2p0', '","', '([^', 'tolower', 'asi', 'INTR', '0s2', 'newThirdWord', '_key', 'myhost', 'iA', 'UAL', 'known_hosts', '//[^', 'rvm', 'bkey', 'WEEK', 'ox', 'USE_RC_SUB', '8859', 'PROD', 'month', '"$(', 'javaUsr', '_for_it', '>"/{}"', 'yosemi', 'stools', 'bhajans', 'MacOS', 'EC', '_params', ')")"', 'Container', 'x2', '/.*$/&', 'non', '\\([', 'mp3_directory', 'S_PER_', 'colocated', 'STRING_', 'TIONS', '".+\\.(', 'mypathname', 'tach', '<--', "'\\\\\\$", '_and', '/{}"', 'LP', 'pv', 'symlink', 'amd', 'c16', '/^,/', '@/', 'compressedP', 'acquire', 'tacao', 'wor', 'tence', 'DIR_TEMP', 'oli', 'process_one', 'bar', 'rew', 'gc_', '"\\', 'es_M', '>.*/<', 'TERMINATE', ',:[^=]*=,', 'maxlife', "'.*", ',¤', '_vim', 'UED', 'BL', 'c2', 'jklfilea', 'file_1', 'verted', 'fsUp', 'mywebsite', '*/@', 'allexport', "'*/*", '(",', 'CT', 'myletter', '3306', '"+"};', 'mlin', 'tainer', '/\'"\'"\'/\\\'"\'"\'/', 'ksh', 'raw_folder', '/^\\:.*$/', 'changelog', 'TTY', 'ение', 'k6V', 'Filename', ')=', 'Sample', 'w3', 'removedPro', 'xzcat', '_TO_', 'd_file', 'json_encode', '_Com', ".]+$'", 'DBNAME', '\\}\\)./\\', 'dir_data', 'macosx', 'jas', 'MyHTM', 'rqyl', 'G_DIR', 'ooop', 'mount', "'^'", 'DING', 'Wee', 'spenx', ':.*', '".*?")', 'aluno1', 'FI', 'DITION', 'stringtofind', 'rfc', 'P', 'daniel', 'faker', '.,}*', 'mk', 'TeSt123', 'dbmspool', '722', 'apri', 'xback', 'ssword', "'.*\\'", '_folders', 'LOCKFILE', 'blacklist', 'no', 'utf', 'controller', 'F00', 'trunk', '=[^=', 'location', '_existing', 'gsub', '.]*\\)/.*', ']),', '".*\\', '77', '200000005q', '0000', 'access_log', "'='", 'Again', 'rbind', 'ub', 'TDO', 'blow', 'fmask', 'wholename', '\u200c', '50M', 'Gene', 'DOFILE', 'zz', 'not', 'vlc', '\\///', '-/-', 'pic', '540', '20ts', '45', '=\'[^"|', 'esac', 'profile', 'unequal', 'tok', 'VPN_CALLRECOR', 'myip', 'my_command', 'ot', '$/.', 'OUTPU', 'FF0000', 'additional', 'X_GNAME', '++-', 'plication', 'derby', 'proxyhost', 'PROG', 'promptvars', '!=""', 'Ri', 'k5', 'cifs', 'gradle', "/^\\.\\///'", '/\\-.*//', 'ALIASES', '\'*\'"${', '0777', ":@'", 'current_working_data', '20480', 'ForwardAgent', '??', 'gre', '_eng', 'zzz_pdfs', '}{\\"', '__pycache__', '":[', 'newerct', 'eldatei', 'myfile_split', "'^[^.][^.]*\\.[^.][^.]*\\.[^.][^.]*\\.[^.][^.]*$'", '2013120', 'BI', '_pattern_to_exclude', '".*\\$"', '(/.$/,"",$', 'orm', '75', 'ti', 'Programming', 'ui', 'BigFile', 'A_', '19929', 'build', 'README', 'thentication', 'offset', 'nvwrap', 'her', ',;', 'KM', 'weekly', 'directory1', 'brief', '55', 'im', 'cmdhist', 'NP', 'set', '="%"', 'count_em_', 'text3', 'at', "'*/.*'", 'of_line', '_copy', 'WORKSTATION', '.*\\(', 'rate', '"*.?', 'default_java_dir', '_file_', '`;`', "'\\\\|\\\\|\\'", 'dir_to_zip', 'indow', 'lang', '/]', '=.+', 'Parts', 'TEMPFILE', 'VARIABLE_NAME', 'FILE_PATTERN', 'myregex', '_int', 'XRES', 'ill', '_pipe', 'reinstall', '[/]', ':]]*$/{', 'Alternative', 'gG', 'tt', 'smc', 'dos', '_54', 'compressed', 'pvd', 'mh', '\\/.*$/\\', '@@', "*$//'", 'UEDP0', '\\+;', 'EO', 'StrictHostKeyChecking', ']*($', ')."', '=(╱', '}:){', 'ko', 'fam', 'lmtitle', '|\\.', '/$$', 'bt', '%/*}', 'expression', 'pxe', 'D_CMD', 'gitkey_rsa', 'uzn', 'targethost', 'older', 'dir_txt', 'xBA', ":]]*'", 'mycode', 'myname', '1nr', 'yl', '[^=', 'dfrequ', 'exclude', 'yuicompressor', 'NEWGROUP', 'reso', 'v', '\\\\;', '#<[^>]*>##', 'device_name', 'exp_to_find', '}".*"${', 'DU', 'Command', 'sha1sum', 'w', 'destination', 'uu', 'copy_of_dir_1', 'LFI', 'symdir_output', 'TMPDIR', 'long', '2ban', 'ies', '_machine', 'kun', '30000000', 'equal', '"*[!', 'music_files', '\\)/.*', '-%', 'ms', 'folder2_', '://$.', 'IMAGE', 'XXXXXXX', 'f50000000', '_custom', 'Co', 'mand', 'ango', 'fmt', 'error_log', 'file_of_line', '\\;|', 'gdbName', 'ok', 'Lou', '/\\.[[:', 'lithist', 'data1', '20131202', 'DATA', '~":"\\', 'stored_exception_line', "'^", "]}'", '"+$', 'Hi', 'pem', 'oop', ':]]*', 'expan', '_root', 'LF', 'AMV', ']"', ":'//", '[^:]*$//;', 'alldata', 'аргу', 'idge', 'gconf', 'LC_MESSAGES', '134', '#\\\\#\\', '_US', 'mss', 'gen_root_dirs', '),', 'zsh', '"[/\\\\', 'myldm', "*\\#'", 'ale', '408', 'athaba', '::")', 'LOG_FILE', 'Binary', ',";"', '@[', 'euids', '([', 'prefix_', '‘;’', 'gw', 'removedProjs', '>&', '24000', 'ltq', 'SP', '+$/,"', 'lsof', 'ExpectedResult', 'ppa', 'SECON', '_S', '":"', '*$/;', '1', '40001', 'Model', 'human', 'Ex', 'dsa_key', 'beco', 'SSHKEY', ']\\+\\)', '000', 'listing', 'rpmfusion', 'XRESIDENT', 'Us', 'File1', '[@]}"', 'INTERPRETER', '-*-', 'nez', 'Jun', 'rest_cmd', 'whereever', 'tc', 'provides', 'nume', '=]*=.*/\\', 'orig_name', 'textToSearch', '=@', 'rmin', 'dt', 'scsi', 'oraenv', 'ints', 'image2pipe', 'dirrectory', 'Fri', 'FilesDv', '744', '\\"{}\\"`"', 'dfsg1', 'Spa', 'IGN', 'shadow', 'iew', 'ju', 'sFile', '_DAYS', 'c36', 'githubusercontent', '=][^', 'nas', '"\'"$', 'f54te', 'RDB', 'orint0', 'mykeypair', 'that_wri', 'Converted', 'chr', 'FOLDER1', 'baseline', 'rsyncuser', 'mpc', 'tecmint', '$@', '105', '>', 'path_to_dir', 'xhost', 'dirt', 'ftime', 'vcsup', 'the', '347340107', '|.*/\\(.*\\)/.*|', 'cms', 'Interest', 'quantifier', '^@', 'mtdblock3', '_interruptible', '#\\(', '++*"', 'rules', 'XTIME', 'BUG_OPTS', '5000000', 'tmp_file', 'HST', ']*).*/\\', 'pected', "'*/\\.*'", 'xbm', 'terSet', 'sHostname', 'edited_', 'bass', 'cscope', '+$/,"")}', 'FOO', 'nohup', 'plain', '86', "'..'", 'foxpro', 'exclude4', '"*', '>;<', '"${', 'eserver6', 'Testing', 'fR', 'nded', ']', 'SUBJECT', 'anythingElseIwannaIgnore', 'alFt', 'jdk', 'algd', 'pair', 'sym', 'a1', 'HostsFile', 'bundled', 'ball', 'TENT', '\\”\\\\', '\\*', 'myVariable', 'robo', "='$@$:'", 'Little', '24000c', 'THO', 'TEST_3', ':[', '4t', 'mysy', 'your_e', '67125', '.%', 'hsx', 'cel', 'play', 'time_results', 'sync', 'djangoapps', '201', 'REPLACE', 'dirs_to_remove', 'teach', 'gtest', 'BSD', 'TERMIN', 'DELETEDIR', '後', '_dir_path', 'pdu', 'Msg', '4dc', '2490', 'P255', 'Aa', 'XPRIORITY', 'inkscape', 'host2', 'paths_to_', 'PRETER', ',^', '="\\[', 'fhost', 'vcf', 'fied', 'vo', '`:\\!>"', 'noverbose', 'ck', ')}', 'твия', 'bir', '_source', 'point', '/\\(.*\\)/\\', '_user', 'alR', ']*\\([^.*/\\', 'spugD', 'user_name', 'append', 'ipc', 'floss', 'power', 'Te', 'ipcs', 'PROBES', '"^")', 'arkona', "'/[<>]/{", 'myfile', '#<?', '/${', 'java', "[@]/%/$'\\", 'verbo', 'L_VERSION', 'gs', 'TODO', '2_CMD', "'([[:", 'cup', 'regexp', '-\\*\\.', 'Please', 'yn', '1028', 'ow', 'libx2', '$|\\.', 'reli', '1073741824', 'SharedSup', 'tmux_command', '/^-\\', '\'"%', '_contents', ';"\\\\', "\\///'", 'oldname', '/^\\|', 'erb', 'tr', 'SIMPLE_OPTIMIZATIONS', 'ard', 'components', 'Shell', 'loads', 'ARGV', 'fakenfs', '4p', 'FileWith', '$.', 'Permission', '"\'"\'\'"', '14313', 'uploads', 'ITM', 'really', 'object', 'cfvz', 'thist', 'dEO', 'REGE', '-[', 'SrvDir', '5a', 'tHello', "\\\\|\\'", 'some_delimiter', "/^..//'", '79', 'abs', '"{}.', 'rans', '.*(', '+=', 'thingElse', 'questi', 'SECT', 'EPS', '}):\\', ">'", 'escape', '1s', 'hostname', 'Microsoft', '=":"}', 'FTIME', 'urpo', 'KEYDIR', 'pdfgrep', 'omn', "'&&", '/[^(]*(\\]*\\))', 'map_me', 'ipa', 'email', 'myArray', '".*', 'Sat', 'urn', '\'`"', '100x100', 'outputXY', 'FILESYST', '/\\*\\*/', '/.\\*', 'pne', 'parse', 'ding', 'LC_C', 'mzml', ']*/,"");', 'te', 'recur', 'stripped', 'WSFY321', '>"$', 'compress', 'bash_logout', 'Screenshots', 'STRING_TO_SEARCH', 'rX', 'n0', '001_article', 'STRING_HERE', 'ARY', '][^/]*/\\.\\./', 'some_user', '`.', 'gnome2', 'phantom', 'blah2', 'foofolder', 'htmlguide', 'findme', 'wid', '#*', '_to_keep', 'AU', 'zy', '2TB', 'RELEASES', 'sxw', 'pemkey', ']*=/', 'md5sum', '..', 'avoid', 'spend', 'suspend', 'dst', 'Agen', 'Sou', 'stringWithOrWithoutSpacesToFind', 'l500000', 'p1', 'REMOTE', 'mas', '"$/..', '/.*$/$', '~"`', ']\\)/\\', '*[[:', '\\<', 'w1', '--]', 'onda', 'Direco', 'alias', 'TR', '\'[|"]\'', '_SID', '6u2', '103', '450', 'tl', '5678', 'Flash', '<"$', 'status', '1445', 'CAT', 'processor', ",'", 'received', 'WHITESPACE', 'earnestc', '$"', 'nrz', 'Temp', 'fNR', '/\\[\\', '=$$', 'Up', '\\[', '/$/\\', 'ja', '_source_folder', 'USE_RC_SUBR', 'fileco', 'theDockerContainer', "/:.*//'", '$\'"\\', 'BACKUPNAME', '/\\(', 'win', "/^.*=//'", 'chsh', ']//', 'PORT_NUMBER', 'newtarget', 'lpr', '="{}";', ',<', 'Xcode', 'objectdb', 'Sec', 'remod', 'DSA', 'asm', '664', 'LearnAsSpam', 'phrasse', 'epel', 'Cur', '!/', 'LaunchAgents', 'conv', '`/', 'account', 'dirrmtry', 'remi', 'beispi', 'caniwrite', '/&&', 'somedirA', 'n99999999', 'EDITMSG', '/\\.\\(', '1_amd', 'mcompany', 'some_text_2014', 'Music', './..', 'pcpu', '800', 'thes', 'XXXXX', 'CMD', 'my_driver', 'Sy', ']\\+\\).*/\\', 'halt', 'BY', 'proces', '300x300', '(%', '804180', 'PATTER', 'dup', 'fps', 'keypress', 'CurrentLo', 'ask', 'action', 'libgtest', 'TEMP', '/`', 'agrep', '50000000', 'm4a', '20s', 'homepage', 'ncy', "~')", 'AS', 'mymodule', '23536', 'oravl01', 'vH', 'August', 'sourcefolder', 'ode', "'{}.", '"*~', 'tes', 'fier', 'lpe', 'modu', 'pst', '**.', "'*-*'", 'yyy', 'groovy', '/**/', 'empty', 'real', 'STAR', 'trip', 'el', 'wd', 'pV', '\\/\\*\\*\\/', '33', 'e0', 'cpuset', 'dax', '2005100', 'tached', "}}'", 'ora', '{}.', 'Manufacturer', 'configfile', '0L2', 'dani', '1005', 'wildcard', '-*.', 'lity', 'nasa', 'rootfs', 'tQ', 'excluded', 'destPath', 'svnadmin', 'project', 'vv', 'ools', 'tent', ']*\\)', 'fastcgi_params', 'servi', 'socket', ')', 'zcat', 'clusternode', 'IMAGE1806', '%"', ':/', 'VPN_C', 'archiv', '/,"', 'ci', ']&&', 'te_log', 'mv', 'mory', 'ldconfig', "\\)'", 'penultimate', 'destDir', 'ifprint', 'kaj', '4129', '%%%%', 'X', 'hosts', 'Teste', '_root_dir', 'assuming', '+)/$', 'phantomjs', 'REENID', 'not_equal', 'usic', 'bashrc', '}";', 'nas01', 'dr', "*/'", 'COMING', 'tory', 'It', 'Resul', '|"]\'', 'pn', "///'", '.$"', 'someone', '????', 'documents', 'otherHosttunnel', 'myServer', 'rw', "~'", 'nonsense_dir', '/.*$::"', 'apa', 'ism', '8859P1', 'accept', 'dev', 'depth', '1plus_build5638', '>//', 'x4e', 'archives', 'dis88s8dsf8h8hsd8f', 'allsize', '3q', '8d', 'bsd', "??'", 'usernmae', 'find_', 'gimp', 'your_env', '/', 'yourFile', 'su', 'Control', 'grub', '265534', 'axfr', '\\{\\}.', 'antiword', 'srand', 'WHI', 'uments', '11383', '30', 'virt', 'original_dir', 'ltr', '/**/*/*', 'jsonpath', 'class', 'BAR', "'/^\\", '<]', '1L', 'ext1', 'xz', 'smb', '"("', 'jdwp', 't5', 'hpp', 'cool', 'httpd', '\\/\\\\.', '786q', '314', '72', 'vw', 'sender', 'validate', '^', "'[^:]*$'", 'ABF1_', '2pdf', '").', 'S', 'load', 'L10', 'bwlimit', '61', '/\\/.', 'LI', '/../../', '\\*/', '\\|\\(.*', '#\\|^\\', 'KIP', 'name_of_', 'knowncontent', '1t', '_list', 'dpreferences', '_FILE', 'addresses', 'synce', ';"\\\\=?', '$}[', 'new_hard', '18892', '=$/$;', 'crontab', ':)[^', 'MySQL', 'upstart', 'uni', 'padluv', 'Wor', '={*.', "/*'", 'arq', "\\'", 'novideo', '201406', 'COUN', 'prog', "'{$", 'czfP', 'DC', 'largesplit', '?-??', 'fail', 'tml', 'terval', "'[]'", 'EDI', 'nf', 'teste', 'md5', 'опция_', 'FO', 'urv', 'file_to_exclude', ":]]*$//'", 'culation', 'always', "/$'\\", '_src', '[', "'.\\|./.", 'jw', 'each', '#*/})', '"]~/[', 'too', 'reference', ',\\”%', 'LOGDIR', '|[', '/*/;', 'hal', ".'", 'stuff_you', 'Templates', 'removestr', '[\\', 'ain', 'hy', '4', 'fstab', 'OPTARG', '+"\\%', '+/,', '50c', 'o1', '/\\\'"\'', 'TYPES_RE', '50000', 'attr', 'vm', '_pid', 'pavd', '127', 'samefile', '1200', 'psv', 'alright', 'test', 'diff', '///;', 'than', 'X11', '&$/\\', 'pattern_to_search', '(.*)/$|──', '#\\/.*#\\', 'rectFile', 'gotta', 'infile', "'.*[^", '`:', 'newdir', 'usi', 'OUTPUT_FILE', 'kundendienst', 'snvf', '_SU', 'num', 'SER', 'yourserver', 'myLog', '(/"/,"",$', 'tftp', "'*.[", 'K_VERSION', 'Exist', 'mes', 'libgl', 'projects', 'apr', 'D_20140409', ']+', '"^"', 'dbg', 'DATE', 'inks', 'CI', 'rsync_src', '’', 'jenkins', "'(\\.", '12u', "\\.[^.][^.]*\\.[^.][^.]*\\.[^.][^.]*$'", 'tnt', 'somepath', 'dotglob', 'mR', '_json', '#.*\\(\\.[', 'replaced', 'target_PWD', 'chel', '3iconv', '"{}:', 'aov', 'nfilter', "'^.*/\\.\\", 'ignore_readdir_race', 'srcdir', 'GT_', "$'`", 'Ignore', "'.*/\\.", 'userxyz', "/.*'", 'UID', 'Mac', 'tcl', 'internal', 'Known', 'pdv0', 'norc', 'Symfony', 'uter', '/\'"\'"\'', 'erminal', '&/\\', 'setarch', '--------', 'Time', "\\}//'", 'bsomeknowncontent', 'PIP', '????????', '+%', 'galle', '^$', 'candidate', 'qualidade', '“\\$', 'inval', 'quote', 'ARCHIVE', 'reply', 'yY', 'ntest3', '/\\(.*\\)\\.', 'inx', 'jpg', 'ispi', 'Am', ':]"', '"^`', 'wars', 'verbose', 'luv', 'sole', 'libEG', ']+).*/\\', 'blabla', 'FOR', '20120805', 'TMP_FILE', '={}', 'pkg', ',\\.*/\\', 'hw', '>#\\', 'tions', 'ditor', 'yourcommand', 'INFILE', ':"', '10c', "]}}'", '\'*\'"$*"', 'uptible', 'filea', 'iH', '|",', 'packet_loss', 'subdir1', 'oravl', 'swp', 'hostName', '6b', 'rp', '#--', 'udit', '"\'/', 'ING', '”', 'rails', 'searc', 'c19', 'file_count', 'LBU', 'dir2', '="/', '<<<', 'Ssh', '0bee89b07a248e27c83fc3d59', 'Data', 'execdir', '/"/\\\\"/', 'derful', 'tgt', 'sionName', 'ro', 'createDatabase', '20000000', "='`", 'fileTo', '@/[^/]\\+', 'beta', 'Friend', 'temp__', '{{', 'W1', '\\}\\).*/\\', '/^[', '$/;', 'f9', 'Copy', 'chars', 'hoo', 'album', '()<>&*|$', 'errors', 'vazZ', 'sdc1', "'[[", 'gid', '("[', '_demo', 'file_change', 'tracelog', 'comm', '[^/', 'cover', '<<\\', 'deddir1', 'D_DATE', 'kon', 'ccess', 'noreply', '(-{', 'DEST', 'filename', 'dis', 'sh_lock_file', "'{}'\\;", 'T_NUMB', 'vagrant', 'input_prog', '838', 'PAR', 'HR', '$\\>/"$\'\\', ")}')", 'PATH_TO_LOGS', 'mHello', '/^...', '}[', '(/^[', 'r', 'ignore2', ':*"', '\'*[+{;"\\\\=?', '#...', 'o_any', 'foo_fn', 'orrow', '*}', 'cmn', ')")', '}}"', 'X4', '[^[:', 'coffee', ']/}', 'FOO_NO_LEAD', 'folder2', 'nodejs', 'b07a248e27c83fc3d59', 'DO', 'UNDOFILE', 'my_wonderful_sed_script', '\\$#\\', '))"', ',/^$/', 'gi', 'hrt', ']*/,', 'varado', 'USE_', 'b07', 'rex', '()"|"$', 'FOLDERS', 'unexec', 'waf', 'null', 'dragon', 'boost', '16', 'bbbb', 'none', 'alle', 'dernier', '=`/', 'M', 'iting', 'trant', 'FILESYSTEM', '173', 'nemcompany', 'be_replaced', 'SSHKEYPASS', '置換', '[^\\.', 'outputs', 'UUUUUUUUUUUUUUUUU', 'zcvf', 'DOM', 'topscripts', '94m', 'rame', "##'", 'side', 'MAT', 'gnuplot_cmd_', 'shere', 'n_max', 'St', '_re', 'emp', 'splitter', '235p', '*-*$"', 'db286', "'/'", 'nombre', '/(*)\\\\', ')))/', 'mp3lame', 'lver', ';!{', '752010', 'configure', "/{}'", 'god', '#;', 'yourword', 'apparent', 'uwin', '{$,="\\', '50k', '56_release', 'exp_to_find_in_folders', 'IFO', 'privatekeystuffdis88s8dsf8h8hsd8f', "$':", 'pathTo', 'maile', '2q', 'xxx_', '}\\.[', '_do', 'passw', '}\\(\\:', 'ConnectTimeout', '/\\/\\/', '/\\.\\/\\.', 'P10', '/({', 'Symfon', 'suffix', 't2_t', 'ia', 'thing', '=~/', '_h', 'wheree', 'games', 'tbz', '2000000c', 'exlude', '([^)]*)$/\\', 'hot', 'backup_path', '="./";', '070', 'archive', 'directoryForTransfer', 'exmk', 'REGEX', '_LOG', 'мен', 'EOL', 'END_GROUP', 'вия', 'AUTO', 'EU', 'sflow_log', 'flow', '"*$', "'.*'", 'Basic', 'Temporary', 'set1', 'cmdstr', '_to_replace', 'whatchanged', 'ng', ");}'", 'Jour', 'chStr', 'Are', '\'"$', 'MIN', 'wef4', 'dle', 'jce_po', '1q', 'HTTP', '…', 'ubso', '/^#', 'REQUIRED_FILES', 'pos', 'conf', 'und', 'You', 'nl', 'Loader', 'dg', 'logcheck', ',"', '0123', 'sessname', 'wait', 'MCAD', '/://', 'ni', 'rtf', 'avel', 'ttify', '/^.*[/]', 'R22', 'pgr', 'libmp3lame', 'goneer', 'jobs', 'substr', 'BGG', 'commandfile', 'txt_bak', 'ORIG_DIR', 'sy', 'xon', "'\\./[", ']+$/', '/="\\', 'PHANTOM_', 'somewhereelse', 'iB', 'Iw', '(/', 'Yes', '2days', 'userid', 'fd', '|\\', 'FILES_TO_PROCES', 'z0', 'TARGET_PATH', 'App', '14ubuntu1_amd64', 'Y', 'durchsuchender', '>|', 'srchfor', 'MAPP', 'dig', 'original', 'cope', 'CopyTO', 'ml', '/^/"', 'Forward', '‘*.', '#")', 'cart', 'SS_FILE', "'.[:", 'eth9', 'ldt', 'anotherhost', 'F', 'MOD_DATE1', '10p', 'which', 'hat', 'MALA', 'ACT', 'dge_', 'secret_file', '_rememb', 'socks5', 'tta', 'fv', 'PING', 'skipdir2', 'netac', '/^\\(.\\{', 'pte', 'andom', "'&&'", 'rol', 'disabled', 'apps', 'item_2', ']+-){', 'ASCI', 'vendor', '*/*', 'sent', 'date_time', 'STrans', ',${=;', 'six', '-\\(', 'insert', 'numbers', '8999', 'e89', 'www_new', 'git', 'compute', '",",', '>{}</', '\\(.*\\/\\)\\(.', 'mar', 'adress', 'inputfile', 'Expansion', ')\\.', 'ilder', '=.', 'ProcessFile', 'next', 'tobeco', 'HU', '\\)\\.[', 'INPUT_FILE_NAME', 'MA', '“', 'uf', 'iface', 'agen', 'dirname', '08', '/[^(]*', 'rf', 'pwi', "='\\", '/[^', 'success', "'/#", 'Match', 'erated', 'datei', 'QTDIR', 'Mo', 'weservice', 'ify', 'LD_PRELOAD', '->', "'%", '\\;>', '$))/../', 'rc', '&/', 'youtub', 'n256', 'f', 'wtmp', 'w4', '__temp__', 'csup', 'Compression', '/:\\/="\\', '/**', '\\///"', '}>', '/:', 'T_YOUR_', '\\\\*~', 'xvfJ', 'TODAY', '="|', 'ud', 'CA', 'Y_', 'all_html_files', 'DS', '\\/].*$/\\\\', 'ITER', 'binary', 'file001abc', '+\\.\\', 'Unix_', 'don', 'replace', 'Chapter1', 'tzvf', 'olderthan', 'zv', '/../', 'TARGET_DIR', 'ABLE', '87', "/***'", '2014', 'searchName', 'vivek', '????????????????????????????????', 'NR', 'Iseconds', 'newerm', 'inet', 'somedir', 'apache_user', 'gro', 'ost', 'Access', '"/', 'x86_64', 'sc', 'YNC', 'kefile', 'expr', 'jsp', 'emails_', 'Err', '"--------------------------------', "'*/", 'e', 'orak', 'STARTDIR', 'PI', 'update', 'uid', 'di', 'properties', 'bzip2', "/\\//'|", 'serves', '(||', 'GA', 'itect', '1316256', '.*$/"/', 'fu', "*$//'`", 'ncccccccccccc', 'itest', 'gur', 'qr', 'arg1', 'ho', 'yourstring', '"$/', 'self', '(!', ',:', 'pathfolder', 'jz', 'DIR_TO_', 'ru', 'provi', '"\\\\\\', 'exp_to_replace', 'over', '-((', '3c', 'kundendien', '_APP', ']*\\).*$/\\', 'File_', '(.*\\.', 'oav', 'mt', 'Incoming', 'tape', 'goodb', '.]+', 'sfolder', 'ecmint', 'logo', ':`', 'seam', '+-\\', 'XXXXXXXX', '_directory', 'zvr', '5100', 'favicon', ']"))', ',/[^/.', 'mvim', 'par', 'info', 'chro', 'Bdisk', 'MacVim_source_folder', 'TT', 'mongod', '\\', 'kConfig', 'of', '_to', 'yaffs2', 'website', '&;', '"[$', 'inals', "'/$/", 'xzvf', '置換後', "$)'", 'done', "?'", 'vars', '}"`/`', '(?', 'share_folder', 'Connect', 'keyfile', ')-', 'folder1_', 'actual', '$/{', 'STRIN', 'pdm', "\\)$'", 'myCleverScript', 'otherUser', 'usb', 'aeiouy', 'fromdos', 'tents', 'remoteToRemote_IP', 'accepted_hits', 'EN', 'ma', '=$"', 'iter', 've', '"/\'', 'eck', '"^.*~$\\|^.*#$"', 'blanktest', "')'", "'@'", '/\\\\\\/\\', "'.*\\..*'", '"/,"', 'mksdcard', '\\+', 'ela', 'input_file_name', 'yml', 'WHITE', "'/^$/", ":]]*$'", ':;', 'the_stuff_you', 'FOO_BACK', '):', '!=', '"*:*"', 'musicuser', ']”', '_port', 'ln0e', 'mobi', 'bsome', 'Linkin', 'js', 'Bca', ',|,', '73', 'tdn0', 'MON', '}[\\.]){', 'Error', '/^...\\.*/\\', '<==', 'YOUR_', 'ful', 'backups', 'ted', '/^(.*\\/\\.', 'ControlPath', 'systemctl', 'deslash', 'client', '...', 'DI', 'por', 'XCPU', 'database', '../../', 'nk2', 'release', 'accessed', 'PRELOAD', 'libsomefile', '?].', 'options', 'wc', 'w12', '"./[', '\'"{}"', 'prefe', '128128', '760c', 'svg', 'remoteuser', 'е', 'Me', 'ch', 'scoop', 'dirs', '".+\\.', '../<', '\\?"', 'Saf', 'down', 'zcp', 'latest', '"{}"\'', 'remoteip', 'N1', 'SECONDS', ';/^.\\{', 'etime', 'dska10', 'yuv420p', '1024k', 'bulk2', 'newparse', 'empname', 'Project', '_START', 'outdir', '3_t', 'OLD_', '_TEMP', 'XXX', 'mjpeg', 'D', 'pidof', 'store', 'dave', 'destination_dir', ":]'", 'dog', 'server_id', 'rmt0', 'rwx', '555', 'Iwanna', 'OOL', 'inside', ';’', '])|', 'anti', 'vbox', 'SpoolIn', '/\\.\\', '-{', 'RECOR', 'enablerepo', 'mer', 'Line', '\'"${', "';", '/%', 'xtrace', 'XUSERID', 'markdown_github', 'aeiou', '\'\\\'\'"', '.$', 'DBA', 'ras', 'shel', '500M', 'miter', '>-<', 'Rr', '1782', '_To', 'flu', 'btsync', 'PS4', 'package', 'XY', '_real', '"^[', 'exclude_pattern', '置換前', '".#"', 'fglrx', 'remount', 'Lit', '"$@', "/\\.$/'", 'oBar', 'mtdb', 'ic', "&/'", 'misc', '_working', '.*$/', 'yui', '{}"\'', 'Hvz', 'keygen', 'ory', '56789', 'OFFSET', 'har', '//=@', ");'", 'x0c', 'uk', '_LRG', 'usr', 'win0', 'Nn', "','", 'myLogFile', 'Strict', 'drag', "'\\[.*\\]'", 'my_temp_dir', 'ugD', 'middle', 'eW', 'dbca', '?")', '="`', 'newline', 'IT_', 'MD5SUMS', 'version1_', 'G1806', 'setit', 'sftponly', 'sha1', 'orap', 'v_', '8f', 'filedumper', 'EXTENSION', 'catalog', '}".*', 'answer', 'exampleuser', 'OBJECT', 'JBO', "=\\)'", 'DAILY_TEST_FOLDER', 'neww', 'nce', ']\\/[^/]*$/', 'On', '$\\)/\\', 'Ii', '/.*', 'table', 'SOME_SITE', 'paste', 'kp', 'sqlmaster', '‑', 'config', '\\.[', 'readable', "'</", '\\$/', 'audit', '\\)', 'pwd', 'uname_m', 'days', 'tible', 'xpilot', '002', 'ravi', 'banana', 'baume', 'u01', 'board', ');.*\\?>"', 'w32', 'mailx', 'eger', '=$()', '\\{', 'cassan', '@"', 'ccc', 'hogehoge', '\\//,"', 'remotehost', '=$;', 'nfs', '_sync', 'cor', 'V_', '6M', 'wwwrun', 'l12', 'iq', 'webdir', 'source_file_', '312', "'.*\\.*'", 'tempfile', 'FirstFile', 'NOP', 'ProgramFiles', 'export', 'aif', 'FOL', 'Com', "’\\'", 'qfi', "'/^[^\\", 'creationix', 'pberrypi', 'compressedfile', 'staller', 'finalName', 'vd', 'unca', '/;/\\\\;/', 'final', 'DIR_DE', 'q0', 'dC', "]'`", '7430', 'etLo', 'host', 'dir_to_start', '\\}\\)', 'O_', 'lm', 'hexdump', 'HER', 't1_t2_t', 'phen', '-*', 'YST', 'File', 'complete', 'pcapps', 'xCA', 'cs', '"?"-":"', ".*\\)'", 'shp_all', 'colocatedserver', 'tf', 'dur', 'upvoter', 'Y_GNAME', 'Semaphore', '}):[^', 'LOC', ':]]+', "]*($|/)'", 'rsh', 'copy_of_folder1', '20000k', 'ú', 'texture', 'active_record', 'STORAGEFOLDER', 'tx2', '1700', 'precise', 'ALCASE', 'EXIT', 'и', "//'", 'hc', "]/'", 'xr', 'rh', 'Park', 'ano', '12345', '.*$//;', '/[()', 'libx264', 'art', 'produ', '+=`', 'cations', 'Owner', '/.*/\\”&\\”/’', '\\"";', 'harry', '/\\{\\}', "/'{}'.", 'FindCommandExamples', 'js_output_file', 'PATH_TO_OUTPUT_FOLDER', '/\\.$//', "':'", 'encoding', 'GHBAG', 'trans', 'BS', 'you_search', 'MyHTMLFile', "'-'", '#^~#$', 'display', 'groups', 'esa', 'nbconvert', '$:', 'http', 'urllist', "$\\)'", '_any', '++]=$', '|', 'other', 'им', 'EDT', '200557600c', 'aluno2', '\\$/\\', 'park', '\\./\\.\\*', 'jzb', '2007', '-/,"', 'separate', 'Kowalski', 'a24', 'cassandra', 'Linux', 'vt', '={', 'fprint0', '_unknown', '1tr', 'LIM', 'deploy', "'/@", 'someletters_12345_moreleter', 'bob', 'home', 'recalcitrant', 'lastpipe', 'FILE', '37', '_64', 'file9', '06d', '_OP', 'luke', '123456', 'FOOBAR', 'cl', 'bri', 'WHATEVER', 'WIDTHS', '/,//', 'mynewtitle', 'sourcedir', 'ssh', 'pete', 'sca', 'queue', 'malloc', 'S_RE', 'f2', '();}', 'p_build', 'N_C', 'R_USER', '_dir_1', '.\\', 'UG', 'relative', 'symlin', 'Status', 'dbc', 'lass', 'tial', 'Text', '2009', 'TS', 'Books', '"),', 'Stor', 'l100', 'jhai', 'nouser', '.]*"', "`'[[:", 'sup', '8n', "##*.}'", 'sdcard0', 'log_errors', 'Cpu', '+"$', 'webshare', '300000k', 'calMachine', ';$!', 'scalar', 'path_to_', 'NameTo', 'crip', 'starting', 'from_dir', 'OT_DIR', 'DOGS', 'env', 'OUTPUT_', '/\\\\', 'XX', 'pingResults', 'word1', 'ilProgram', 'HOGE', '")', 'ignore1', 'excludedir2', "'{", 'wn', 'STRING', '5s', 'DiG', 'outdated', '/^[^/]*\\', 'STORE', 'variable', '-??-??"', 'utility', 'dif', '4000m', 'Interval', 'online_admin', "'*-*", 'searchpath', 'pand', 'LearnAsHam', 'DS_Store', 'wobgalaxy02', '70', 'stalle', 'dirln', 'DELETE', 'pts', '-)', 'c1024', ':",', 'nr', '1__CUSTOM_', '{}"', 'tmpoldfile', 'killall', '#/', 'LittleFile', 'bananas', 'htaccess', 'VE', '51', '?^?/[', 'QU', '750', 'videos', '76', 'relea', 'filelist', 'Where', 'yourscript', ']\\+\\)).*/\\', 'ib', 'googleTest', '"*.???"', 'Targ', 'XXXXXXXXXXXXXXXXXXXXXXX', "'([", 'dl1000000', 'XUSER', 'fstype', 'trace_list', 'tota', 'ggen', 'vmwa', 'theDo', 'ARKER', '|/\\.', 'jkl', 'cpuinfo', '1thumb', '\\!>"', 'stream', 'audio', 'yosemite', 'suid', 'serveripa', 'localinstall', '","}', '_to_stdout', 'birthday', '15', ']*`', 'demo_bkp', 'ring', 'game', "'\\.[^\\.", 'ote', 'myNew', 'unformat', 'fileA', '[^:]*:*/', 'remote_file', 'old_nullglob', '\'"\\', 'RO', '$/<--', 'tps', 'uld', '{})', 'XST', 'newSecondWord', 'hashed', 'PREFIX', 'the_command_i_still_rememb', 'Música', 'F1_', 'eo', 'texthere', 'jumps', 'bc', 'FOO_', '=""', '")/', 'UU', 'my_teed_file', 'examples', 'part_of_the_command_i_still_remember_here', 'lSh', '0c', 'response', '$?', '/)', ']\\.[', 'WHOLINE', 'DSAAuthentication', '/$(', 'sitename', 'CL', 'good', 'cdup', 'MPG', 'YOUR_STARTING_DIRECTORY', 'pad', 'STDER', '_lock', 'mission', '}"/*."${', 'testproject', 'my_cool', 'ator', "'\\''", 'my_iso_file', '_socket', 'vsz', 'DROP', 'gilds', 'Dropbox', 'MyCProgram', 'DRI', 'strftime', 'cker', 'SHELL', "='*", 'X11R6', 'machbook', 'apropos', 'phras', 'hda', 'mp3', 'dirstructure', '_Ver', '_2pc', '3d59', 'default_java', 'counts', '_all', 'gnuwin', 'netstat', 'minute', 'EXPR', 'yone', 'module', 'Jj', 'newEmptyPHP', 'uint', '%%\\', 'pankaj', '38', 'MKTEMP', 'somename', 'tern', 'prune', '/\\...../\\', "/\\&$/\\\\\\\\/'", 'media', "/'", 'AY', 'debug', ',/<', 'rwX', 'content', 'SERVER', 'cookie', 'teed_file', '968746', '430', 'tarball', 'ss2s', 'programname', 'global', '_20120821', '/\\.\\/(.', '2966', 'cb', '+\\-', 'unique_folder1', 'midi1', "*'", 'denied', "'[^[:", 'By', 'logs', '=`', 'PHP_EOL', 'bluecove', 'mtime', ']}"', '.*?>##', 'dbf', 'cache', 'follow', 'xlogs', '(\\]*\\))', 'silent', '_pdfs', 'Automatically', 'FFER', 'MAILBCC', '/^\\\\', "'[!.]*'", 'your_command_here', "'(?", '("-', 'splint_3', 'Username', '$/..', 'tunneluser', 'progcomp', "=//'", '_PATH', 'pong', '#"', 'prepare', 'dfgghrtgdg53', ')[', 'travelphotos', 'f12', '>;"', 'COLUM', '1000001', 'ExtJSLogin', "'$", 'destina', 'machnum', 'devnet', '89', 'CE', '-"${', 'BOX', 'o5', ']*\\)$#\\', '!~', '/.*$/\\', 'REQUI', 'Media', 'else', "://'", '012', 'rn1', ']\\.', 'myProcess', 'lastbatch', '686', 'newBase', 'fred', 'remote_machine', '/(..)\\', '",$(', 'baz', 'pam', 'word', 'LBUFFER', 'HWaddr', 'uUSER', 'prepen', 'bdump', '">"', '+/', '\\\\}\\\\[', '_SUBDIR', 'kt', '155', ";}')/", 'td', '"\'"\'', '/.*(', ',\\!', 'bang', 'picasso', '/$|──', '".*/.*', 'I', "\\]'", 'LDP', 'nRHI', 'proj', 'characterSet', 'lname', 'ora10', '"[[', '_ec', '}/^$/{', '_ker', 'full_f', 'HTT', 'length', 'scription', 'XSH', '_decode', 'Z_', '}"\'', '*\\)\\{', 'your_command', 'utshell', 'repos1', 'ка', 'path', 'ether', '\\”,|', 'kes', 'MIT_', ']*//', 'Ar', '()*', 'FF', 'jan', '64BIT', 'lob', 'extension', '6q', 'newermt', 'term', 'zo', 'txtlist', 'lin', 'my_daemon', 'hay', 'ils', 'Hd', 'checkold', '0755', 'yourproject', 'day', '1806', '725', 'CompareFile', 'vnm1', 'match', 'cosi', '99', 'h', 'whoami', 'Automatical', '+=/^.*', 'mtdblk4', '.{', 'whatch', 'P2', ',\\(.*\\)', '("\\.', 'RESP', 'BACKUPDIR', 'lspc', "'|", 'R22222', '\\|\\.', 'wri', 'orschiro', '"(', 'landof', 'my_dir', '1990', 'statement', '/#', 'existing', '/.*)', 'bam', '100M', 'sfer', "\\.*'", 'cts', ',${', '24', 'ablerepo', 'zip', 'ALL', ']]', 'smbfs', 'PDF', 'abspath', 'xt', 'Tested', '100m', 'LA', 'nline', 'allener', '*\\\\{', 'JBOSS_CONSOLE', 'fileB_', 'tac', 'two', 'outputXYZ', '$$`', '_loss', 'PHANTOM_JS', 'large_', '##"', '_MED', 'cre', 'watch', "'^($|\\", 'eng', '".*/.*\\.\\(', 'output_file', 'script2', 'ruv', 'vend', "/,$//'`", 'inner', 'devtoolset', 'rm5g', 'VAR', '676', 'xrange', 'FIEL', 'x9fa', 'selection', 'Tecmint', 'geoiplookup', 'Fir', 'gen', '\\(/.*', 'COLLECT', '_CACHE_', '194', '/--/\\', 'Maildir', 'your', 'PROJEC', 'Z_0', '15i', 'define', 'javac', 'pmud', 'л', 'dext', 'ces_relative', ']+)', 'fox', '("-",', 'PID', '(/"/,"', 'Jcf', 'ne', 'nameof', 'New', 'ач', 'lt', '**...*', '[(][)][[:', 'realtime', ':]]+$//"', "'[[:", '\\/$', 'ckTime', 'ig', 'NS', 'L1', 'dvportgroups', ']+\\.[', 'unix', 'checksums', 'etimes', 'CON', 'research', '#^', 'today', '\\”', 'ROU', '="@$', '20821', '".*/.*\\.', 'ext2', '-??"', 'b16', '(.*));.*\\?>/', 'terminated', 'argv', 'NID', 'absolute_path', 'vf', ':-.}', 'teste1', 'splint', "\\//'", 'removepat', 'NOPOR', '>|<', 'GIF', 'ANTOM_', '"(?<=', 'logoff', 'myUSBdisk', 'amount', 'mynew', 'somelocation', "'^[^.][^.]*", '"."*~', '()/,/^}/', '175', 'cvzf', 'snapsho', 'string2', '_po', 'aries', '40107', 'mkv', 'rsa', '/\\/,', 'sums', 'yourprojectfoldername', 'exclude_me', ',/,', 'subset', 'ex', 'Oct', 'subsubdirectory', 'pub', 'funcname', '#*/', '3v', 'myCle', 'tar_root', 'N', 'Srv', 'MYUSERS', '20131204', 'pat', 'forum', 'ISO', '~#$', 'C', 'cvs', 'ry', '10_Rec', 'ORAL', 'prune_me', 'FULFI', 'mach', '_TO_PROCESS', 'id3v2_version', 'GI', '#.*/##"', 'files2', 'ume', 'older_log', '‘{}’', '&"/\'', 'iI', 'magi', '420', 'newFirst', 'Or', 'MASTER_SITE', "+$'", 'replacement', '*\\)', 'htmlgui', 'CONT', 'new_dir', "'/./,$!", '/\\$/\\', 'sx', 'libvorbis', ');.*\\?>/', 'Musi', 'week', '999', 'edwar', 'cluster', 'FOLDER_ROOT', 'PRI', 'ali', "'#", ']\\+\\)\\..*/\\', 'xef', '_HOST', ')"\'', 'mysql_access', 'MOUN', 'nvraL', 'гу', 'lencode', 'asks', 'SSHSOCKE', 'bash_log', 'fast', 'About', 'direxpand', 'suming', 'myprompt', 'FAT', 'd2', 'apl', '_foo_', 'groupb', 'cour', 'QUEUE_PIDS', '\\;"', '1956', 'ABCD', 'arch', '666', "++}}'", ');.*\\?', 'SRV', 'rprivate', 'bower', "='+$", 'targetdirectory', '7c83fc3d59', '500k', 'OUT', 'mycomp', "'/\\.'", '#\\(.*\\)', '356', 'up', '1251', 'YOUR', "'.')", 'reen', 'json', 'OK', 'tofind', '/^([^', 'CK', 'Rv', 'AL', '$@)', 'new_directory', 'nCPU', '=\\*.{', 'pd', ']+\\.', 'blast', 'rtvpl', '\\*\\:\\*', 'Payload', 'rapt', '\\\\&/', 'opendns', 'Little_Com', '/\\"', 'tkConfig', 'zone', "/$/,/'", "}$'", 'race', 'imp', 'йствия', '\\”,|”%', '0k', '1024MiB', 'part_of_the_command_i_still_rememb', 'PathOn', ':|', 'pathToShell', 'Only', 'pattern1', 'nts', 'iz', 'hello', 'temp', 'ypatt', 'SEX', 'k8', 'ds', 'user', 'sox', '/$/', "'}", 'joblist', 'dum', '));"', 'xtype', 'junk', 'value', 'ROUTE', 'Applications', '300M', "/::'`", 'tex', 'eded', 'repository', 'TISER', ':~/.', 'Nad', 'SRC', 'maildrop', 'Safari', 'айла', 'api', 'lpi', 'mitime', 'harr', 'subl', 'lrt', 'popd', '3rdparty', 'cal', 'au', 'G4', 'aplugin', ':$/{', '>>"', 'hover', '1234567890', 'lhrt', 'fon', 'data_report_P', 'BO', 'hDaxd1', 'PathAndName', 'compiler', 'OUTPUT_LOCATION', 'be', 'TUAL', 'eserver', 'SSHPASS', '"{}"|', 'methodNameHere', 'OM', 'POR', '_12345', 'dos2', 'GN', '_dirname', '4t5teerdfgghrtgdg53', 'resources', 'Hostname', 'drt', 'undo', 'kv', '1024M', 'data_', 'linprocfs', 'inter', 'Grep', 'toprintto', 'ffs2', '"*-', 'Icons', 'curl', '(\\|', 'insr', '(/.', 'GNAME', 'nTeste', 'v1000001', 'sk', '/.*/\\”&\\', 'LAB', '$>', 'xy', '160', 'netaccounts', 'mysymbol', 'c8', 'cookies', '"{},\\!\'\\\'\'"', '770', '/**/.', '/\\.', 'Back', 'filenames', "'\\(.*", 'localuser', 'amp', 'whe', '0s', 'Poo', 'PATH_TO_FOLDER', '^.]*', 'top', 'bind', 'BASH', "{'", 'mnewer', 'STORA', 'excludes', 'raw', 'PIPESTATUS', 'tailCount', '\\.*/\\', 'ression', 'k', 'ASE', 'USR1', 'Lines', 'jar', '90', 'compressor', 'delimiter', 'letion', 'text', 'NF', ',/[^/.][^/]*/\\.\\./,/,;', 'czvPf', 'public', 'DISPLAY', '.${', 'TEXAMP', '||"`', 'htm', '_HOME', 'lpath', 'START', 'stream_get_contents', 'remote_server', '\\/', 'invalid_command', 'ppp', '=\\[', 'ON', 'stephen', 'myFolder', 'nAutor', 'sameq', '_release', '/\\$,\\', 'tvf', '0i', 'AssemblyInfo', '="\\[\\', "':", 'LGT_', 'x70', 'HO', 'std', 'sdt5z', '\\)\\(', 'node', "='+", 'oc', 'Po', 'VOB', 'heapdump', 'regextype', 'GET', '"\\\\', 'getent', 'pend', 'cycy', 'devicemapper', 'depsAlastmodified', 'devtool', 'del', 'paralle', '/.*|\\', '##*.}', 'Application', 'SOURCE_DIR', 't0', 'apresen', '2', ';}', 'MAC', 'extended', 'meth', 'REMO', 'new_group', 'tobe', '%/{', 'SPAM', '“%', 's3', "\\.'", 'Icon', 'known', 'ipynb', 'xterm', "]'", 'mssql', '_ocr', '="/"}{$', '\\)/.\\', 'cut', '_folder1', 'include', ']+-[', 'Subject', 'inner_dir', 'tition', "$@:@'", '"\\\\\\\'"', '|[/]', 'nect', 'Z0', 'tata', 'vmx', 'midas', '[^>]*>#\\', '1000k', 'old', '1494500000', 'b1', ']+/,', 'dbbackup', 'prac', '_one', '(/-/,"', '"`"', '220', 'mbox', 'deo', 'XXXXXX', 'ovBc', 'lent', '1440', 'FindCommand', 'org', 'Aut', '775', 'scriptDest', 'uid_demo', 'roid', '700', 'any_command_what_produ', '"{}")/..', '&&(||', '".*[/]', 'ument', "='", 'inary', 'BACKUP', 'LIENT', 'ELF', 'firew', "'<", 't2', 'pell', 'Fs', "[^.]*'", 'shots', 'try', 'ftp', 'DIR_TO_CLEAN', '3400', 'scripty', 'dups', 'EUE_PIDS', 'se', 'n10', "/.*/./'", 'STATUS', '%.', '(/^[^', 'ORITY', "/)'", 'Hni', 'avzn', 'JU', 'ol', 'iscsi', 'cp125', 'gcc', '/\\).*', '_script', '(*', '/\\&$/\\', 'licy', 'W', 'markdown', 'saxon', 'nline2', '^[\\', 'ts', '*".', 'rl', "=''", 'MASTERSOCK', 'afispugD', 'ORACLE_SID', 'oP', 'packagename', 'ref', 'xxd', '9781', 'u2080', 'nonExisting', "\\\\\\\\/'", 'sysPassword', 'reall', 'vid', 'INATION', 'my_cool_script', ',,', 'compiled', 'wsu', 'date_dif', 'source_path', 'icon', 'needed_dir', 'pment', '\\(\\:', 'file_', 'linebuffered', 'QMA', 'nslookup', '_SPACE', 'RHI', '\\|\\(^', 'опция_поис', 'BASH_VERSION', 'your_user', 'ning', 'FS_', 'funrol', 'doct', 'STRING_TO_', "\\.]*'", 'ves', '~', 'inarray', 'tesst', 'ying', 'cing', ".]*'", 'asorti', 'invalid_dir', '|^', 'fnNT', 'aHn', '$(($', '255', 'when', 'lit', 'have', '4dc9', '/\\(.*\\)+.*/\\', 'py', 'PROJECT', "':%", '10d', 'truncate', 'path_to_folders', 'nrk', 'working', 'someWhereElse', 'stru', 'regex', '_still', 'Pick', 'ted_', 'diffe', 'imagesdir', 'DRIVER_DIR', "'\\[.*", '\\/="\\', 'repository_dir', 'cps_build', 'cyhomepage', '"$"\'', '$(/', 'D44203', "'.*/\\", 'jail', 'pi', '_part', '(/|$)', 'adbkey', 'open', 'andrews', '_to_SEARCH', 'old2', "'/\\", 'simpleGrep', 'dalvarado', '1F', 'tobecopied', 'Manufa', 'fish', 'x_', '1a', '2000', 'kers', 'ou', ',\\}', 'x800', '_2014', '"$"/*', '1431', 'lp', 'amin', '[$', "/..,//'", 'xxxx', '(/|$)|/\\.', "\\'*\\'", '-)"', '0111', 'unit', 'Cookbook', 'broken', ']=', 'sa', '"?', 'array_sum', 'hv', 'nfs4', 'walski', 'na', 'dist', '167125', 'fpdf', 'Musik', '*=', 'secure', 'OM_', '200k', 'very', 'www', 'excludedir3', 'iregex', 'java_ee_sd', '0be', "'[()]'", '‘%’', 'dest', 'ush', 'gar', 'czvP', '=\\*.', 'bmp', '46', 'WS_', 'ct', 'total', 'wer', 'ered', 'ach', '8hsd8f', 'shp', '1US', '21204', 'Age', 'ku', '---\\', 'here', '<sos>', 'size', "}'`", 'avzru', 'bkup', 'any', 'mn1', 'TempFolder', 'write', 'pE', '(/^\\.\\//,"', 'fr', 'forea', 'fun', '0551', 'mM', 'macs', 'RC_SUB', 'tiff', 'DIR_TO_ZIP', 'execute_command', '="=";}{', '?.*?', '595a', 'ist', 'Q', 'some_script', '_LOCKFILE', 'your_text', 'gpw', 'A1', 'atus', 'console', 'das', '\\+"', 'XVIRTUAL', ']*\\.', 'ample', 'mirrors', 'lookup', 'A10', 'Magento', 'forever', '31', 'stringYouWannaFind', "'(?<==).*'", 'yea', 'libs', 'joo', 'ker', 'Articles', 'Disk', 'fg', 'remotely', 'pvdmu', 'PyErr', '?}', 'FFIX', 'wp_version', 'yourRemoteDir', 'FS', 'LaunchAgen', '&";$\\="\\', 'conda', 'оис', 'From', 'NGV', '$."', 'No', 'vor', 'pma', 'complica', ')";', 'timestamp', 'FILE_', ']$//', ',/[^/.][^/]*/\\.\\./', 'keystuff', 'LE', 'Script', 'cripts', 'pth', 'ima', ')!"', 'sql', 'VLABEL', 'tempname', "'##'", '/\\\\/.*$/\\', 'nels', '10001', 'adata', 'cè', '/){', 'l56', '\'*[+{;"\\\\=?~()<>&*|$', 'tran', "}';", 'The', ':]]+\\.[[:', 'ep', 'to_sort', '91', 'oraclelicen', '700k', 'idecode', ',*/', 'lename', 'life', '[^"]*"', 'my_db', 'tablename', 'peter', 'ils_', '\\”%', 'ACH', 'coming', '66', 'excluded_path', 'Mar', '\\)/\\', 'Vr', '$#', 'texturedata', 'sunk', "*:'", "'.+\\.\\'", '20140', "'./.*'", 'Et', 'TRAIL', 'nTest', '".[^.]*"', 'b8', '"$@"\'', 'mountpoint', 'leGrep', 'ttt', 'command', 'PROJ_PATH', '].*?', "'./\\.*'", 'FIND', 'rn', '/^.*[/]\\[/]\\$/\\/', 'image', '/^$//;', '\\(\\.', 'nextfile', '],}*', 'invalidTemplate', 'katalogi', 'hashmap', 'Ale', 'xml', '_per', 'TZ', 'UN', 'HnT', "'*$", 'ries', '"\'\'"\'"\'"\'"\'\'"', 'param1', '":"")}\'', '--', 'EBF', 'junk3', 'funroll', 'group2', '16379', '[.][', 'IFE', '^#', 'comments', 'DS_STORE', 'bozo', 'iTMSTransporter', "'\\\\\\", 'umn', "'#!/", 'testuser', 'gpg', "')", '70x70', 'daily_backup', '36', '_access', '"];', 'wx', '*/|', 'ORS', 'source_dir', '.*:\\', 'sleep', '*$/', 'MASTER', ".\\*'", "???'", "#^./##'", 'sta', ']?[', 'anas', 'ия', 'myNewPathOnCurrentLocalMachine', 'arc', 'libexec', 'da', 'mid', 'FOO_NO_', ']/[', 'FILENAME', 'km', 'quanti', 'PYTHONPID', 'myarchive', 'mended', 'exportfiles', '..";', 'jan92', 'uv', 'ncode', 'PER', 'gnuplotscript', 's', 'tonics', 'machine', "'.*(", 'vboxusers', 'what_to_find', 'ffmpeg', ',$!', 'NUMCPU', 'ING_DIR', "'????????????????????????????????", 'Cust', 'An', '_html', '\\;/', '`]', '-$((', 'dll', 'lame', 'loop0', '4d', 'localtime', 'gned', 'cosx', '/]*', 'qO', 'testsite', '212', 'ice', 'script1', 'BackUp', '640', 'daymonth', 'bolic', '51213c1', 'nal', 'BCC', 'screen', 'PATH_to_SEARCH', '1i', 'choice', 'ace', '{.}', 'lopment', '<|', 'RBENV', 'd0000', 'Items', '/^[[:', 'some_data', '000000', 'with', 'my_partition', 'Na', 'already', 'erl', 'subdir3', '9', 'xftp', '30m', 'IDT', 'indows', 'trc', 'android', 'ael', 'WHERE', 'mysqldb', '-\\!', '/^/.', '\\\\?\\', '][^', 'aw', 'PACE', 'coreuti', '"|', 'theDir', 'query_to_avoid', 'tom', '766', '>\\<]', 'ondWord', 'LL', 'some_phrase', 'CR2', '_DIR', '`/*', 'USER_FOOBAR', 'draw', 'vcodec', '\\)"', 'BACK', 'Lin', 'folder_', 'avhe', 'tdggendska10', 'checkout', 'nccccc', '+[^', 'michel', 'case', "=[^=]*'", 'count', '_2', '`];', 'ing', 'lpi103', 'cksum', 'somecommand', "~$'", 'cketLo', ',\\\\', 'createdb', "'+%+:", 'hard', '*/.*"', 'lver1', 'JS', 'Ft', '\\$")', 'xmllint', 'HEAD', 'txt', 'extern', '\\&/', 'rave', 'foot', 'er_here', 'ipython', '8e2', '*([^', 'touch', '*/"', 'password', 'any_command_what_produces_relative_path_names', "=$($)'", 'Messages', 'Developer', 'system_split', 'antToFind', "'/[<>", '_half', 'input1', '24222', 'jq', '\\".*\\"', 'matlab', 'some', '3G', 'Users', 'Ser', '117672808', 'azZ', 'Ea', 'non_secret_file', 'NA', 'hazcheez', '&&', '\\"\\', '*\\([^', 'company1', 'pdftotext', '00ne', '/*\\.', 'function', '26c', "'\\.(", 'Program', 'ANSW', 'my_folder', 'lpi104', 'A2', '\\.(', 'luser', '10000c', 'lpicpio', '.]\\+\\)', 'GB', 'TotalSize', 'adow', 'sica', 'Xvf', 'hand', ',^\\*/', 'myHost', '80', 'enabled', '3be', '/^/"/', 'bus', '**/*', 'see', 'input2', 'mycd', '|.*/\\(.*\\)', 'Dd', '86400', '_question', 'somebash', 'kadots', '11767', 'masterfilelist', 'R_', 'REPL', 'iTunesArt', 'occur', 'stCh', 'SrchStr', 'ALIAS', 'bytes', 'shn', '2p', 'copy_it', '18583', 'a4', 'unchAgen', 'lines_per_file', 'targetDir', '/[:', '_World', ']`', 'First', '/([^', '=<', ',\\', "/@'|", 'umnOfNumber', 'world', '/{.[!.],}*', 'OLUTE', '/[^(]*(\\([^', "'\\'", 'restart_rails', 'hist', '/-', 'Pl', 'tmpnewfile', 'paths_to_archive', "]*//'", '_server', 'linen', '"\'', 'your_file', 'tication', 'cif', 'west', 'line', 'CC', 'grep', 'fault', 'Is', 'json_xs', 'mmin', 'Cam2', 'OMMAND', '&)', 'LE_OP', ';/^\\', '.*\\.\\./.*', 'Tm', 'acce', 'IPv6', 'ctrl', 'testfileasdf', '}${', 'kibab', "}'", 'pdfs', 'wget', 'option', 'P4', 'monfichi', ":'|", 'SCARG', 'leter', 'netinst', 'vz', '100c', 'NAME_KER', 'l40pe0', 'directories', 'orac', 'SELF', '200M', '_DE', 'raspberrypi', 'server_b', '37418', '/*/*/*/*', '=.*/\\', 'init', '_split', 'cfg', 'MD5SUM', 'PY', ']");', 'finger', 'hotcopy', 'polkadots', "$}'", 'bar_fn', 'FOO_NO_TRAIL_SPACE', 'pbzip2', '#-+-', 'ORACLE', '_dir2', ':~/', '"-"', 'TY', '/{}', 'mono', 'Xnoagent', 'default', 'find', '[@]/%', 'WD_Disk_1', 'rea', 'NE', '.]*', '100000', "'>", 'u20806', 'YOUR_PATH', '\\)/&', 'd_directory', 'existent', 'specific', 'apro', '==$', 'local_dir', 'phe', 'stefan', 'TMP', 'own', '45a8', '\\"${', 'iver', 'crashing', 'Size', 'jsmi', '365', 'LINES_PER_', 'имя_файла', 'Host', 'dl1', "'.+\\.\\(", '31467125', '/"/\\\\', ":]]')", 'df', 'ATION', '"././%', '/^[^/]*\\(.*\\)', 'seems', 'a8', "'%#", 'ty', 'dlrt', 'ен', 'remote_IP', 'stored_ex', 'canonicalize', 'reliable', 'RAW_', ';/;', '\\"${"', 'Path', '1E', 'user2', 'k7n', "%.*/%%'", 'che', 'what', ']}"`"', 'objects', 'youtube', 'DAIL', 'gc_maxlifetime', 'flac', 'dreftymac', '<>&*|$', '])"', '/...', '/^.\\{', '09', '5638', 'ec2', '185', '\'"\'', '/^/$', 'tmpdir', 'ENT_PID', 'ldd', 'erruptible', 'low_log', 'r_project_dir', '3', 'le', 'US', 'rTesting', '2775', 'AT_HOST', 'retty', '&\'"', 'stdlib', '---', '\\”,\\”%', '//};', '\\"}\'"', '))/../)', 'dep', '-\xad', 'tobeexclude', '_string', 'tponly', "'=", 'myFile', 'Tmp', "]\\+');", 'sm', 'real1', 'WOR', 'iv', 'x64', 'eric', "'!", 'oL', 'Waldo', '@/;', 'dirTo', '1Esc', 'YourPar', 'XSTATE', 'company', 'tree', 'ev', '&/\\.&/\\', '$"),', 'D_import_script_ABF1_6', 'cc', 'forge', 'somefiles', 'contain', 'CR', 'cert', 'someuser', 'remoted', 'aesthes', 'ustom', '.*/\\', 'newdirpathname', 'composer', '/")";', ']\\\\', 'TMP_DIR', 'Ev', '";}\'', 'testscript', 'xx', 'postg', 'execute', '/.*#\\', ')).*"', 'extra', '_SOURCE', "'*,", 'iostat', 'erday', "'.*/.", 'TMSG', 'ARED', 'controlchars', '30fps', 'edefg', 'capa', '415', 'remote_server_name', '].', 'wordC', '/?(?:.)*/', '~\')";', 'WORKST', 'heap', '\\}\\.', 'pingResul', 'urpose', 'u', '*/*/*/*', 'you_search_for_it', 'libEGL', 'L8888', 'utar', 'Ip', 'genja', 'з', '-(*', '/,,/,', 'hwaddr', '%.*}', 'pluginpath', 'ksknet', 'FilesDvorak', 'dra', '╲', '040', '/*/', '\\{\\}', 'pted_', '6579', 'gen_root_dir', 'add', 'WithoutSpa', "[^/]*$'", '0x', 'KE', 'Bar', 'xargste', '[!', '_pend', 'n1000', 'evaluate', '”*.', 'find_error', '/^', 'height', 'DIREC', '@^@\\', 'a862', 'pdv', 'quiet', 'old1', '/^../', 'nsanity', '/^.*', 'emon', '^.*', 'Include', 'stderr', 'the_stuff_you_test', 'Ima', 'ogv', 'todo', 'ER', "`'", 'oa', 'pswd', 'runInstaller', '41525360', '=\\|', 'leil', "`':\\", "@^.*/@@'", '\\;', 'fav', '2808', 'ips', 'ipcam', 'MOUNT_OPTS', 'me', 'hrtgdg53', 'ious', 'CH', 'weesa', 'mail', 'totext', 'October', 'tim', '"$@"', 'FIELD', 'Add', 'ilr', 'п', 'born', 'cpp', 'crashes', 'seq', 'dba_2pc', '/[^:]*:', '.*/#&/', ',"\\', 'GOPATH', 'ph', 'ps', 'ader', 'ess', '"{},\\!', 'jvm', '\\?', "++}'", "}'\\'''", 'saveIFS', 'xarg', '144', '22', 'EB', 'gzip', 'huge', 'ssl_eng', 'thepath', '_kerbero', 'pigz', '][', 'custom', '_replacement', 'cp', 'message_track', 'managers', 'dPro', 'our_repos', 'tardir', 'ae', '1888', 'matchNameHere', 'text2', 'cy', '\\|<"', 'sizes', '_link', '|.*/\\/.*|', 'IZ', '10minutes', 'GROUP', '/(.)$/$', 'HERE', "#.*/##'", ',*.', 'apple', '\\{}', 'vds', 'libGLE', '!(*', 'T_', ";'", 'totalLineCnt', 'username2', 'some_directory', 'MP', 'YOUR_DIR', '-', 'uale', 'exclude_me_too_anywhere', 'php', 'common_header', 'des', 'Zip', '\\:', 'PATTERN2', 'raL', 'oldstring', 'zenity', 'Exception', '/}";', 'aq', '=]\\+', 'CURRENT_PID_FROM_LOCKFILE', 'message', 'ogg', 'SYMLINK_PATH', 'domains', 'ABSOLUTE_PATH', 'QUI', '1551', 'fL', 'ser', '$))/', 'liwZ', 'VAR1', 'myvar', 'om', 'current', 'ite', 'to_dir', ':.\\', 'cation', 'tra', 'printto', 'am', 'ANET', '798', 'new_name', 'FOLDER2', 'deferred', 'filesOfInterest', 'ption', 'MYPROG', '96', 'tee', 'sh_lock', '_sed_script', 'iou', 'ce', '>(', 'YourPr', 'random', 'fghfilea', 'nbLines', '")")', 'zk', 'dsl', '"./', ')\\', 'MyDo', "-'", '/\\.\\/<', '(?=</', 'folder1', '.\\*\\.', 'iazZ', 'BZIP2_CMD', 'mylogfile', '}\\""', 'sso', 'detail', 'mygzfile', 'sf8h8hsd8f', 'zah', '_ME', 'stc', 'DE', 'al', 'selection1', 'ntfs', 'proc_load_average', 'proxy', 'somesymlink', 'orary', 'slc02oxm', '-([', 'ric', '420p', 'gz', 'logi', 'npmjs', 'ROOT', '";"', '2012', 'gurukul', '82', 'file_of_line_numbers', 'as', '"/[^/]{', '1def', 'TAR', '".\\', 'cape', 'vnc', 'SEARCH_DIR', 'AVI', 'subj', 'tes_to_stdout', 'RA', 's2', 'xe', 'numstat', '399', '20150105', '/*.', 'KST', 'revm', '#\\(.*\\)/.*#\\', '"}\'', ')]*)', '10000', ',|', 'assemb', '_average', '"???-??', 'sd', 'openssl', "'./\\", 'co', 'music', 'newcompany', 'rence1', 'GEFILE', 'THIS', 'list_2', 'jcvf', 'rfs', '---------------------------------------";}\'', 'utes', 'waiting', 'junk1', 'RBEN', '&\\""', 'PRO', 'cccc', 'incoming', 'varname', 'GEFOLDER', 'VARIABLE', 'cygpath', ':)?', 'MD5SUM_OP', '12s', 'oa0V', 'net', 'Lear', "/.*\\.//'", 'firefox', '>).*', 'my_ip', 'hdaX', '_name', '6000', "'/,|%", 'FILE_NAME', '1500', 'septe', 'link_name', 'Video', '_0', 'backup1', '".*[/]\\.', "'@", 'doc', '="-"}{', 'Volume', '7c8', 'ootp', '10010p', 'U0', 'NEW_PIDS', 'ST', "'^.*/\\.\\$'", '800x800', 'dummy', ']/%', 'wnj', 'json_decode', '”%', 'gitkey', 'DESTFOLDER', '*’', '180', 'bu', ':]]*$"', 'casso', 'Uv', 'icsearch', 'washere1', '67', 'HTM', '/.\\+\\', 'ange', '\\\\', 'Pi', 'newlink', 'developer', "'[/]'", 'ANSWER', 'gild', 'dir_output', 'ilation', 'tipng', 'that', 'parrent', 'type_descriptor', 'sfv', "'^$'", '|.*', 'medi', ']+[', 'my_directory', 'NUMB', '|.*/\\/.*|\\', '_spec', 'mnt_dir', 'ject', '/^\\(', 'nch', 'libbass', 'Client', '<>&', 'Ad', 'temp1', 'опция_действия', 'gt', '_archive', 'Book', '123', 'fnord', '"/"', '34', 'cscape', 'j2', 'lib', "/\\.\\///'", 'umber', '?.*?/[', 'dendien', 'Why', 'lates', 'VARIABLENAME', 'lah', 'ozuma', '=",', 'unexport', '/}$.', ']:', 'myName', '"/${', 'x1', 'MESSA', '0V', 'temp_file_', 'safe_del', 'app1', 'someName', 'MyScript', ':)?([', 'matchtext', 'filepattern', 'World', 'UAL_DIR', "'...'", ',|%', 'Sample_51770BL1_R1', 'bs', '"`/`', 'rectories', 'morelogs', 'F1pp', 'ionice', 'name2', 'RES', 'н', '‘*.*’', 'tiesOnly', 'CAD', 'ent', 'durchsuch', 'eb', '\\(.*\\)', 'rd', ',";"}\'', 'teststring', '">/', 'pw0', 'pg_', '2655341', '/:$//;', 'EUR', '"*,', '".*/\\"', 'file40000', 'ama', '=¨', 'etl', 'Mb', "'*", 'loca', 'by', '_KER', 'five', 'OOT', 'pon', '3473', 'Sh', '/{$', 'checkjobs', 'emacs', 'plit', "'.*/.{", ':]]{', '=/;', 'fifo_name', 'plist', 'find_graph', 'executable', 'currentDir', 'shift', 'zgrep', '/\\+.*/\\', 'ob', '2671', 'uniq', 'close', ':]]/{', 'ExtJS', 'ABSOLUTE', 'field', 'cdr', 'AL_FILE', 'outSpa', 'sparent', 'pemk', 'inject', '[^/.', 'SOCK', '"\\.?.*"', 'Modifying', 'ALE', 'Process', 'chrom', 'someFile', 'Base', 'inr', 'cuments', '_secret_file', '51200c', 'usercontent', '/,/,', 'sem', '=(', ":]]\\+$//'", 'MY_DIRECTORY', 'filedir', 'vfat', '"<', 'SN', 'photo', '43', 'pkglist', 'hometest', 'pps', '!.*/', 'stringWith', 'undef', 'zcpvf', 'A5', 'TO_IGN', 'localize', ':/{', 'eu', '/.*/\\', 'goutputs', 'then', 'E_', 'vhosts', '5000', '_f', 'qstat', 'user_file', '!=""&&', '",', 'ORA', '>/<', 'md', 'thumb', 'SOURCE_FILE', 'olduser', 'results', '`,', "'{}/*'", 'MO', 'unique', '_content', 'mn', '}").', 'unchanged', 'д', 'D1080', '/^/$:', '*/', 'progam', 'text_pattern', 'folder_list', 'Ge', '00000', "='*'", '^\\<', "-]*'", '1138', 'PR', '="[', 'any_command_', 'chine', 'Headers', '1000000', "]*'", 'lower', 'fd0', '300b', 'raptor', "$/'", '.*"', 'sam', '200', '*/*/*/*/*/*/*', 'LINES', 'TER', 'unity', 'oE', 'MYPROGRAMNAME_KER', 'Tu', 'x00', "=$'", '(\\|#', 'SUM', '#.*', 'ugo', '`/{', 'specified', '_0001', 'tify', '<>;', '.*', ']++;', 'another_pattern_to_exclude', 'checkwinsize', '(/[^', '06', 'usergroup', '**', 'goodbye', 'personal', '<=>', '/.*"', 'o', 'srvm', '.]*\\)/.*|\\', 'pix_fmt', 'MySearch', 'totals', 'file', 'type', '!/^\\//', '_you', 'u1', 'YOURFILE', '7', 'header', 'ecri', 'dereference', '$\\="\\', 'releasever', '".")', 'check_script_', '"./.*"', 'ared', 'Vol', '/((:[^:', '*', 'SD_', 'ame', '/[^:]*', 'OPEN', "/'/\\\\\\'/", 'newfile', ":'", 'nullmailer', 'dir1', '"\'"', 'TIMIZ', 'ald', '/.}.', 'repo', 'TestProject', '<)', "'.+\\-[", 'Ralph', '/.//', 'S1', '“*.', 'nv', 'Hello_World', '*-*', 'TestIP', 'file_to_', 'DEV_LEVEL', 'bapps', 'j8', 'jav', 'myprogram', 'set2', 'bur', 'uuid', 'burger', '/^\\.', 'olddir', 'asci', '##*/}', 'axy', '_line', 'packet', 'after', 'newer', 'red_ex', 'subdomainB', 'csv1', 'searchtext', 'bot', '\\\\[', '600', 'Ba', 'STAMP', 'E_PIDS', '_dev', 'whi', 'Zli', 'uuencode', "}'`;", '+.*/\\', 'ITE', 'Ee', 'hugefile', 'sys', 'bam2', 'php_user', '1278999698', 'idmv', '?', '-\\%', 'use', ']+=$', '="--', 'fl', 'stringWithOrWithoutSpa', 'basedir', 'ere', 'deleting', 'pyc', '00k', 'remo', '[^/]*$', 'applicationY', ":]]*$//'.", 'oldest_to_keep', '500MB', '_command', 'scripts', 'PEN', "'%.", '8f1db286', 'COUNTER', 'shell', 'xv', 'aut', 'distro', 'PN_C', 'dc1', 'Apr', "-.'", 'SD_custom', 'dnf', '2429', '"-----------------------------------------------------------------------";}\'', 'ED', 'control', '~"`;', 'alread', 'dName', 'asdf', '2G', 'UUID', 'KER', 'DESTINATION', 'k6', 'RELEAS', '/$.', 'sqlite3', '{$}{\\"', 'tmp', 'movflags', 'need', 'rence', 'GOBIN', 'Sin', 'fake', '022', '"^#"', 'sublime', '2960', '="--------', 'llvm', 'tput', 'uname', 'ITLIM', 'MD', 'MY', 'eY', '":', 'April', 'LOAD', 'reenshots', ']$"', 'SCRIPT', '/#\\~/$', 'vE', 'lando', 'newFirstWord', 'ate', 'Subscription', 'r_', 'ver', 'mind', '.)', 'localdomain', 'rtuv', 'subdirectory', '=]*', 'CONTAINER_ID', 'zcf', 'SI', 'base64_decode', '10s', '"]*"', 'bie', '\\)/.*;\\', 'sd8f', 'tJS', '_num', '"%%%%%', 'tion', 'fthub', 'ipath', 'vim0', '«', 'Xno', 'dir3', 'bash_aliases', 'PS', 'Hcm2', 'ordinary', 'ss', 'damian', 'ear', 'euid', '_rails', '(‘', 'collection', '~\\]*\\)/\\([^', ',:[^=]*=,:;,;', 'blowfish', 'wp', 'ax', 'eE', '&//', '198888', 'copies', '3339', 'x11', 'force', '<>', 'login', 'kado', '\\\\\\\\{', 'saved', '`', 'localhost', 'INPUT', '40s', 'x11vnc', 'addr', "}')|", 'remodup', ")')", 'stdo', '10lines', '201111', 'NOPORTDO', 'pix_', 'pamVd', 'ozu', 'rmdir', 'name_pattern', '8s', 'youhostname', 'JUKEBOX', 'dba_2pc_pending', 'caca', 'p48', '\\?.', '-"$', '_sd_customize', '://[^/]*/\\).*', 'filetypes', '21', '{$', '".*\\.$', "/:$//'", '_file', 'stored_exception', '.*\\?', 'postgres', 'say', '_admin', "'^$')", 'shared_group', 'rfd0', 'LOT_', '$$/', '01', '[.*', 'ection', 'book', 'IP', '698', 'Usr', 'ONBOOT', "'\\''...'\\'''", 'STRING_TO_SEARCH_FOR', 'ine_log', 'Password', '>.<', 'ward', 'th', '_phrase', 'lrx', 'Xrun', 'TD', "}'>./", 'nrg', '[]', 'vP', 'debian_chroot', 'ATUS', "='<", 'suffixes', 'dirlinks', 'myProcessName', 'RV', 'teste2', ']+"', 'rsync', 'false', 'target_directory', 'nln', 'views', '100q', '104', 'tomorrow', 'fic', 'architecture', 'ht', 'dvt', 'SPLITLIM', 'Expan', 'опция_поиска']